# Different Agentic Systems - Workflow

Xiaohui Chen

In [1]:
# Imports and Environment Setup
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

# Load environment variables
load_dotenv(override=True)

# Verify API keys are loaded
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

# Initialize clients
openai_client = OpenAI()
anthropic_client = Anthropic()


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


## 1. Prompt Chaining Design Pattern

### What is Prompt Chaining?

**Prompt Chaining** is an agentic workflow pattern where a complex task is decomposed into a sequence of smaller, focused subtasks. Each step in the chain:
- Takes input from the previous step's output
- Performs a specific, well-defined operation
- Produces structured output for the next step

This approach offers several advantages over single-prompt solutions:
- **Improved accuracy**: Each LLM call focuses on one thing, reducing cognitive load
- **Better debuggability**: You can inspect intermediate outputs to identify where things go wrong
- **Modularity**: Individual steps can be modified or improved independently
- **Quality gates**: You can add validation or human review between steps

### Example Workflow Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         PROMPT CHAINING WORKFLOW                            │
└─────────────────────────────────────────────────────────────────────────────┘

     ┌──────────┐      ┌──────────┐      ┌──────────┐      ┌──────────┐
     │          │      │          │      │          │      │          │
     │  INPUT   │ ───▶ │  STEP 1  │ ───▶ │  STEP 2  │ ───▶ │  STEP 3  │ ───▶ OUTPUT
     │          │      │          │      │          │      │          │
     └──────────┘      └──────────┘      └──────────┘      └──────────┘
                            │                 │                 │
                            ▼                 ▼                 ▼
                       Intermediate      Intermediate      Intermediate
                        Output 1          Output 2          Output 3
```

### AG News Classification Chain

For our experiment, we'll classify news articles using this 3-step chain:

```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│                 │     │                 │     │                 │
│  STEP 1:        │     │  STEP 2:        │     │  STEP 3:        │
│  Extract        │ ──▶ │  Analyze        │ ──▶ │  Classify       │
│  Entities &     │     │  Domain         │     │  Final          │
│  Keywords       │     │  Signals        │     │  Category       │
│                 │     │                 │     │                 │
└─────────────────┘     └─────────────────┘     └─────────────────┘
        │                       │                       │
        ▼                       ▼                       ▼
  Named entities,         Domain indicators      World | Sports |
  key topics,             and reasoning          Business | Sci/Tech
  geographic refs
```

**Categories in AG News:**
- 🌍 **World** - International news and events
- ⚽ **Sports** - Athletic events, teams, players
- 💼 **Business** - Companies, markets, economy
- 🔬 **Sci/Tech** - Science and technology news


In [5]:
# Load the AG News dataset from HuggingFace
from datasets import load_dataset

# Load the dataset
ag_news = load_dataset("ag_news")

# Display dataset structure
print("=" * 60)
print("AG NEWS DATASET ANALYSIS")
print("=" * 60)

# Dataset splits
print("\n📊 Dataset Splits:")
for split_name, split_data in ag_news.items():
    print(f"  - {split_name}: {len(split_data):,} rows")

# Total rows
total_rows = sum(len(split) for split in ag_news.values())
print(f"\n📈 Total rows: {total_rows:,}")

# Column information
print("\n📋 Columns:")
print(f"  - Number of columns: {len(ag_news['train'].features)}")
for col_name, col_type in ag_news['train'].features.items():
    print(f"  - '{col_name}': {col_type}")

# Label mapping
print("\n🏷️ Label Column: 'label'")
print("  This is the target column for classification!")
print("\n  Label Mapping:")
label_names = ag_news['train'].features['label'].names
for idx, name in enumerate(label_names):
    print(f"    {idx} → {name}")

# Sample data preview
print("\n📝 Sample Data (first 3 rows from train):")
for i in range(3):
    sample = ag_news['train'][i]
    print(f"\n  [{i}] Label: {sample['label']} ({label_names[sample['label']]})")
    print(f"      Text: {sample['text'][:100]}...")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

AG NEWS DATASET ANALYSIS

📊 Dataset Splits:
  - train: 120,000 rows
  - test: 7,600 rows

📈 Total rows: 127,600

📋 Columns:
  - Number of columns: 2
  - 'text': Value('string')
  - 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'])

🏷️ Label Column: 'label'
  This is the target column for classification!

  Label Mapping:
    0 → World
    1 → Sports
    2 → Business
    3 → Sci/Tech

📝 Sample Data (first 3 rows from train):

  [0] Label: 2 (Business)
      Text: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\b...

  [1] Label: 2 (Business)
      Text: Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,...

  [2] Label: 2 (Business)
      Text: Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about th...


### Step 1: Entity & Keyword Extraction

> **Note:** We will use only the **test set** (7,600 rows) for our experiments, ignoring the training data.

**Objective:** Extract structured information from raw news text that will help downstream steps make better classification decisions.

In this step, the LLM will analyze each news article and extract:
- **News Source**: The wire service or publication (e.g., Reuters, AP)
- **Named Entities**: People, organizations, teams, companies mentioned
- **Key Topics**: Main subjects and themes discussed (2-4 topics)
- **Geographic References**: Countries, cities, regions mentioned
- **Domain-Specific Keywords**: Significant terms that signal the article's domain (5-10 keywords)

**Why this step matters:**
- Converts unstructured text into structured, analyzable components
- Identifies signals that are strongly associated with specific categories
- Reduces noise by focusing on the most informative elements
- Creates a foundation for the domain analysis in Step 2

---

#### Understanding the AG News Text Format

The `text` field in AG News follows a specific structure:

```
[Headline] ([Source]) [Source] - [Article Body]...
```

**Example:**
```
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling...
│                                        │         │         │
└── Headline ────────────────────────────┘         │         └── Article Body
                                          └── Source (repeated)
```

This structure means:
- The **headline** comes first and summarizes the story
- The **source** appears twice (in parentheses and before the dash)
- The **article body** follows after the dash

---

#### System Prompt for Step 1

The system prompt instructs the LLM on its role and expected output format:

```
You are an expert information extraction system specialized in analyzing news articles.

Your task is to extract key information from a news article that will help classify it into one of four categories: World, Sports, Business, or Sci/Tech.

The input text follows this format: [Headline] ([Source]) [Source] - [Article Body]

Extract the following information and return it as a JSON object:

1. "source": The news source/wire service (e.g., "Reuters", "AP", "AFP")
2. "headline": The article headline (text before the source in parentheses)
3. "entities": A list of named entities (people, organizations, teams, companies)
4. "topics": A list of 2-4 main topics or themes discussed
5. "locations": A list of geographic references (countries, cities, regions)
6. "keywords": A list of 5-10 domain-specific or significant keywords
7. "summary": A one-sentence summary of the article's main point

Rules:
- Be concise and precise
- Only extract information that is explicitly stated or clearly implied
- Return ONLY valid JSON, no additional text or explanation
- If a field has no relevant items, return an empty list []

Example output format:
{
    "source": "Reuters",
    "headline": "Wall St. Bears Claw Back Into the Black",
    "entities": ["Wall Street", "short-sellers"],
    "topics": ["stock market", "trading"],
    "locations": ["United States"],
    "keywords": ["stocks", "bears", "rally", "markets", "trading", "investors"],
    "summary": "Short-sellers on Wall Street recover losses as market conditions shift."
}
```

---

#### Converting Raw Data to User Prompts

Each news article from the test dataset needs to be formatted as a user message for the LLM.

**Raw Data Format (from AG News test set):**
```python
{
    "text": "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling...",
    "label": 2  # Ground truth: Business (used for evaluation, not shown to LLM)
}
```

**User Prompt Format:**
```
Analyze the following news article and extract the requested information:

---
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling...
---
```

**Conversion Function Pattern:**
```python
def create_step1_user_prompt(article_text: str) -> str:
    return f"""Analyze the following news article and extract the requested information:

---
{article_text}
---"""
```

This wrapper:
1. Provides clear instruction on what to do
2. Delimits the article text with markers (`---`) for clarity
3. Keeps the original text intact (including the headline+source+body structure)


In [ ]:
# ============================================================
# STEP 1: Entity & Keyword Extraction
# ============================================================

# System prompt for Step 1
STEP1_SYSTEM_PROMPT = """You are an expert information extraction system specialized in analyzing news articles.

Your task is to extract key information from a news article that will help classify it into one of four categories: World, Sports, Business, or Sci/Tech.

The input text follows this format: [Headline] ([Source]) [Source] - [Article Body]

Extract the following information and return it as a JSON object:

1. "source": The news source/wire service (e.g., "Reuters", "AP", "AFP")
2. "headline": The article headline (text before the source in parentheses)
3. "entities": A list of named entities (people, organizations, teams, companies)
4. "topics": A list of 2-4 main topics or themes discussed
5. "locations": A list of geographic references (countries, cities, regions)
6. "keywords": A list of 5-10 domain-specific or significant keywords
7. "summary": A one-sentence summary of the article's main point

Rules:
- Be concise and precise
- Only extract information that is explicitly stated or clearly implied
- Return ONLY valid JSON, no additional text or explanation
- If a field has no relevant items, return an empty list []

Example output format:
{
    "source": "Reuters",
    "headline": "Wall St. Bears Claw Back Into the Black",
    "entities": ["Wall Street", "short-sellers"],
    "topics": ["stock market", "trading"],
    "locations": ["United States"],
    "keywords": ["stocks", "bears", "rally", "markets", "trading", "investors"],
    "summary": "Short-sellers on Wall Street recover losses as market conditions shift."
}"""

def create_step1_user_prompt(article_text: str) -> str:
    """Convert raw article text to Step 1 user prompt."""
    return f"""Analyze the following news article and extract the requested information:

---
{article_text}
---"""

def run_step1_extraction(article_text: str) -> dict:
    """Run Step 1 extraction using Claude Sonnet 4."""
    user_prompt = create_step1_user_prompt(article_text)
    
    # Expected output is ~300-500 tokens (JSON with 7 fields)
    # Setting to 2048 for safety margin on longer articles
    response = anthropic_client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2048,
        system=STEP1_SYSTEM_PROMPT,
        messages=[
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # Parse the JSON response
    response_text = response.content[0].text
    try:
        extracted_data = json.loads(response_text)
    except json.JSONDecodeError:
        # If parsing fails, return the raw text for debugging
        extracted_data = {"error": "Failed to parse JSON", "raw_response": response_text}
    
    return extracted_data

# ============================================================
# Process Test Data (100 samples) - SEQUENTIAL EXECUTION
# ============================================================

# Get the test dataset
test_data = ag_news['test']
label_names = ag_news['test'].features['label'].names

# Sample size for experiment (using subset for faster iteration)
NUM_SAMPLES = 100  # Using 100 samples (full test set has 7,600 rows)

# Number of results to print (for display)
NUM_TO_PRINT = 3

# Store results in order (index aligned with test_data)
step1_results = []

print("=" * 70)
print("STEP 1: Entity & Keyword Extraction")
print("=" * 70)
print(f"\nProcessing {NUM_SAMPLES} samples from test set using Claude Sonnet 4.5...\n")

# Progress tracking
PROGRESS_INTERVAL = 10

for i in range(NUM_SAMPLES):
    sample = test_data[i]
    article_text = sample['text']
    ground_truth_label = sample['label']
    
    # Run Step 1 extraction
    extraction_result = run_step1_extraction(article_text)
    
    # Store result with metadata
    step1_results.append({
        "index": i,
        "original_text": article_text,
        "ground_truth_label": ground_truth_label,
        "ground_truth_name": label_names[ground_truth_label],
        "step1_extraction": extraction_result
    })
    
    # Print progress every PROGRESS_INTERVAL articles
    if (i + 1) % PROGRESS_INTERVAL == 0 or (i + 1) == NUM_SAMPLES:
        print(f"  ✓ Processed {i + 1}/{NUM_SAMPLES} articles ({(i + 1) / NUM_SAMPLES * 100:.1f}%)")

print("=" * 70)
print(f"STEP 1 RESULTS (showing first {NUM_TO_PRINT} of {len(step1_results)})")
print("=" * 70)

# Display results for first NUM_TO_PRINT articles only
for result in step1_results[:NUM_TO_PRINT]:
    print(f"\n{'─' * 70}")
    print(f"Article {result['index'] + 1}")
    print(f"Ground Truth: {result['ground_truth_name']}")
    print(f"{'─' * 70}")
    print("\nExtracted Information:")
    print(json.dumps(result['step1_extraction'], indent=2))

print(f"\n{'=' * 70}")
print(f"Step 1 complete. {len(step1_results)} results stored in 'step1_results'")
print(f"Ready for Step 2.")
print(f"{'=' * 70}")


STEP 1: Entity & Keyword Extraction

Processing 100 samples from test set using Claude Sonnet 4.5...

  ✓ Processed 10/100 articles (10.0%)
  ✓ Processed 20/100 articles (20.0%)
  ✓ Processed 30/100 articles (30.0%)
  ✓ Processed 40/100 articles (40.0%)
  ✓ Processed 50/100 articles (50.0%)
  ✓ Processed 60/100 articles (60.0%)
  ✓ Processed 70/100 articles (70.0%)
  ✓ Processed 80/100 articles (80.0%)
  ✓ Processed 90/100 articles (90.0%)
  ✓ Processed 100/100 articles (100.0%)
STEP 1 RESULTS (showing first 3 of 100)

──────────────────────────────────────────────────────────────────────
Article 1
Ground Truth: Business
──────────────────────────────────────────────────────────────────────

Extracted Information:
{
  "source": "",
  "headline": "Fears for T N pension after talks",
  "entities": [
    "Turner Newall",
    "Federal Mogul",
    "Unions"
  ],
  "topics": [
    "pension concerns",
    "labor negotiations",
    "corporate restructuring"
  ],
  "locations": [],
  "keywords":

### Step 2: Domain Signal Analysis

**Objective:** Analyze the extracted information from Step 1 to identify domain-specific signals and reasoning that point towards each of the four categories.

In this step, the LLM will:
- **Analyze entities** for domain associations (e.g., "Federal Reserve" → Business, "FIFA" → Sports)
- **Evaluate topics** for category alignment
- **Consider geographic context** (international locations often signal World news)
- **Examine keywords** for domain-specific terminology
- **Provide reasoning** for why certain signals suggest specific categories

**Why this step matters:**
- Bridges the gap between raw extraction and final classification
- Makes the classification reasoning explicit and inspectable
- Allows the model to weigh different signals before making a final decision
- Creates an audit trail of the decision-making process

---

#### How Step 2 Works

Step 2 receives the **structured output from Step 1** and analyzes it to produce domain signals:

```
Step 1 Output (Input to Step 2)          Step 2 Output
┌─────────────────────────────┐          ┌─────────────────────────────┐
│ {                           │          │ {                           │
│   "source": "Reuters",      │          │   "domain_signals": {       │
│   "headline": "...",        │   ───▶   │     "world": [...],         │
│   "entities": [...],        │          │     "sports": [...],        │
│   "topics": [...],          │          │     "business": [...],      │
│   "locations": [...],       │          │     "sci_tech": [...]       │
│   "keywords": [...],        │          │   },                        │
│   "summary": "..."          │          │   "strongest_signals": "...",│
│ }                           │          │   "reasoning": "..."        │
└─────────────────────────────┘          └─────────────────────────────┘
```

---

#### System Prompt for Step 2

```
You are an expert news analyst specializing in categorizing news articles into four categories: World, Sports, Business, and Sci/Tech.

You will receive extracted information from a news article including:
- Source (news agency)
- Headline
- Named entities (people, organizations, companies)
- Topics discussed
- Geographic locations mentioned
- Keywords
- Summary

Your task is to analyze this information and identify signals that suggest which category the article belongs to.

For each category, identify relevant signals from the extracted data:
- **World**: International affairs, politics, government, diplomacy, conflicts, humanitarian issues
- **Sports**: Athletic events, teams, players, competitions, leagues, scores, tournaments
- **Business**: Companies, markets, economy, finance, stocks, trade, corporate news
- **Sci/Tech**: Science, technology, software, hardware, research, innovation, digital products

Return your analysis as a JSON object with:

1. "domain_signals": An object with four keys (world, sports, business, sci_tech), each containing a list of relevant signals found
2. "signal_strength": An object rating the strength of signals for each category as "none", "weak", "moderate", or "strong"
3. "strongest_category": The category with the strongest signals
4. "reasoning": A brief explanation (1-2 sentences) of why the strongest category is most likely

Rules:
- Be objective and base your analysis only on the provided information
- A signal can only support ONE category (don't double-count)
- Return ONLY valid JSON, no additional text
- If no signals exist for a category, use an empty list []

Example output format:
{
    "domain_signals": {
        "world": ["United Nations mentioned", "diplomatic talks topic"],
        "sports": [],
        "business": ["stock market keywords"],
        "sci_tech": []
    },
    "signal_strength": {
        "world": "strong",
        "sports": "none",
        "business": "weak",
        "sci_tech": "none"
    },
    "strongest_category": "world",
    "reasoning": "The article discusses UN diplomatic negotiations, with strong international affairs signals and only peripheral business context."
}
```

---

#### Converting Step 1 Output to Step 2 User Prompts

Each Step 1 result contains the `step1_extraction` field with the extracted JSON. This needs to be formatted as input for Step 2.

**Step 1 Output (stored in `step1_results`):**
```python
{
    "index": 0,
    "original_text": "...",
    "ground_truth_label": 2,
    "ground_truth_name": "Business",
    "step1_extraction": {
        "source": "Reuters",
        "headline": "Wall St. Bears Claw Back Into the Black",
        "entities": ["Wall Street", "short-sellers"],
        "topics": ["stock market", "trading"],
        "locations": ["United States"],
        "keywords": ["stocks", "bears", "rally", "markets"],
        "summary": "Short-sellers recover losses as market conditions shift."
    }
}
```

**Step 2 User Prompt Format:**
```
Analyze the following extracted information from a news article and identify domain signals for classification:

---
{
    "source": "Reuters",
    "headline": "Wall St. Bears Claw Back Into the Black",
    "entities": ["Wall Street", "short-sellers"],
    "topics": ["stock market", "trading"],
    "locations": ["United States"],
    "keywords": ["stocks", "bears", "rally", "markets"],
    "summary": "Short-sellers recover losses as market conditions shift."
}
---
```

**Conversion Function Pattern:**
```python
def create_step2_user_prompt(step1_extraction: dict) -> str:
    extraction_json = json.dumps(step1_extraction, indent=2)
    return f"""Analyze the following extracted information from a news article and identify domain signals for classification:

---
{extraction_json}
---"""
```

This conversion:
1. Takes the `step1_extraction` dictionary from Step 1 results
2. Serializes it to formatted JSON for readability
3. Wraps it with clear instructions and delimiters


In [11]:
# ============================================================
# STEP 2: Domain Signal Analysis
# ============================================================

# System prompt for Step 2
STEP2_SYSTEM_PROMPT = """You are an expert news analyst specializing in categorizing news articles into four categories: World, Sports, Business, and Sci/Tech.

You will receive extracted information from a news article including:
- Source (news agency)
- Headline
- Named entities (people, organizations, companies)
- Topics discussed
- Geographic locations mentioned
- Keywords
- Summary

Your task is to analyze this information and identify signals that suggest which category the article belongs to.

For each category, identify relevant signals from the extracted data:
- **World**: International affairs, politics, government, diplomacy, conflicts, humanitarian issues
- **Sports**: Athletic events, teams, players, competitions, leagues, scores, tournaments
- **Business**: Companies, markets, economy, finance, stocks, trade, corporate news
- **Sci/Tech**: Science, technology, software, hardware, research, innovation, digital products

Return your analysis as a JSON object with:

1. "domain_signals": An object with four keys (world, sports, business, sci_tech), each containing a list of relevant signals found
2. "signal_strength": An object rating the strength of signals for each category as "none", "weak", "moderate", or "strong"
3. "strongest_category": The category with the strongest signals
4. "reasoning": A brief explanation (1-2 sentences) of why the strongest category is most likely

Rules:
- Be objective and base your analysis only on the provided information
- A signal can only support ONE category (don't double-count)
- Return ONLY valid JSON, no additional text
- If no signals exist for a category, use an empty list []

Example output format:
{
    "domain_signals": {
        "world": ["United Nations mentioned", "diplomatic talks topic"],
        "sports": [],
        "business": ["stock market keywords"],
        "sci_tech": []
    },
    "signal_strength": {
        "world": "strong",
        "sports": "none",
        "business": "weak",
        "sci_tech": "none"
    },
    "strongest_category": "world",
    "reasoning": "The article discusses UN diplomatic negotiations, with strong international affairs signals and only peripheral business context."
}"""

def create_step2_user_prompt(step1_extraction: dict) -> str:
    """Convert Step 1 extraction output to Step 2 user prompt."""
    extraction_json = json.dumps(step1_extraction, indent=2)
    return f"""Analyze the following extracted information from a news article and identify domain signals for classification:

---
{extraction_json}
---"""

def run_step2_analysis(step1_extraction: dict) -> dict:
    """Run Step 2 domain signal analysis using GPT-5.1."""
    user_prompt = create_step2_user_prompt(step1_extraction)
    
    # Note: Newer OpenAI models use 'max_completion_tokens' instead of 'max_tokens'
    response = openai_client.chat.completions.create(
        model="gpt-5.1",
        max_completion_tokens=2048,
        messages=[
            {"role": "system", "content": STEP2_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # Parse the JSON response
    response_text = response.choices[0].message.content
    try:
        analysis_result = json.loads(response_text)
    except json.JSONDecodeError:
        # If parsing fails, return the raw text for debugging
        analysis_result = {"error": "Failed to parse JSON", "raw_response": response_text}
    
    return analysis_result

# ============================================================
# Process All Step 1 Results
# ============================================================

# Store Step 2 results (same order as step1_results)
step2_results = []

# Number of results to print (for display)
NUM_TO_PRINT = 3

print("=" * 70)
print("STEP 2: Domain Signal Analysis")
print("=" * 70)
print(f"\nProcessing {len(step1_results)} samples using GPT-5.1...\n")

# Progress tracking
PROGRESS_INTERVAL = 10

for i, step1_result in enumerate(step1_results):
    # Get the Step 1 extraction output
    step1_extraction = step1_result['step1_extraction']
    
    # Run Step 2 analysis
    analysis_result = run_step2_analysis(step1_extraction)
    
    # Store result with metadata (carry forward from Step 1)
    step2_results.append({
        "index": step1_result['index'],
        "original_text": step1_result['original_text'],
        "ground_truth_label": step1_result['ground_truth_label'],
        "ground_truth_name": step1_result['ground_truth_name'],
        "step1_extraction": step1_extraction,
        "step2_analysis": analysis_result
    })
    
    # Print progress every PROGRESS_INTERVAL articles
    if (i + 1) % PROGRESS_INTERVAL == 0 or (i + 1) == len(step1_results):
        print(f"  ✓ Processed {i + 1}/{len(step1_results)} articles ({(i + 1) / len(step1_results) * 100:.1f}%)")

print("=" * 70)
print(f"STEP 2 RESULTS (showing first {NUM_TO_PRINT} of {len(step2_results)})")
print("=" * 70)

# Display results for first NUM_TO_PRINT articles only
for result in step2_results[:NUM_TO_PRINT]:
    print(f"\n{'─' * 70}")
    print(f"Article {result['index'] + 1}")
    print(f"Ground Truth: {result['ground_truth_name']}")
    print(f"{'─' * 70}")
    print("\nStep 2 Analysis (Domain Signals):")
    print(json.dumps(result['step2_analysis'], indent=2))

print(f"\n{'=' * 70}")
print(f"Step 2 complete. {len(step2_results)} results stored in 'step2_results'")
print(f"Ready for Step 3.")
print(f"{'=' * 70}")


STEP 2: Domain Signal Analysis

Processing 100 samples using GPT-5.1...

  ✓ Processed 10/100 articles (10.0%)
  ✓ Processed 20/100 articles (20.0%)
  ✓ Processed 30/100 articles (30.0%)
  ✓ Processed 40/100 articles (40.0%)
  ✓ Processed 50/100 articles (50.0%)
  ✓ Processed 60/100 articles (60.0%)
  ✓ Processed 70/100 articles (70.0%)
  ✓ Processed 80/100 articles (80.0%)
  ✓ Processed 90/100 articles (90.0%)
  ✓ Processed 100/100 articles (100.0%)
STEP 2 RESULTS (showing first 3 of 100)

──────────────────────────────────────────────────────────────────────
Article 1
Ground Truth: Business
──────────────────────────────────────────────────────────────────────

Step 2 Analysis (Domain Signals):
{
  "domain_signals": {
    "world": [],
    "sports": [],
    "business": [
      "Corporate entities mentioned: Turner Newall and its parent firm Federal Mogul",
      "Topics include pension concerns linked to a company and corporate restructuring",
      "Keywords such as \"parent firm\" a

### Step 3: Final Classification

**Objective:** Make the final classification decision based on the domain signal analysis from Step 2, outputting one of the four categories: World, Sports, Business, or Sci/Tech.

This is the final step in our prompt chain where all the accumulated analysis converges into a single classification decision.

In this step, the LLM will:
- **Review the domain signals** identified for each category
- **Consider signal strengths** (none, weak, moderate, strong)
- **Evaluate the reasoning** from Step 2
- **Make the final decision** with confidence level
- **Provide justification** for the classification

**Why this step matters:**
- Synthesizes all previous analysis into an actionable output
- Adds a layer of deliberation before final decision
- Provides confidence scores for downstream use
- Creates complete traceability from raw text → extraction → signals → classification

---

#### How Step 3 Works

Step 3 receives the **domain signal analysis from Step 2** and produces the final classification:

```
Step 2 Output (Input to Step 3)          Step 3 Output (Final)
┌─────────────────────────────┐          ┌─────────────────────────────┐
│ {                           │          │ {                           │
│   "domain_signals": {...},  │          │   "classification": "...",  │
│   "signal_strength": {...}, │   ───▶   │   "confidence": "...",      │
│   "strongest_category": ".",│          │   "justification": "..."    │
│   "reasoning": "..."        │          │ }                           │
│ }                           │          │                             │
└─────────────────────────────┘          └─────────────────────────────┘
```

---

#### System Prompt for Step 3

```
You are a news classification system that makes final categorization decisions.

You will receive a domain signal analysis for a news article, containing:
- Domain signals identified for each category (World, Sports, Business, Sci/Tech)
- Signal strength ratings for each category
- The strongest category identified
- Reasoning from the analysis

Your task is to make the FINAL classification decision.

The four categories are:
- **World**: International news, politics, government, diplomacy, conflicts, global events
- **Sports**: Athletic competitions, teams, players, leagues, tournaments, scores
- **Business**: Companies, markets, finance, economy, stocks, corporate news, trade
- **Sci/Tech**: Technology, science, software, hardware, research, innovation, internet

Return your decision as a JSON object with:

1. "classification": The final category (must be exactly one of: "World", "Sports", "Business", "Sci/Tech")
2. "confidence": Your confidence level ("high", "medium", or "low")
3. "justification": A brief explanation (1-2 sentences) of why you chose this category

Rules:
- You MUST choose exactly ONE category
- Base your decision on the signal analysis provided
- If signals are ambiguous, use the strongest_category as a tiebreaker
- Return ONLY valid JSON, no additional text

Example output format:
{
    "classification": "Business",
    "confidence": "high",
    "justification": "Strong business signals including stock market keywords and corporate entities, with no competing signals from other categories."
}
```

---

#### Converting Step 2 Output to Step 3 User Prompts

Each Step 2 result contains the `step2_analysis` field with the domain signal analysis. This needs to be formatted as input for Step 3.

**Step 2 Output (stored in `step2_results`):**
```python
{
    "index": 0,
    "original_text": "...",
    "ground_truth_label": 2,
    "ground_truth_name": "Business",
    "step1_extraction": {...},
    "step2_analysis": {
        "domain_signals": {
            "world": [],
            "sports": [],
            "business": ["Wall Street entity", "stock market topic", "trading keywords"],
            "sci_tech": []
        },
        "signal_strength": {
            "world": "none",
            "sports": "none",
            "business": "strong",
            "sci_tech": "none"
        },
        "strongest_category": "business",
        "reasoning": "Clear business signals with stock market and trading terminology."
    }
}
```

**Step 3 User Prompt Format:**
```
Based on the following domain signal analysis, make the final classification decision:

---
{
    "domain_signals": {
        "world": [],
        "sports": [],
        "business": ["Wall Street entity", "stock market topic", "trading keywords"],
        "sci_tech": []
    },
    "signal_strength": {
        "world": "none",
        "sports": "none",
        "business": "strong",
        "sci_tech": "none"
    },
    "strongest_category": "business",
    "reasoning": "Clear business signals with stock market and trading terminology."
}
---
```

**Conversion Function Pattern:**
```python
def create_step3_user_prompt(step2_analysis: dict) -> str:
    analysis_json = json.dumps(step2_analysis, indent=2)
    return f"""Based on the following domain signal analysis, make the final classification decision:

---
{analysis_json}
---"""
```

This conversion:
1. Takes the `step2_analysis` dictionary from Step 2 results
2. Serializes it to formatted JSON
3. Wraps it with clear instruction to make the final decision

---

#### Evaluation

After Step 3, we can compare the `classification` output against the `ground_truth_name` to calculate accuracy:

```python
# Predicted: step3_result['step3_classification']['classification']
# Actual:    step3_result['ground_truth_name']
```


In [14]:
# ============================================================
# STEP 3: Final Classification
# ============================================================

# System prompt for Step 3
STEP3_SYSTEM_PROMPT = """You are a news classification system that makes final categorization decisions.

You will receive a domain signal analysis for a news article, containing:
- Domain signals identified for each category (World, Sports, Business, Sci/Tech)
- Signal strength ratings for each category
- The strongest category identified
- Reasoning from the analysis

Your task is to make the FINAL classification decision.

The four categories are:
- **World**: International news, politics, government, diplomacy, conflicts, global events
- **Sports**: Athletic competitions, teams, players, leagues, tournaments, scores
- **Business**: Companies, markets, finance, economy, stocks, corporate news, trade
- **Sci/Tech**: Technology, science, software, hardware, research, innovation, internet

Return your decision as a JSON object with:

1. "classification": The final category (must be exactly one of: "World", "Sports", "Business", "Sci/Tech")
2. "confidence": Your confidence level ("high", "medium", or "low")
3. "justification": A brief explanation (1-2 sentences) of why you chose this category

Rules:
- You MUST choose exactly ONE category
- Base your decision on the signal analysis provided
- If signals are ambiguous, use the strongest_category as a tiebreaker
- Return ONLY valid JSON, no additional text

Example output format:
{
    "classification": "Business",
    "confidence": "high",
    "justification": "Strong business signals including stock market keywords and corporate entities, with no competing signals from other categories."
}"""

def create_step3_user_prompt(step2_analysis: dict) -> str:
    """Convert Step 2 analysis output to Step 3 user prompt."""
    analysis_json = json.dumps(step2_analysis, indent=2)
    return f"""Based on the following domain signal analysis, make the final classification decision:

---
{analysis_json}
---"""

def run_step3_classification(step2_analysis: dict) -> dict:
    """Run Step 3 final classification using Claude Haiku 4.5."""
    user_prompt = create_step3_user_prompt(step2_analysis)
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=1024,
        system=STEP3_SYSTEM_PROMPT,
        messages=[
            {"role": "user", "content": user_prompt}
        ]
    )
    
    # Parse the JSON response
    response_text = response.content[0].text
    
    # Strip markdown code blocks if present (```json ... ``` or ``` ... ```)
    cleaned_text = response_text.strip()
    if cleaned_text.startswith("```"):
        # Remove opening ``` or ```json
        lines = cleaned_text.split("\n")
        # Remove first line (```json or ```)
        lines = lines[1:]
        # Remove last line (```)
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        cleaned_text = "\n".join(lines)
    
    try:
        classification_result = json.loads(cleaned_text)
    except json.JSONDecodeError:
        # If parsing fails, return the raw text for debugging
        classification_result = {"error": "Failed to parse JSON", "raw_response": response_text}
    
    return classification_result

# ============================================================
# Process All Step 2 Results
# ============================================================

# Store Step 3 results (same order as step2_results)
step3_results = []

# Number of results to print (for display)
NUM_TO_PRINT = 3

print("=" * 70)
print("STEP 3: Final Classification")
print("=" * 70)
print(f"\nProcessing {len(step2_results)} samples using Claude Haiku 4.5...\n")

# Progress tracking
PROGRESS_INTERVAL = 10

for i, step2_result in enumerate(step2_results):
    # Get the Step 2 analysis output
    step2_analysis = step2_result['step2_analysis']
    
    # Run Step 3 classification
    classification_result = run_step3_classification(step2_analysis)
    
    # Store result with all metadata (carry forward from previous steps)
    step3_results.append({
        "index": step2_result['index'],
        "original_text": step2_result['original_text'],
        "ground_truth_label": step2_result['ground_truth_label'],
        "ground_truth_name": step2_result['ground_truth_name'],
        "step1_extraction": step2_result['step1_extraction'],
        "step2_analysis": step2_analysis,
        "step3_classification": classification_result
    })
    
    # Print progress every PROGRESS_INTERVAL articles
    if (i + 1) % PROGRESS_INTERVAL == 0 or (i + 1) == len(step2_results):
        print(f"  ✓ Processed {i + 1}/{len(step2_results)} articles ({(i + 1) / len(step2_results) * 100:.1f}%)")

print("=" * 70)
print(f"STEP 3 RESULTS (showing first {NUM_TO_PRINT} of {len(step3_results)})")
print("=" * 70)

# Display results for first NUM_TO_PRINT articles only
for result in step3_results[:NUM_TO_PRINT]:
    print(f"\n{'─' * 70}")
    print(f"Article {result['index'] + 1}")
    print(f"Ground Truth: {result['ground_truth_name']}")
    print(f"{'─' * 70}")
    print("\nStep 3 Classification (Final Decision):")
    print(json.dumps(result['step3_classification'], indent=2))
    
    # Quick comparison
    predicted = result['step3_classification'].get('classification', 'N/A')
    actual = result['ground_truth_name']
    match = "✓ CORRECT" if predicted == actual else "✗ INCORRECT"
    print(f"\n  Predicted: {predicted} | Actual: {actual} | {match}")

print(f"\n{'=' * 70}")
print(f"Step 3 complete. {len(step3_results)} results stored in 'step3_results'")
print(f"Ready for evaluation.")
print(f"{'=' * 70}")


STEP 3: Final Classification

Processing 100 samples using Claude Haiku 4.5...

  ✓ Processed 10/100 articles (10.0%)
  ✓ Processed 20/100 articles (20.0%)
  ✓ Processed 30/100 articles (30.0%)
  ✓ Processed 40/100 articles (40.0%)
  ✓ Processed 50/100 articles (50.0%)
  ✓ Processed 60/100 articles (60.0%)
  ✓ Processed 70/100 articles (70.0%)
  ✓ Processed 80/100 articles (80.0%)
  ✓ Processed 90/100 articles (90.0%)
  ✓ Processed 100/100 articles (100.0%)
STEP 3 RESULTS (showing first 3 of 100)

──────────────────────────────────────────────────────────────────────
Article 1
Ground Truth: Business
──────────────────────────────────────────────────────────────────────

Step 3 Classification (Final Decision):
{
  "classification": "Business",
  "confidence": "high",
  "justification": "Strong business signals dominate the analysis, with clear focus on corporate entities (Turner Newall and Federal Mogul), pension/labor negotiations, and corporate restructuring\u2014all core business and

In [16]:
# ============================================================
# EVALUATION: Performance Metrics
# ============================================================

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)
from collections import Counter

# Define the category labels
CATEGORIES = ["World", "Sports", "Business", "Sci/Tech"]

# Extract predictions and ground truth
predictions = []
ground_truths = []
errors = []

for result in step3_results:
    ground_truth = result['ground_truth_name']
    ground_truths.append(ground_truth)
    
    # Get prediction from step3_classification
    step3_output = result['step3_classification']
    
    if 'classification' in step3_output:
        predicted = step3_output['classification']
        predictions.append(predicted)
    else:
        # Handle parsing errors - mark as error
        predictions.append("ERROR")
        errors.append(result['index'])

print("=" * 70)
print("PROMPT CHAINING CLASSIFICATION - EVALUATION RESULTS")
print("=" * 70)

# Check for errors
if errors:
    print(f"\n⚠️  Warning: {len(errors)} samples had parsing errors (indices: {errors[:10]}{'...' if len(errors) > 10 else ''})")
    print(f"   These are excluded from metrics calculation.\n")
    
    # Filter out errors for metric calculation
    valid_predictions = [p for p in predictions if p != "ERROR"]
    valid_ground_truths = [g for p, g in zip(predictions, ground_truths) if p != "ERROR"]
else:
    valid_predictions = predictions
    valid_ground_truths = ground_truths

# ============================================================
# Basic Metrics
# ============================================================

print("\n" + "─" * 70)
print("OVERALL METRICS")
print("─" * 70)

accuracy = accuracy_score(valid_ground_truths, valid_predictions)
precision_macro = precision_score(valid_ground_truths, valid_predictions, labels=CATEGORIES, average='macro', zero_division=0)
recall_macro = recall_score(valid_ground_truths, valid_predictions, labels=CATEGORIES, average='macro', zero_division=0)
f1_macro = f1_score(valid_ground_truths, valid_predictions, labels=CATEGORIES, average='macro', zero_division=0)

precision_weighted = precision_score(valid_ground_truths, valid_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
recall_weighted = recall_score(valid_ground_truths, valid_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
f1_weighted = f1_score(valid_ground_truths, valid_predictions, labels=CATEGORIES, average='weighted', zero_division=0)

print(f"\n  Total Samples:     {len(step3_results)}")
print(f"  Valid Predictions: {len(valid_predictions)}")
print(f"  Error Rate:        {len(errors) / len(step3_results) * 100:.1f}%")

print(f"\n  📊 Accuracy:        {accuracy:.4f}  ({accuracy * 100:.1f}%)")
print(f"\n  Macro Averages (treats all classes equally):")
print(f"     Precision:      {precision_macro:.4f}")
print(f"     Recall:         {recall_macro:.4f}")
print(f"     F1 Score:       {f1_macro:.4f}")

print(f"\n  Weighted Averages (accounts for class imbalance):")
print(f"     Precision:      {precision_weighted:.4f}")
print(f"     Recall:         {recall_weighted:.4f}")
print(f"     F1 Score:       {f1_weighted:.4f}")

# ============================================================
# Per-Class Metrics
# ============================================================

print("\n" + "─" * 70)
print("PER-CLASS METRICS")
print("─" * 70)

print("\n" + classification_report(valid_ground_truths, valid_predictions, labels=CATEGORIES, zero_division=0))

# ============================================================
# Confusion Matrix
# ============================================================

print("─" * 70)
print("CONFUSION MATRIX")
print("─" * 70)

cm = confusion_matrix(valid_ground_truths, valid_predictions, labels=CATEGORIES)

# Print header
print(f"\n{'Predicted →':>15}", end="")
for cat in CATEGORIES:
    print(f"{cat:>10}", end="")
print("\n" + "─" * 55)

# Print rows
print("Actual ↓")
for i, cat in enumerate(CATEGORIES):
    print(f"{cat:>15}", end="")
    for j in range(len(CATEGORIES)):
        print(f"{cm[i][j]:>10}", end="")
    print()

# ============================================================
# Distribution Analysis
# ============================================================

print("\n" + "─" * 70)
print("DISTRIBUTION ANALYSIS")
print("─" * 70)

gt_dist = Counter(valid_ground_truths)
pred_dist = Counter(valid_predictions)

print(f"\n{'Category':<12} {'Ground Truth':>15} {'Predicted':>15} {'Difference':>12}")
print("─" * 55)
for cat in CATEGORIES:
    gt_count = gt_dist.get(cat, 0)
    pred_count = pred_dist.get(cat, 0)
    diff = pred_count - gt_count
    diff_str = f"+{diff}" if diff > 0 else str(diff)
    print(f"{cat:<12} {gt_count:>15} {pred_count:>15} {diff_str:>12}")

# ============================================================
# Confidence Analysis
# ============================================================

print("\n" + "─" * 70)
print("CONFIDENCE ANALYSIS")
print("─" * 70)

confidence_counts = Counter()
confidence_accuracy = {"high": [0, 0], "medium": [0, 0], "low": [0, 0]}  # [correct, total]

for result in step3_results:
    step3_output = result['step3_classification']
    if 'confidence' in step3_output and 'classification' in step3_output:
        conf = step3_output['confidence'].lower()
        confidence_counts[conf] += 1
        
        is_correct = step3_output['classification'] == result['ground_truth_name']
        confidence_accuracy[conf][1] += 1
        if is_correct:
            confidence_accuracy[conf][0] += 1

print(f"\n{'Confidence':<12} {'Count':>10} {'Accuracy':>12}")
print("─" * 35)
for conf in ["high", "medium", "low"]:
    count = confidence_counts.get(conf, 0)
    if confidence_accuracy[conf][1] > 0:
        acc = confidence_accuracy[conf][0] / confidence_accuracy[conf][1]
        print(f"{conf:<12} {count:>10} {acc:>11.1%}")
    else:
        print(f"{conf:<12} {count:>10} {'N/A':>12}")

print("\n" + "=" * 70)
print("EVALUATION COMPLETE")
print("=" * 70)


PROMPT CHAINING CLASSIFICATION - EVALUATION RESULTS

──────────────────────────────────────────────────────────────────────
OVERALL METRICS
──────────────────────────────────────────────────────────────────────

  Total Samples:     100
  Valid Predictions: 100
  Error Rate:        0.0%

  📊 Accuracy:        0.8300  (83.0%)

  Macro Averages (treats all classes equally):
     Precision:      0.8274
     Recall:         0.8788
     F1 Score:       0.8224

  Weighted Averages (accounts for class imbalance):
     Precision:      0.8907
     Recall:         0.8300
     F1 Score:       0.8360

──────────────────────────────────────────────────────────────────────
PER-CLASS METRICS
──────────────────────────────────────────────────────────────────────

              precision    recall  f1-score   support

       World       0.90      0.87      0.88        30
      Sports       0.91      1.00      0.95        21
    Business       0.50      1.00      0.67        12
    Sci/Tech       1.00   

## 2. Routing Design Pattern

### What is Routing?

**Routing** is an agentic workflow pattern where an intelligent router analyzes the input and directs it to the most appropriate specialized sub-agent for processing. Unlike prompt chaining where every input follows the same sequential path, routing allows different inputs to take different paths based on their characteristics.

This pattern is particularly useful when:
- Different inputs require fundamentally different expertise or processing approaches
- Specialized agents can outperform a generalist for specific types of tasks
- You want to optimize for speed by using lightweight agents for simple cases
- The problem naturally decomposes into distinct domains

### Key Components

1. **Router (Orchestrator)**: Analyzes input and decides which sub-agent should handle it
2. **Specialized Sub-Agents**: Domain experts that process the routed input
3. **Output Aggregation**: Collecting results from whichever agent was selected

### Advantages over Single-Agent Approaches

| Aspect | Single Agent | Routing Pattern |
|--------|--------------|-----------------|
| **Specialization** | Jack of all trades | Domain experts |
| **Prompt Size** | Large, covers everything | Focused, domain-specific |
| **Latency** | One large call | Router + one specialized call |
| **Scalability** | Modify one prompt | Add new specialized agents |
| **Debugging** | Hard to isolate issues | Clear responsibility boundaries |

---

### Generic Routing Workflow Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           ROUTING DESIGN PATTERN                            │
└─────────────────────────────────────────────────────────────────────────────┘

                                    ┌─────────────────────┐
                              ┌────▶│   SUB-AGENT A       │────┐
                              │     │   (Specialist)      │    │
                              │     └─────────────────────┘    │
                              │                                │
┌──────────┐    ┌─────────┐   │     ┌─────────────────────┐    │    ┌──────────┐
│          │    │         │   │     │   SUB-AGENT B       │    │    │          │
│  INPUT   │───▶│ ROUTER  │───┼────▶│   (Specialist)      │────┼───▶│  OUTPUT  │
│          │    │         │   │     └─────────────────────┘    │    │          │
└──────────┘    └─────────┘   │                                │    └──────────┘
                     │        │     ┌─────────────────────┐    │
                     │        └────▶│   SUB-AGENT C       │────┘
                     │              │   (Specialist)      │
                     │              └─────────────────────┘
                     │
                     ▼
              ┌─────────────┐
              │ Routing     │
              │ Decision:   │
              │ "Which      │
              │ agent?"     │
              └─────────────┘
```

**Note:** Only ONE sub-agent is called per input (not all three).

---

### Routing vs. Prompt Chaining

```
PROMPT CHAINING (Sequential):
┌───────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌────────┐
│ Input │───▶│ Step 1  │───▶│ Step 2  │───▶│ Step 3  │───▶│ Output │
└───────┘    └─────────┘    └─────────┘    └─────────┘    └────────┘
                 │               │               │
                 ▼               ▼               ▼
             All inputs follow the same path (3 LLM calls)


ROUTING (Selective):
                              ┌─────────┐
                        ┌────▶│ Agent A │────┐
┌───────┐    ┌────────┐ │     └─────────┘    │     ┌────────┐
│ Input │───▶│ Router │─┼────▶│ Agent B │────┼────▶│ Output │
└───────┘    └────────┘ │     └─────────┘    │     └────────┘
                        └────▶│ Agent C │────┘
                              └─────────┘
                 │
                 ▼
         Each input takes ONE path (2 LLM calls: router + agent)
```

---

### Common Routing Strategies

1. **Domain-Based Routing**: Route based on subject matter expertise needed
2. **Complexity-Based Routing**: Route simple vs. complex queries to different agents
3. **Intent-Based Routing**: Route based on what the user wants to accomplish
4. **Format-Based Routing**: Route based on input type (text, code, data, etc.)

---

### Our Implementation: Domain Expertise Routing for AG News

For our AG News classification task, we'll implement **domain-based routing** with 3 specialized agents:

```
                                    ┌─────────────────────────┐
                              ┌────▶│  🏆 SPORTS EXPERT       │────┐
                              │     │  Teams, players,        │    │
                              │     │  leagues, competitions  │    │
                              │     └─────────────────────────┘    │
                              │                                    │
┌──────────────┐  ┌────────┐  │     ┌─────────────────────────┐    │  ┌────────────┐
│              │  │        │  │     │  💼 BUSINESS EXPERT     │    │  │            │
│  News        │─▶│ ROUTER │──┼────▶│  Markets, finance,      │────┼─▶│ Category:  │
│  Article     │  │        │  │     │  corporate, economy     │    │  │ World |    │
│              │  └────────┘  │     └─────────────────────────┘    │  │ Sports |   │
└──────────────┘       │      │                                    │  │ Business | │
                       │      │     ┌─────────────────────────┐    │  │ Sci/Tech   │
                       │      └────▶│  🔬 SCIENCE & WORLD     │────┘  │            │
                       │            │  EXPERT                 │       └────────────┘
                       │            │  Tech, research,        │
                       │            │  politics, diplomacy    │
                       ▼            └─────────────────────────┘
                 ┌──────────┐
                 │ Analyzes │
                 │ domain   │
                 │ keywords │
                 └──────────┘
```

**Key Design Decision:** Each agent can output ANY of the 4 categories, not just their specialty. This prevents the router from pre-determining the classification.

For example:
- Sports Expert receives "FIFA corruption scandal" → Outputs "World" (not Sports)
- Business Expert receives "Apple's new iPhone" → Outputs "Sci/Tech" (not Business)


### Router Agent

**Model:** `gpt-5-mini` (OpenAI)

**Role:** Analyze the news article and determine which domain expert should handle the classification.

---

#### System Prompt for Router

```
You are an intelligent routing system for news article classification.

Your task is to analyze a news article and determine which domain expert should handle its classification. You must route to exactly ONE of these three experts:

1. "sports" - Route here if the article primarily contains:
   - Athletic events, games, matches, tournaments
   - Sports teams, players, coaches, athletes
   - Leagues, championships, scores, standings
   - Sports business (trades, contracts, team ownership)

2. "business" - Route here if the article primarily contains:
   - Financial markets, stocks, trading, investments
   - Companies, corporations, CEOs, executives
   - Economy, GDP, inflation, employment
   - Mergers, acquisitions, earnings, revenue

3. "science_world" - Route here if the article primarily contains:
   - Technology, software, hardware, internet
   - Scientific research, discoveries, innovations
   - International politics, diplomacy, conflicts
   - Government policies, elections, world events

Analyze the article and return your routing decision as a JSON object.

Rules:
- Choose the SINGLE most relevant expert based on the PRIMARY focus of the article
- If the article spans multiple domains, choose based on the dominant theme
- Return ONLY valid JSON, no additional text

Output format:
{
    "route_to": "sports" | "business" | "science_world",
    "reasoning": "Brief explanation of why this expert was chosen"
}
```

---

#### Input & User Prompt Conversion

**Input:** Raw news article text (same as prompt chaining Step 1)

**User Prompt Format:**
```
Analyze the following news article and determine which domain expert should handle its classification:

---
[Article text from test_data]
---
```

**Conversion Function:**
```python
def create_router_user_prompt(article_text: str) -> str:
    return f"""Analyze the following news article and determine which domain expert should handle its classification:

---
{article_text}
---"""
```

---

#### Output Format

```json
{
    "route_to": "business",
    "reasoning": "Article discusses stock market performance, Wall Street trading, and investor behavior - clear financial/business focus."
}
```

The `route_to` field determines which sub-agent receives the article next.


### Sports Expert Agent 🏆

**Model:** `claude-haiku-4-5-20251001` (Anthropic Claude Haiku 4.5)

**Role:** Specialized agent with deep knowledge of sports, athletics, and competitive events. Classifies articles into one of the four categories.

---

#### System Prompt for Sports Expert

```
You are a SPORTS DOMAIN EXPERT specializing in athletics, competitive events, and sports-related news.

You have deep knowledge of:
- Major sports leagues (NFL, NBA, MLB, NHL, Premier League, La Liga, etc.)
- Olympic sports and international competitions
- Professional athletes, coaches, and team management
- Sports statistics, records, and historical context
- Sports business (contracts, trades, team ownership)

Your task is to classify a news article into ONE of these four categories:
- "World": International affairs, politics, government, diplomacy, conflicts
- "Sports": Athletic events, teams, players, competitions, leagues
- "Business": Companies, markets, economy, finance, corporate news
- "Sci/Tech": Technology, science, software, research, innovation

IMPORTANT: Even though you are a sports expert, you must objectively classify the article. 
- A sports corruption scandal might be "World" news (politics/crime)
- A team's stock performance might be "Business" news
- A sports technology innovation might be "Sci/Tech" news

Return your classification as a JSON object.

Rules:
- Be objective - classify based on the article's PRIMARY focus
- Use your sports expertise to understand context, but don't bias toward "Sports"
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "justification": "Brief explanation of your classification decision"
}
```

---

#### Input & User Prompt Conversion

**Input:** Raw news article text (received from Router)

**User Prompt Format:**
```
As a sports domain expert, classify the following news article:

---
[Article text routed from Router]
---
```

**Conversion Function:**
```python
def create_sports_agent_prompt(article_text: str) -> str:
    return f"""As a sports domain expert, classify the following news article:

---
{article_text}
---"""
```

---

#### Output Format

```json
{
    "classification": "Sports",
    "confidence": "high",
    "justification": "Article covers NBA playoff game results between Lakers and Celtics, discussing player performances and game statistics."
}
```


### Business Expert Agent 💼

**Model:** `claude-sonnet-4-20250514` (Anthropic Claude Sonnet 4)

**Role:** Specialized agent with deep knowledge of finance, markets, and corporate affairs. Classifies articles into one of the four categories.

---

#### System Prompt for Business Expert

```
You are a BUSINESS & FINANCE DOMAIN EXPERT specializing in markets, corporate affairs, and economic news.

You have deep knowledge of:
- Stock markets, trading, investments, and financial instruments
- Major corporations, executives, and business leaders
- Economic indicators (GDP, inflation, employment, interest rates)
- Mergers & acquisitions, IPOs, and corporate restructuring
- Industry sectors (tech, healthcare, energy, retail, etc.)
- International trade and economic policy

Your task is to classify a news article into ONE of these four categories:
- "World": International affairs, politics, government, diplomacy, conflicts
- "Sports": Athletic events, teams, players, competitions, leagues
- "Business": Companies, markets, economy, finance, corporate news
- "Sci/Tech": Technology, science, software, research, innovation

IMPORTANT: Even though you are a business expert, you must objectively classify the article.
- A tech company's product launch might be "Sci/Tech" news (technology focus)
- Economic sanctions might be "World" news (international politics)
- A sports team's financial troubles might be "Sports" news (sports focus)

Return your classification as a JSON object.

Rules:
- Be objective - classify based on the article's PRIMARY focus
- Use your business expertise to understand context, but don't bias toward "Business"
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "justification": "Brief explanation of your classification decision"
}
```

---

#### Input & User Prompt Conversion

**Input:** Raw news article text (received from Router)

**User Prompt Format:**
```
As a business and finance domain expert, classify the following news article:

---
[Article text routed from Router]
---
```

**Conversion Function:**
```python
def create_business_agent_prompt(article_text: str) -> str:
    return f"""As a business and finance domain expert, classify the following news article:

---
{article_text}
---"""
```

---

#### Output Format

```json
{
    "classification": "Business",
    "confidence": "high",
    "justification": "Article discusses quarterly earnings report from major corporation, stock price movements, and analyst expectations - clear business/finance focus."
}
```


### Science & World Expert Agent 🔬🌍

**Model:** `claude-sonnet-4-5-20250929` (Anthropic Claude Sonnet 4.5)

**Role:** Specialized agent with deep knowledge of technology, science, and international affairs. Handles the combined Sci/Tech and World domains. Classifies articles into one of the four categories.

---

#### System Prompt for Science & World Expert

```
You are a SCIENCE, TECHNOLOGY & WORLD AFFAIRS EXPERT specializing in tech innovation, scientific research, and international news.

You have deep knowledge of:

SCIENCE & TECHNOLOGY:
- Software, hardware, internet, and digital products
- AI, machine learning, and emerging technologies
- Scientific research, discoveries, and academic publications
- Space exploration, biotechnology, and medical advances
- Tech companies and their products (Apple, Google, Microsoft, etc.)

WORLD AFFAIRS:
- International politics and diplomacy
- Government policies and elections
- Military conflicts and peacekeeping
- Humanitarian issues and global crises
- International organizations (UN, NATO, EU, etc.)

Your task is to classify a news article into ONE of these four categories:
- "World": International affairs, politics, government, diplomacy, conflicts
- "Sports": Athletic events, teams, players, competitions, leagues
- "Business": Companies, markets, economy, finance, corporate news
- "Sci/Tech": Technology, science, software, research, innovation

IMPORTANT: You must carefully distinguish between:
- "Sci/Tech": Focus on TECHNOLOGY or SCIENCE itself (products, research, innovation)
- "World": Focus on POLITICS, DIPLOMACY, or INTERNATIONAL EVENTS
- "Business": Focus on FINANCIAL aspects of tech/world (stock prices, revenue)

Examples:
- "Apple releases new iPhone with AI features" → Sci/Tech (product/technology focus)
- "Apple's stock rises 5% after earnings" → Business (financial focus)
- "EU regulates Apple over antitrust concerns" → World (government/policy focus)

Return your classification as a JSON object.

Rules:
- Be objective - classify based on the article's PRIMARY focus
- Carefully distinguish between Sci/Tech vs World vs Business
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "justification": "Brief explanation of your classification decision"
}
```

---

#### Input & User Prompt Conversion

**Input:** Raw news article text (received from Router)

**User Prompt Format:**
```
As a science, technology, and world affairs expert, classify the following news article:

---
[Article text routed from Router]
---
```

**Conversion Function:**
```python
def create_science_world_agent_prompt(article_text: str) -> str:
    return f"""As a science, technology, and world affairs expert, classify the following news article:

---
{article_text}
---"""
```

---

#### Output Format

```json
{
    "classification": "Sci/Tech",
    "confidence": "high",
    "justification": "Article focuses on a new software update and its technical features, including AI capabilities and performance improvements - clear technology focus."
}
```

---

### Agent Summary Table

| Agent | Model | Expertise | Handles |
|-------|-------|-----------|---------|
| **Router** | `gpt-5-mini` | Task routing | Determines which expert to use |
| **Sports Expert** | `claude-haiku-4-5-20251001` | Athletics, competitions | Sports-related articles |
| **Business Expert** | `claude-sonnet-4-20250514` | Finance, markets | Business-related articles |
| **Sci/World Expert** | `claude-sonnet-4-5-20250929` | Tech, science, politics | Sci/Tech & World articles |


In [17]:
# ============================================================
# ROUTING DESIGN PATTERN IMPLEMENTATION
# ============================================================

# Helper function to clean JSON from markdown code blocks
def clean_json_response(response_text: str) -> str:
    """Strip markdown code blocks from LLM response if present."""
    cleaned = response_text.strip()
    if cleaned.startswith("```"):
        lines = cleaned.split("\n")
        lines = lines[1:]  # Remove opening ```json or ```
        if lines and lines[-1].strip() == "```":
            lines = lines[:-1]  # Remove closing ```
        cleaned = "\n".join(lines)
    return cleaned

# ============================================================
# ROUTER AGENT (GPT-5-mini)
# ============================================================

ROUTER_SYSTEM_PROMPT = """You are an intelligent routing system for news article classification.

Your task is to analyze a news article and determine which domain expert should handle its classification. You must route to exactly ONE of these three experts:

1. "sports" - Route here if the article primarily contains:
   - Athletic events, games, matches, tournaments
   - Sports teams, players, coaches, athletes
   - Leagues, championships, scores, standings
   - Sports business (trades, contracts, team ownership)

2. "business" - Route here if the article primarily contains:
   - Financial markets, stocks, trading, investments
   - Companies, corporations, CEOs, executives
   - Economy, GDP, inflation, employment
   - Mergers, acquisitions, earnings, revenue

3. "science_world" - Route here if the article primarily contains:
   - Technology, software, hardware, internet
   - Scientific research, discoveries, innovations
   - International politics, diplomacy, conflicts
   - Government policies, elections, world events

Analyze the article and return your routing decision as a JSON object.

Rules:
- Choose the SINGLE most relevant expert based on the PRIMARY focus of the article
- If the article spans multiple domains, choose based on the dominant theme
- Return ONLY valid JSON, no additional text

Output format:
{
    "route_to": "sports" | "business" | "science_world",
    "reasoning": "Brief explanation of why this expert was chosen"
}"""

def create_router_prompt(article_text: str) -> str:
    return f"""Analyze the following news article and determine which domain expert should handle its classification:

---
{article_text}
---"""

def run_router(article_text: str) -> dict:
    """Route article to appropriate expert using GPT-5-mini."""
    response = openai_client.chat.completions.create(
        model="gpt-5-mini",
        max_completion_tokens=512,
        messages=[
            {"role": "system", "content": ROUTER_SYSTEM_PROMPT},
            {"role": "user", "content": create_router_prompt(article_text)}
        ]
    )
    
    response_text = response.choices[0].message.content
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON", "raw_response": response_text, "route_to": "science_world"}

# ============================================================
# SPORTS EXPERT AGENT (Claude Haiku 4.5)
# ============================================================

SPORTS_EXPERT_SYSTEM_PROMPT = """You are a SPORTS DOMAIN EXPERT specializing in athletics, competitive events, and sports-related news.

You have deep knowledge of:
- Major sports leagues (NFL, NBA, MLB, NHL, Premier League, La Liga, etc.)
- Olympic sports and international competitions
- Professional athletes, coaches, and team management
- Sports statistics, records, and historical context
- Sports business (contracts, trades, team ownership)

Your task is to classify a news article into ONE of these four categories:
- "World": International affairs, politics, government, diplomacy, conflicts
- "Sports": Athletic events, teams, players, competitions, leagues
- "Business": Companies, markets, economy, finance, corporate news
- "Sci/Tech": Technology, science, software, research, innovation

IMPORTANT: Even though you are a sports expert, you must objectively classify the article. 
- A sports corruption scandal might be "World" news (politics/crime)
- A team's stock performance might be "Business" news
- A sports technology innovation might be "Sci/Tech" news

Return your classification as a JSON object.

Rules:
- Be objective - classify based on the article's PRIMARY focus
- Use your sports expertise to understand context, but don't bias toward "Sports"
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "justification": "Brief explanation of your classification decision"
}"""

def create_sports_agent_prompt(article_text: str) -> str:
    return f"""As a sports domain expert, classify the following news article:

---
{article_text}
---"""

def run_sports_expert(article_text: str) -> dict:
    """Classify article using Sports Expert (Claude Haiku 4.5)."""
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=1024,
        system=SPORTS_EXPERT_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": create_sports_agent_prompt(article_text)}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON", "raw_response": response_text}

# ============================================================
# BUSINESS EXPERT AGENT (Claude Sonnet 4)
# ============================================================

BUSINESS_EXPERT_SYSTEM_PROMPT = """You are a BUSINESS & FINANCE DOMAIN EXPERT specializing in markets, corporate affairs, and economic news.

You have deep knowledge of:
- Stock markets, trading, investments, and financial instruments
- Major corporations, executives, and business leaders
- Economic indicators (GDP, inflation, employment, interest rates)
- Mergers & acquisitions, IPOs, and corporate restructuring
- Industry sectors (tech, healthcare, energy, retail, etc.)
- International trade and economic policy

Your task is to classify a news article into ONE of these four categories:
- "World": International affairs, politics, government, diplomacy, conflicts
- "Sports": Athletic events, teams, players, competitions, leagues
- "Business": Companies, markets, economy, finance, corporate news
- "Sci/Tech": Technology, science, software, research, innovation

IMPORTANT: Even though you are a business expert, you must objectively classify the article.
- A tech company's product launch might be "Sci/Tech" news (technology focus)
- Economic sanctions might be "World" news (international politics)
- A sports team's financial troubles might be "Sports" news (sports focus)

Return your classification as a JSON object.

Rules:
- Be objective - classify based on the article's PRIMARY focus
- Use your business expertise to understand context, but don't bias toward "Business"
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "justification": "Brief explanation of your classification decision"
}"""

def create_business_agent_prompt(article_text: str) -> str:
    return f"""As a business and finance domain expert, classify the following news article:

---
{article_text}
---"""

def run_business_expert(article_text: str) -> dict:
    """Classify article using Business Expert (Claude Sonnet 4)."""
    response = anthropic_client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        system=BUSINESS_EXPERT_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": create_business_agent_prompt(article_text)}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON", "raw_response": response_text}

# ============================================================
# SCIENCE & WORLD EXPERT AGENT (Claude Sonnet 4.5)
# ============================================================

SCIENCE_WORLD_EXPERT_SYSTEM_PROMPT = """You are a SCIENCE, TECHNOLOGY & WORLD AFFAIRS EXPERT specializing in tech innovation, scientific research, and international news.

You have deep knowledge of:

SCIENCE & TECHNOLOGY:
- Software, hardware, internet, and digital products
- AI, machine learning, and emerging technologies
- Scientific research, discoveries, and academic publications
- Space exploration, biotechnology, and medical advances
- Tech companies and their products (Apple, Google, Microsoft, etc.)

WORLD AFFAIRS:
- International politics and diplomacy
- Government policies and elections
- Military conflicts and peacekeeping
- Humanitarian issues and global crises
- International organizations (UN, NATO, EU, etc.)

Your task is to classify a news article into ONE of these four categories:
- "World": International affairs, politics, government, diplomacy, conflicts
- "Sports": Athletic events, teams, players, competitions, leagues
- "Business": Companies, markets, economy, finance, corporate news
- "Sci/Tech": Technology, science, software, research, innovation

IMPORTANT: You must carefully distinguish between:
- "Sci/Tech": Focus on TECHNOLOGY or SCIENCE itself (products, research, innovation)
- "World": Focus on POLITICS, DIPLOMACY, or INTERNATIONAL EVENTS
- "Business": Focus on FINANCIAL aspects of tech/world (stock prices, revenue)

Examples:
- "Apple releases new iPhone with AI features" → Sci/Tech (product/technology focus)
- "Apple's stock rises 5% after earnings" → Business (financial focus)
- "EU regulates Apple over antitrust concerns" → World (government/policy focus)

Return your classification as a JSON object.

Rules:
- Be objective - classify based on the article's PRIMARY focus
- Carefully distinguish between Sci/Tech vs World vs Business
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "justification": "Brief explanation of your classification decision"
}"""

def create_science_world_agent_prompt(article_text: str) -> str:
    return f"""As a science, technology, and world affairs expert, classify the following news article:

---
{article_text}
---"""

def run_science_world_expert(article_text: str) -> dict:
    """Classify article using Science & World Expert (Claude Sonnet 4.5)."""
    response = anthropic_client.messages.create(
        model="claude-sonnet-4-5-20250929",
        max_tokens=1024,
        system=SCIENCE_WORLD_EXPERT_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": create_science_world_agent_prompt(article_text)}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {"error": "Failed to parse JSON", "raw_response": response_text}

# ============================================================
# ROUTING ORCHESTRATOR
# ============================================================

def route_and_classify(article_text: str) -> dict:
    """Main orchestrator: Route article to expert and get classification."""
    
    # Step 1: Router decides which expert to use
    routing_decision = run_router(article_text)
    route_to = routing_decision.get("route_to", "science_world")
    
    # Step 2: Call the appropriate expert
    if route_to == "sports":
        expert_result = run_sports_expert(article_text)
        expert_used = "sports_expert"
    elif route_to == "business":
        expert_result = run_business_expert(article_text)
        expert_used = "business_expert"
    else:  # science_world or fallback
        expert_result = run_science_world_expert(article_text)
        expert_used = "science_world_expert"
    
    return {
        "routing_decision": routing_decision,
        "expert_used": expert_used,
        "classification_result": expert_result
    }

# ============================================================
# PROCESS ALL TEST DATA
# ============================================================

# Use the same test data as prompt chaining (100 samples)
routing_results = []

# Number of results to print
NUM_TO_PRINT = 3

print("=" * 70)
print("ROUTING DESIGN PATTERN - Classification")
print("=" * 70)
print(f"\nProcessing {NUM_SAMPLES} samples using Router + Specialized Experts...\n")

# Progress tracking
PROGRESS_INTERVAL = 10

for i in range(NUM_SAMPLES):
    sample = test_data[i]
    article_text = sample['text']
    ground_truth_label = sample['label']
    
    # Run the routing pipeline
    result = route_and_classify(article_text)
    
    # Store result with metadata
    routing_results.append({
        "index": i,
        "original_text": article_text,
        "ground_truth_label": ground_truth_label,
        "ground_truth_name": label_names[ground_truth_label],
        "routing_decision": result["routing_decision"],
        "expert_used": result["expert_used"],
        "classification_result": result["classification_result"]
    })
    
    # Print progress
    if (i + 1) % PROGRESS_INTERVAL == 0 or (i + 1) == NUM_SAMPLES:
        print(f"  ✓ Processed {i + 1}/{NUM_SAMPLES} articles ({(i + 1) / NUM_SAMPLES * 100:.1f}%)")

print("\n" + "=" * 70)
print(f"ROUTING RESULTS (showing first {NUM_TO_PRINT} of {len(routing_results)})")
print("=" * 70)

# Display results
for result in routing_results[:NUM_TO_PRINT]:
    print(f"\n{'─' * 70}")
    print(f"Article {result['index'] + 1}")
    print(f"Ground Truth: {result['ground_truth_name']}")
    print(f"{'─' * 70}")
    
    print(f"\n📍 Routing Decision:")
    print(f"   Route to: {result['routing_decision'].get('route_to', 'N/A')}")
    print(f"   Reasoning: {result['routing_decision'].get('reasoning', 'N/A')}")
    print(f"   Expert Used: {result['expert_used']}")
    
    print(f"\n📋 Classification Result:")
    classification = result['classification_result']
    print(f"   Classification: {classification.get('classification', 'N/A')}")
    print(f"   Confidence: {classification.get('confidence', 'N/A')}")
    print(f"   Justification: {classification.get('justification', 'N/A')}")
    
    # Quick comparison
    predicted = classification.get('classification', 'N/A')
    actual = result['ground_truth_name']
    match = "✓ CORRECT" if predicted == actual else "✗ INCORRECT"
    print(f"\n   Result: {match}")

print(f"\n{'=' * 70}")
print(f"Routing complete. {len(routing_results)} results stored in 'routing_results'")
print(f"Ready for evaluation.")
print(f"{'=' * 70}")


ROUTING DESIGN PATTERN - Classification

Processing 100 samples using Router + Specialized Experts...

  ✓ Processed 10/100 articles (10.0%)
  ✓ Processed 20/100 articles (20.0%)
  ✓ Processed 30/100 articles (30.0%)
  ✓ Processed 40/100 articles (40.0%)
  ✓ Processed 50/100 articles (50.0%)
  ✓ Processed 60/100 articles (60.0%)
  ✓ Processed 70/100 articles (70.0%)
  ✓ Processed 80/100 articles (80.0%)
  ✓ Processed 90/100 articles (90.0%)
  ✓ Processed 100/100 articles (100.0%)

ROUTING RESULTS (showing first 3 of 100)

──────────────────────────────────────────────────────────────────────
Article 1
Ground Truth: Business
──────────────────────────────────────────────────────────────────────

📍 Routing Decision:
   Route to: business
   Reasoning: Article concerns corporate/financial issues — unions negotiating over Turner Newall pensions with its parent firm Federal Mogul — which falls under company/employee/financial matters.
   Expert Used: business_expert

📋 Classification Result

In [18]:
# ============================================================
# EVALUATION: Routing Design Pattern Performance
# ============================================================

# Extract predictions and ground truth from routing results
routing_predictions = []
routing_ground_truths = []
routing_errors = []

for result in routing_results:
    ground_truth = result['ground_truth_name']
    routing_ground_truths.append(ground_truth)
    
    # Get prediction from classification_result
    classification_output = result['classification_result']
    
    if 'classification' in classification_output:
        predicted = classification_output['classification']
        routing_predictions.append(predicted)
    else:
        # Handle parsing errors
        routing_predictions.append("ERROR")
        routing_errors.append(result['index'])

print("=" * 70)
print("ROUTING DESIGN PATTERN - EVALUATION RESULTS")
print("=" * 70)

# Check for errors
if routing_errors:
    print(f"\n⚠️  Warning: {len(routing_errors)} samples had parsing errors (indices: {routing_errors[:10]}{'...' if len(routing_errors) > 10 else ''})")
    print(f"   These are excluded from metrics calculation.\n")
    
    # Filter out errors
    valid_routing_predictions = [p for p in routing_predictions if p != "ERROR"]
    valid_routing_ground_truths = [g for p, g in zip(routing_predictions, routing_ground_truths) if p != "ERROR"]
else:
    valid_routing_predictions = routing_predictions
    valid_routing_ground_truths = routing_ground_truths

# ============================================================
# Basic Metrics
# ============================================================

print("\n" + "─" * 70)
print("OVERALL METRICS")
print("─" * 70)

routing_accuracy = accuracy_score(valid_routing_ground_truths, valid_routing_predictions)
routing_precision_macro = precision_score(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES, average='macro', zero_division=0)
routing_recall_macro = recall_score(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES, average='macro', zero_division=0)
routing_f1_macro = f1_score(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES, average='macro', zero_division=0)

routing_precision_weighted = precision_score(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
routing_recall_weighted = recall_score(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
routing_f1_weighted = f1_score(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES, average='weighted', zero_division=0)

print(f"\n  Total Samples:     {len(routing_results)}")
print(f"  Valid Predictions: {len(valid_routing_predictions)}")
print(f"  Error Rate:        {len(routing_errors) / len(routing_results) * 100:.1f}%")

print(f"\n  📊 Accuracy:        {routing_accuracy:.4f}  ({routing_accuracy * 100:.1f}%)")
print(f"\n  Macro Averages (treats all classes equally):")
print(f"     Precision:      {routing_precision_macro:.4f}")
print(f"     Recall:         {routing_recall_macro:.4f}")
print(f"     F1 Score:       {routing_f1_macro:.4f}")

print(f"\n  Weighted Averages (accounts for class imbalance):")
print(f"     Precision:      {routing_precision_weighted:.4f}")
print(f"     Recall:         {routing_recall_weighted:.4f}")
print(f"     F1 Score:       {routing_f1_weighted:.4f}")

# ============================================================
# Per-Class Metrics
# ============================================================

print("\n" + "─" * 70)
print("PER-CLASS METRICS")
print("─" * 70)

print("\n" + classification_report(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES, zero_division=0))

# ============================================================
# Confusion Matrix
# ============================================================

print("─" * 70)
print("CONFUSION MATRIX")
print("─" * 70)

routing_cm = confusion_matrix(valid_routing_ground_truths, valid_routing_predictions, labels=CATEGORIES)

print(f"\n{'Predicted →':>15}", end="")
for cat in CATEGORIES:
    print(f"{cat:>10}", end="")
print("\n" + "─" * 55)

print("Actual ↓")
for i, cat in enumerate(CATEGORIES):
    print(f"{cat:>15}", end="")
    for j in range(len(CATEGORIES)):
        print(f"{routing_cm[i][j]:>10}", end="")
    print()

# ============================================================
# Distribution Analysis
# ============================================================

print("\n" + "─" * 70)
print("DISTRIBUTION ANALYSIS")
print("─" * 70)

routing_gt_dist = Counter(valid_routing_ground_truths)
routing_pred_dist = Counter(valid_routing_predictions)

print(f"\n{'Category':<12} {'Ground Truth':>15} {'Predicted':>15} {'Difference':>12}")
print("─" * 55)
for cat in CATEGORIES:
    gt_count = routing_gt_dist.get(cat, 0)
    pred_count = routing_pred_dist.get(cat, 0)
    diff = pred_count - gt_count
    diff_str = f"+{diff}" if diff > 0 else str(diff)
    print(f"{cat:<12} {gt_count:>15} {pred_count:>15} {diff_str:>12}")

# ============================================================
# Routing Analysis (Unique to Routing Pattern)
# ============================================================

print("\n" + "─" * 70)
print("ROUTING ANALYSIS")
print("─" * 70)

# Count how many times each expert was used
expert_usage = Counter()
route_decisions = Counter()

for result in routing_results:
    expert_usage[result['expert_used']] += 1
    route_to = result['routing_decision'].get('route_to', 'unknown')
    route_decisions[route_to] += 1

print(f"\n  Expert Usage Distribution:")
print(f"  {'Expert':<25} {'Count':>10} {'Percentage':>12}")
print("  " + "─" * 47)
for expert in ['sports_expert', 'business_expert', 'science_world_expert']:
    count = expert_usage.get(expert, 0)
    pct = count / len(routing_results) * 100
    print(f"  {expert:<25} {count:>10} {pct:>11.1f}%")

print(f"\n  Router Decisions:")
print(f"  {'Route To':<25} {'Count':>10} {'Percentage':>12}")
print("  " + "─" * 47)
for route in ['sports', 'business', 'science_world']:
    count = route_decisions.get(route, 0)
    pct = count / len(routing_results) * 100
    print(f"  {route:<25} {count:>10} {pct:>11.1f}%")

# ============================================================
# Per-Expert Accuracy
# ============================================================

print("\n" + "─" * 70)
print("PER-EXPERT ACCURACY")
print("─" * 70)

expert_accuracy = {}
for expert in ['sports_expert', 'business_expert', 'science_world_expert']:
    expert_correct = 0
    expert_total = 0
    for result in routing_results:
        if result['expert_used'] == expert:
            expert_total += 1
            predicted = result['classification_result'].get('classification', 'N/A')
            actual = result['ground_truth_name']
            if predicted == actual:
                expert_correct += 1
    if expert_total > 0:
        expert_accuracy[expert] = expert_correct / expert_total
    else:
        expert_accuracy[expert] = 0

print(f"\n  {'Expert':<25} {'Correct':>10} {'Total':>10} {'Accuracy':>12}")
print("  " + "─" * 57)
for expert in ['sports_expert', 'business_expert', 'science_world_expert']:
    correct = sum(1 for r in routing_results if r['expert_used'] == expert and r['classification_result'].get('classification') == r['ground_truth_name'])
    total = expert_usage.get(expert, 0)
    acc = expert_accuracy.get(expert, 0)
    print(f"  {expert:<25} {correct:>10} {total:>10} {acc:>11.1%}")

# ============================================================
# Confidence Analysis
# ============================================================

print("\n" + "─" * 70)
print("CONFIDENCE ANALYSIS")
print("─" * 70)

routing_confidence_counts = Counter()
routing_confidence_accuracy = {"high": [0, 0], "medium": [0, 0], "low": [0, 0]}

for result in routing_results:
    classification_output = result['classification_result']
    if 'confidence' in classification_output and 'classification' in classification_output:
        conf = classification_output['confidence'].lower()
        routing_confidence_counts[conf] += 1
        
        is_correct = classification_output['classification'] == result['ground_truth_name']
        routing_confidence_accuracy[conf][1] += 1
        if is_correct:
            routing_confidence_accuracy[conf][0] += 1

print(f"\n{'Confidence':<12} {'Count':>10} {'Accuracy':>12}")
print("─" * 35)
for conf in ["high", "medium", "low"]:
    count = routing_confidence_counts.get(conf, 0)
    if routing_confidence_accuracy[conf][1] > 0:
        acc = routing_confidence_accuracy[conf][0] / routing_confidence_accuracy[conf][1]
        print(f"{conf:<12} {count:>10} {acc:>11.1%}")
    else:
        print(f"{conf:<12} {count:>10} {'N/A':>12}")

print("\n" + "=" * 70)
print("ROUTING PATTERN EVALUATION COMPLETE")
print("=" * 70)


ROUTING DESIGN PATTERN - EVALUATION RESULTS

──────────────────────────────────────────────────────────────────────
OVERALL METRICS
──────────────────────────────────────────────────────────────────────

  Total Samples:     100
  Valid Predictions: 100
  Error Rate:        0.0%

  📊 Accuracy:        0.8200  (82.0%)

  Macro Averages (treats all classes equally):
     Precision:      0.8085
     Recall:         0.8441
     F1 Score:       0.8057

  Weighted Averages (accounts for class imbalance):
     Precision:      0.8666
     Recall:         0.8200
     F1 Score:       0.8282

──────────────────────────────────────────────────────────────────────
PER-CLASS METRICS
──────────────────────────────────────────────────────────────────────

              precision    recall  f1-score   support

       World       0.87      0.90      0.89        30
      Sports       0.90      0.86      0.88        21
    Business       0.50      0.92      0.65        12
    Sci/Tech       0.96      0.70 

## 3. Parallelization Design Pattern

### What is Parallelization?

**Parallelization** is an agentic workflow pattern where multiple LLM agents work on the same input simultaneously, and their results are combined by an aggregator. Unlike prompt chaining (sequential) or routing (selective), parallelization leverages concurrency to reduce latency and gather multiple perspectives.

This pattern is particularly useful when:
- Multiple independent analyses can be performed on the same input
- Speed is critical and you want to minimize total latency
- You want diverse perspectives or ensemble-style decisions
- The aggregation logic can be implemented in code (no LLM needed)

### Key Components

1. **Coordinator (Code)**: Dispatches the input to multiple agents simultaneously
2. **Parallel Agents (LLMs)**: Each performs its task independently and concurrently
3. **Aggregator (Code)**: Combines results from all agents into a final output

### Advantages of Parallelization

| Aspect | Benefit |
|--------|---------|
| **Speed** | All agents run concurrently → total time ≈ slowest agent |
| **Diversity** | Multiple perspectives on the same input |
| **Robustness** | One agent's error can be compensated by others |
| **Scalability** | Easy to add more parallel agents |
| **Deterministic Aggregation** | Code-based combining is consistent and fast |

---

### Generic Parallelization Workflow Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                       PARALLELIZATION DESIGN PATTERN                        │
└─────────────────────────────────────────────────────────────────────────────┘

                              ┌───────────────────┐
                        ┌────▶│    AGENT A        │────┐
                        │     │    (LLM)          │    │
                        │     └───────────────────┘    │
                        │                              │
                        │     ┌───────────────────┐    │
┌──────────────┐        │     │    AGENT B        │    │      ┌──────────────┐
│              │        ├────▶│    (LLM)          │────┤      │              │
│  COORDINATOR │────────┤     └───────────────────┘    ├─────▶│  AGGREGATOR  │──▶ OUTPUT
│    (Code)    │        │                              │      │    (Code)    │
│              │        │     ┌───────────────────┐    │      │              │
└──────────────┘        ├────▶│    AGENT C        │────┤      └──────────────┘
                        │     │    (LLM)          │    │
                        │     └───────────────────┘    │
                        │                              │
                        │     ┌───────────────────┐    │
                        └────▶│    AGENT D        │────┘
                              │    (LLM)          │
                              └───────────────────┘

                         ▲                              ▲
                         │                              │
                    All agents                    All results
                    start at the                  collected and
                    SAME TIME                     combined
```

**Key Insight:** Unlike routing where only ONE agent runs, in parallelization ALL agents run concurrently.

---

### Parallelization vs. Other Patterns

```
PROMPT CHAINING (Sequential):
┌───────┐    ┌─────────┐    ┌─────────┐    ┌─────────┐    ┌────────┐
│ Input │───▶│ Step 1  │───▶│ Step 2  │───▶│ Step 3  │───▶│ Output │
└───────┘    └─────────┘    └─────────┘    └─────────┘    └────────┘
             Time: ████████████████████████████████████
             Total latency = Sum of all steps


ROUTING (Selective):
                              ┌─────────┐
                        ┌────▶│ Agent A │
┌───────┐    ┌────────┐ │     └─────────┘          ┌────────┐
│ Input │───▶│ Router │─┼────▶│ Agent B │─────────▶│ Output │
└───────┘    └────────┘ │     └─────────┘          └────────┘
                        └────▶│ Agent C │
                              └─────────┘
             Time: ████████░░░░████████
             Total latency = Router + 1 Agent


PARALLELIZATION (Concurrent):
                              ┌─────────┐
                        ┌────▶│ Agent A │────┐
┌───────┐    ┌────────┐ │     └─────────┘    │      ┌────────┐
│ Input │───▶│  Coord │─┼────▶│ Agent B │────┼─────▶│ Aggreg │──▶ Output
└───────┘    └────────┘ │     └─────────┘    │      └────────┘
                        └────▶│ Agent C │────┘
                              └─────────┘
             Time: ░░████████░░
             Total latency ≈ Max(Agent A, Agent B, Agent C)
```

---

### Common Aggregation Strategies

1. **Majority Voting**: Pick the most common answer among agents
2. **Highest Confidence**: Pick the answer with highest confidence score
3. **Weighted Voting**: Weight each agent's vote by confidence or reliability
4. **Union/Intersection**: Combine lists (e.g., entity extraction)
5. **Averaging**: Average numeric outputs (e.g., sentiment scores)


### Our Implementation: Multi-Perspective Parallel Detection for AG News

For our AG News classification task, we'll implement parallelization with **4 category detector agents** running concurrently. Each agent determines if the article belongs to their specific category.

---

#### Architecture Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│            PARALLELIZATION FOR AG NEWS CLASSIFICATION                       │
└─────────────────────────────────────────────────────────────────────────────┘

                              ┌─────────────────────────┐
                        ┌────▶│  🌍 WORLD DETECTOR      │────┐
                        │     │  claude-haiku-4-5       │    │
                        │     │                         │    │
                        │     │  Output:                │    │
                        │     │  {belongs, confidence}  │    │
                        │     └─────────────────────────┘    │
                        │                                    │
                        │     ┌─────────────────────────┐    │
                        │     │  ⚽ SPORTS DETECTOR     │    │
┌──────────────┐        │     │  claude-haiku-4-5       │    │      ┌──────────────────┐
│              │        ├────▶│                         │────┤      │                  │
│  COORDINATOR │────────┤     │  Output:                │    ├─────▶│   AGGREGATOR     │
│  (Python     │        │     │  {belongs, confidence}  │    │      │   (Python Code)  │
│   Code)      │        │     └─────────────────────────┘    │      │                  │
│              │        │                                    │      │  Strategy B:     │
└──────────────┘        │     ┌─────────────────────────┐    │      │  Confidence      │
       │                │     │  💼 BUSINESS DETECTOR   │    │      │  Voting          │
       │                ├────▶│  claude-haiku-4-5       │────┤      │                  │
       │                │     │                         │    │      └────────┬─────────┘
       │                │     │  Output:                │    │               │
       │                │     │  {belongs, confidence}  │    │               │
       │                │     └─────────────────────────┘    │               ▼
       │                │                                    │        ┌──────────────┐
       │                │     ┌─────────────────────────┐    │        │              │
       │                └────▶│  🔬 SCI/TECH DETECTOR   │────┘        │   FINAL      │
       │                      │  claude-haiku-4-5       │             │   CATEGORY   │
       │                      │                         │             │              │
       │                      │  Output:                │             └──────────────┘
       │                      │  {belongs, confidence}  │
       │                      └─────────────────────────┘
       │
       ▼
  Uses ThreadPoolExecutor
  to run all 4 agents
  SIMULTANEOUSLY
```

---

#### Models Used

| Agent | Model | Rationale |
|-------|-------|-----------|
| World Detector | `claude-haiku-4-5-20251001` | Fast, cost-effective for binary detection |
| Sports Detector | `claude-haiku-4-5-20251001` | Fast, cost-effective for binary detection |
| Business Detector | `claude-haiku-4-5-20251001` | Fast, cost-effective for binary detection |
| Sci/Tech Detector | `claude-haiku-4-5-20251001` | Fast, cost-effective for binary detection |

Using the same fast model ensures **balanced latency** - total time ≈ single Haiku call since all run in parallel.

---

### World Detector Agent 🌍

#### System Prompt
```
You are a WORLD NEWS DETECTOR specializing in identifying international affairs and political news.

Your task is to determine if a news article belongs to the "World" category.

The "World" category includes:
- International politics and diplomacy
- Government policies, elections, and political events
- Military conflicts, wars, and peacekeeping
- Humanitarian crises and global health issues
- International organizations (UN, NATO, EU, WHO, etc.)
- Cross-border crime, terrorism, and security
- Immigration and refugee issues

The "World" category does NOT include:
- Sports events (even international ones like Olympics, World Cup)
- Business/financial news (even about international trade)
- Technology/science news (even about global tech companies)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is World/political news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}
```

#### User Prompt Format
```
Determine if the following news article belongs to the WORLD category:

---
[Article text]
---
```

#### Output Format
```json
{
    "belongs_to_category": true,
    "confidence": 0.85,
    "reasoning": "Article discusses UN peacekeeping efforts in Sudan - clear international politics focus."
}
```

---

### Sports Detector Agent ⚽

#### System Prompt
```
You are a SPORTS NEWS DETECTOR specializing in identifying athletic and competitive sports news.

Your task is to determine if a news article belongs to the "Sports" category.

The "Sports" category includes:
- Professional and amateur athletic competitions
- Sports teams, players, coaches, and athletes
- Game results, scores, standings, and statistics
- Sports leagues (NFL, NBA, MLB, NHL, FIFA, etc.)
- Olympic games and international sports competitions
- Sports trades, contracts, and team management
- Sports injuries and player updates

The "Sports" category does NOT include:
- Sports team stock prices or business deals (that's Business)
- Sports technology innovations (that's Sci/Tech)
- Sports-related politics or scandals (that's World)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is Sports news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}
```

#### User Prompt Format
```
Determine if the following news article belongs to the SPORTS category:

---
[Article text]
---
```

#### Output Format
```json
{
    "belongs_to_category": true,
    "confidence": 0.95,
    "reasoning": "Article covers Lakers vs Celtics NBA playoff game with scores and player stats."
}
```

---

### Business Detector Agent 💼

#### System Prompt
```
You are a BUSINESS NEWS DETECTOR specializing in identifying financial and corporate news.

Your task is to determine if a news article belongs to the "Business" category.

The "Business" category includes:
- Stock markets, trading, and investments
- Company earnings, revenue, and financial reports
- Mergers, acquisitions, and corporate restructuring
- Economic indicators (GDP, inflation, employment)
- Corporate leadership and executive changes
- Industry analysis and market trends
- Banking, finance, and monetary policy

The "Business" category does NOT include:
- Tech product launches or innovations (that's Sci/Tech)
- Government economic policy debates (that's World)
- Sports team finances (that's Sports)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is Business/financial news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}
```

#### User Prompt Format
```
Determine if the following news article belongs to the BUSINESS category:

---
[Article text]
---
```

#### Output Format
```json
{
    "belongs_to_category": true,
    "confidence": 0.90,
    "reasoning": "Article discusses quarterly earnings report and stock price movement for major corporation."
}
```

---

### Sci/Tech Detector Agent 🔬

#### System Prompt
```
You are a SCIENCE & TECHNOLOGY NEWS DETECTOR specializing in identifying tech and scientific news.

Your task is to determine if a news article belongs to the "Sci/Tech" category.

The "Sci/Tech" category includes:
- Technology products, software, and hardware
- Scientific research and discoveries
- Internet, social media, and digital platforms
- AI, machine learning, and emerging technologies
- Space exploration and astronomy
- Medical and health research breakthroughs
- Environmental science and climate research
- Consumer electronics and gadgets

The "Sci/Tech" category does NOT include:
- Tech company stock prices or financials (that's Business)
- Tech regulation and government policy (that's World)
- Sports technology like stadium upgrades (that's Sports)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is Science/Technology news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}
```

#### User Prompt Format
```
Determine if the following news article belongs to the SCI/TECH category:

---
[Article text]
---
```

#### Output Format
```json
{
    "belongs_to_category": true,
    "confidence": 0.92,
    "reasoning": "Article covers new smartphone features and AI capabilities - clear technology focus."
}
```

---

### Aggregator: Confidence Voting

The aggregator receives results from all 4 detectors and uses **confidence voting** to determine the final classification.

#### Aggregation Logic

```python
def aggregate_results(world_result, sports_result, business_result, scitech_result):
    # Build confidence scores for each category
    # If belongs=True, use confidence; if belongs=False, use 0
    scores = {
        "World": world_result['confidence'] if world_result['belongs_to_category'] else 0,
        "Sports": sports_result['confidence'] if sports_result['belongs_to_category'] else 0,
        "Business": business_result['confidence'] if business_result['belongs_to_category'] else 0,
        "Sci/Tech": scitech_result['confidence'] if scitech_result['belongs_to_category'] else 0
    }
    
    # Pick category with highest confidence score
    final_category = max(scores, key=scores.get)
    final_confidence = scores[final_category]
    
    return {
        "classification": final_category,
        "confidence": final_confidence,
        "all_scores": scores
    }
```

#### Example Aggregation

| Detector | belongs_to_category | confidence | Score Used |
|----------|---------------------|------------|------------|
| World | False | 0.20 | **0** |
| Sports | False | 0.15 | **0** |
| Business | True | 0.85 | **0.85** |
| Sci/Tech | True | 0.45 | **0.45** |

**Result:** Business wins with confidence 0.85

---

### Summary

| Component | Implementation | Technology |
|-----------|----------------|------------|
| Coordinator | `ThreadPoolExecutor` | Python |
| World Detector | Binary classifier | Claude Haiku 4.5 |
| Sports Detector | Binary classifier | Claude Haiku 4.5 |
| Business Detector | Binary classifier | Claude Haiku 4.5 |
| Sci/Tech Detector | Binary classifier | Claude Haiku 4.5 |
| Aggregator | Confidence voting | Python |

**Total LLM Calls per Article:** 4 (running in parallel)
**Expected Latency:** ~1 Haiku call (since all run concurrently)


In [20]:
# ============================================================
# PARALLELIZATION DESIGN PATTERN IMPLEMENTATION
# ============================================================

from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Rate limiting: Anthropic has 50 requests/minute limit
# With 4 parallel requests per article, we can process ~12 articles/minute
# Adding delay to stay under limit
REQUESTS_PER_ARTICLE = 4
RATE_LIMIT_PER_MINUTE = 50
MIN_SECONDS_PER_ARTICLE = (REQUESTS_PER_ARTICLE / RATE_LIMIT_PER_MINUTE) * 60 + 0.5  # ~5.3 seconds

# ============================================================
# WORLD DETECTOR AGENT
# ============================================================

WORLD_DETECTOR_SYSTEM_PROMPT = """You are a WORLD NEWS DETECTOR specializing in identifying international affairs and political news.

Your task is to determine if a news article belongs to the "World" category.

The "World" category includes:
- International politics and diplomacy
- Government policies, elections, and political events
- Military conflicts, wars, and peacekeeping
- Humanitarian crises and global health issues
- International organizations (UN, NATO, EU, WHO, etc.)
- Cross-border crime, terrorism, and security
- Immigration and refugee issues

The "World" category does NOT include:
- Sports events (even international ones like Olympics, World Cup)
- Business/financial news (even about international trade)
- Technology/science news (even about global tech companies)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is World/political news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}"""

def run_world_detector(article_text: str) -> dict:
    """Detect if article belongs to World category."""
    user_prompt = f"""Determine if the following news article belongs to the WORLD category:

---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=WORLD_DETECTOR_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'World'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", "category": "World"}

# ============================================================
# SPORTS DETECTOR AGENT
# ============================================================

SPORTS_DETECTOR_SYSTEM_PROMPT = """You are a SPORTS NEWS DETECTOR specializing in identifying athletic and competitive sports news.

Your task is to determine if a news article belongs to the "Sports" category.

The "Sports" category includes:
- Professional and amateur athletic competitions
- Sports teams, players, coaches, and athletes
- Game results, scores, standings, and statistics
- Sports leagues (NFL, NBA, MLB, NHL, FIFA, etc.)
- Olympic games and international sports competitions
- Sports trades, contracts, and team management
- Sports injuries and player updates

The "Sports" category does NOT include:
- Sports team stock prices or business deals (that's Business)
- Sports technology innovations (that's Sci/Tech)
- Sports-related politics or scandals (that's World)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is Sports news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}"""

def run_sports_detector(article_text: str) -> dict:
    """Detect if article belongs to Sports category."""
    user_prompt = f"""Determine if the following news article belongs to the SPORTS category:

---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=SPORTS_DETECTOR_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'Sports'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", "category": "Sports"}

# ============================================================
# BUSINESS DETECTOR AGENT
# ============================================================

BUSINESS_DETECTOR_SYSTEM_PROMPT = """You are a BUSINESS NEWS DETECTOR specializing in identifying financial and corporate news.

Your task is to determine if a news article belongs to the "Business" category.

The "Business" category includes:
- Stock markets, trading, and investments
- Company earnings, revenue, and financial reports
- Mergers, acquisitions, and corporate restructuring
- Economic indicators (GDP, inflation, employment)
- Corporate leadership and executive changes
- Industry analysis and market trends
- Banking, finance, and monetary policy

The "Business" category does NOT include:
- Tech product launches or innovations (that's Sci/Tech)
- Government economic policy debates (that's World)
- Sports team finances (that's Sports)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is Business/financial news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}"""

def run_business_detector(article_text: str) -> dict:
    """Detect if article belongs to Business category."""
    user_prompt = f"""Determine if the following news article belongs to the BUSINESS category:

---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=BUSINESS_DETECTOR_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'Business'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", "category": "Business"}

# ============================================================
# SCI/TECH DETECTOR AGENT
# ============================================================

SCITECH_DETECTOR_SYSTEM_PROMPT = """You are a SCIENCE & TECHNOLOGY NEWS DETECTOR specializing in identifying tech and scientific news.

Your task is to determine if a news article belongs to the "Sci/Tech" category.

The "Sci/Tech" category includes:
- Technology products, software, and hardware
- Scientific research and discoveries
- Internet, social media, and digital platforms
- AI, machine learning, and emerging technologies
- Space exploration and astronomy
- Medical and health research breakthroughs
- Environmental science and climate research
- Consumer electronics and gadgets

The "Sci/Tech" category does NOT include:
- Tech company stock prices or financials (that's Business)
- Tech regulation and government policy (that's World)
- Sports technology like stadium upgrades (that's Sports)

Analyze the article and return your assessment as JSON.

Rules:
- Focus ONLY on whether this is Science/Technology news
- Return a confidence score from 0.0 to 1.0
- Return ONLY valid JSON, no additional text

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Brief explanation"
}"""

def run_scitech_detector(article_text: str) -> dict:
    """Detect if article belongs to Sci/Tech category."""
    user_prompt = f"""Determine if the following news article belongs to the SCI/TECH category:

---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=SCITECH_DETECTOR_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'Sci/Tech'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", "category": "Sci/Tech"}

# ============================================================
# AGGREGATOR (Strategy B: Confidence Voting)
# ============================================================

def aggregate_detector_results(world_result: dict, sports_result: dict, 
                                business_result: dict, scitech_result: dict) -> dict:
    """Aggregate results from all 4 detectors using confidence voting (Strategy B)."""
    
    # Build confidence scores for each category
    # If belongs=True, use confidence; if belongs=False, use 0
    scores = {
        "World": world_result.get('confidence', 0) if world_result.get('belongs_to_category', False) else 0,
        "Sports": sports_result.get('confidence', 0) if sports_result.get('belongs_to_category', False) else 0,
        "Business": business_result.get('confidence', 0) if business_result.get('belongs_to_category', False) else 0,
        "Sci/Tech": scitech_result.get('confidence', 0) if scitech_result.get('belongs_to_category', False) else 0
    }
    
    # Handle edge case: all scores are 0 (no detector claimed the article)
    if all(s == 0 for s in scores.values()):
        # Fall back to highest confidence regardless of belongs_to_category
        fallback_scores = {
            "World": world_result.get('confidence', 0),
            "Sports": sports_result.get('confidence', 0),
            "Business": business_result.get('confidence', 0),
            "Sci/Tech": scitech_result.get('confidence', 0)
        }
        final_category = max(fallback_scores, key=fallback_scores.get)
        final_confidence = fallback_scores[final_category]
        used_fallback = True
    else:
        # Pick category with highest confidence score
        final_category = max(scores, key=scores.get)
        final_confidence = scores[final_category]
        used_fallback = False
    
    return {
        "classification": final_category,
        "confidence": final_confidence,
        "all_scores": scores,
        "used_fallback": used_fallback
    }

# ============================================================
# COORDINATOR (Parallel Execution)
# ============================================================

def parallel_classify(article_text: str) -> dict:
    """Coordinator: Run all 4 detectors in parallel and aggregate results."""
    
    # Define the detector functions
    detectors = [
        ("world", run_world_detector),
        ("sports", run_sports_detector),
        ("business", run_business_detector),
        ("scitech", run_scitech_detector)
    ]
    
    # Run all detectors in parallel
    results = {}
    with ThreadPoolExecutor(max_workers=4) as executor:
        # Submit all tasks
        future_to_detector = {
            executor.submit(func, article_text): name 
            for name, func in detectors
        }
        
        # Collect results
        for future in as_completed(future_to_detector):
            detector_name = future_to_detector[future]
            results[detector_name] = future.result()
    
    # Aggregate results
    aggregated = aggregate_detector_results(
        results['world'],
        results['sports'],
        results['business'],
        results['scitech']
    )
    
    return {
        "detector_results": results,
        "aggregated_result": aggregated
    }

# ============================================================
# PROCESS ALL TEST DATA
# ============================================================

# Store parallelization results
parallel_results = []

# Number of results to print
NUM_TO_PRINT = 3

print("=" * 70)
print("PARALLELIZATION DESIGN PATTERN - Classification")
print("=" * 70)
print(f"\nProcessing {NUM_SAMPLES} samples using 4 parallel detectors...")
print(f"Each article runs through World, Sports, Business, and Sci/Tech detectors simultaneously.")
print(f"⚠️  Rate limiting enabled: ~{MIN_SECONDS_PER_ARTICLE:.1f}s delay between articles to stay under API limits.\n")

# Progress tracking
PROGRESS_INTERVAL = 10

for i in range(NUM_SAMPLES):
    start_time = time.time()
    
    sample = test_data[i]
    article_text = sample['text']
    ground_truth_label = sample['label']
    
    # Run the parallel classification pipeline
    result = parallel_classify(article_text)
    
    # Store result with metadata
    parallel_results.append({
        "index": i,
        "original_text": article_text,
        "ground_truth_label": ground_truth_label,
        "ground_truth_name": label_names[ground_truth_label],
        "detector_results": result["detector_results"],
        "aggregated_result": result["aggregated_result"]
    })
    
    # Print progress
    if (i + 1) % PROGRESS_INTERVAL == 0 or (i + 1) == NUM_SAMPLES:
        print(f"  ✓ Processed {i + 1}/{NUM_SAMPLES} articles ({(i + 1) / NUM_SAMPLES * 100:.1f}%)")
    
    # Rate limiting: ensure minimum time between articles
    elapsed = time.time() - start_time
    if elapsed < MIN_SECONDS_PER_ARTICLE and i < NUM_SAMPLES - 1:
        sleep_time = MIN_SECONDS_PER_ARTICLE - elapsed
        time.sleep(sleep_time)

print("\n" + "=" * 70)
print(f"PARALLELIZATION RESULTS (showing first {NUM_TO_PRINT} of {len(parallel_results)})")
print("=" * 70)

# Display results
for result in parallel_results[:NUM_TO_PRINT]:
    print(f"\n{'─' * 70}")
    print(f"Article {result['index'] + 1}")
    print(f"Ground Truth: {result['ground_truth_name']}")
    print(f"{'─' * 70}")
    
    print(f"\n📊 Detector Results:")
    for detector_name, detector_result in result['detector_results'].items():
        belongs = detector_result.get('belongs_to_category', False)
        conf = detector_result.get('confidence', 0)
        symbol = "✓" if belongs else "✗"
        print(f"   {detector_name.upper():10} │ {symbol} belongs: {str(belongs):5} │ confidence: {conf:.2f}")
    
    print(f"\n📋 Aggregated Scores (Strategy B):")
    scores = result['aggregated_result']['all_scores']
    for cat, score in scores.items():
        bar = "█" * int(score * 20) + "░" * (20 - int(score * 20))
        print(f"   {cat:10} │ {bar} │ {score:.2f}")
    
    print(f"\n🎯 Final Classification: {result['aggregated_result']['classification']}")
    print(f"   Confidence: {result['aggregated_result']['confidence']:.2f}")
    if result['aggregated_result']['used_fallback']:
        print(f"   (Used fallback - no detector claimed the article)")
    
    # Quick comparison
    predicted = result['aggregated_result']['classification']
    actual = result['ground_truth_name']
    match = "✓ CORRECT" if predicted == actual else "✗ INCORRECT"
    print(f"\n   Result: {match}")

print(f"\n{'=' * 70}")
print(f"Parallelization complete. {len(parallel_results)} results stored in 'parallel_results'")
print(f"Ready for evaluation.")
print(f"{'=' * 70}")


PARALLELIZATION DESIGN PATTERN - Classification

Processing 100 samples using 4 parallel detectors...
Each article runs through World, Sports, Business, and Sci/Tech detectors simultaneously.
⚠️  Rate limiting enabled: ~5.3s delay between articles to stay under API limits.

  ✓ Processed 10/100 articles (10.0%)
  ✓ Processed 20/100 articles (20.0%)
  ✓ Processed 30/100 articles (30.0%)
  ✓ Processed 40/100 articles (40.0%)
  ✓ Processed 50/100 articles (50.0%)
  ✓ Processed 60/100 articles (60.0%)
  ✓ Processed 70/100 articles (70.0%)
  ✓ Processed 80/100 articles (80.0%)
  ✓ Processed 90/100 articles (90.0%)
  ✓ Processed 100/100 articles (100.0%)

PARALLELIZATION RESULTS (showing first 3 of 100)

──────────────────────────────────────────────────────────────────────
Article 1
Ground Truth: Business
──────────────────────────────────────────────────────────────────────

📊 Detector Results:
   BUSINESS   │ ✓ belongs: True  │ confidence: 0.85
   WORLD      │ ✗ belongs: False │ confidenc

In [21]:
# ============================================================
# EVALUATION: Parallelization Design Pattern Performance
# ============================================================

# Extract predictions and ground truth from parallelization results
parallel_predictions = []
parallel_ground_truths = []
parallel_errors = []

for result in parallel_results:
    ground_truth = result['ground_truth_name']
    parallel_ground_truths.append(ground_truth)
    
    # Get prediction from aggregated_result
    aggregated = result['aggregated_result']
    
    if 'classification' in aggregated:
        predicted = aggregated['classification']
        parallel_predictions.append(predicted)
    else:
        # Handle errors
        parallel_predictions.append("ERROR")
        parallel_errors.append(result['index'])

print("=" * 70)
print("PARALLELIZATION DESIGN PATTERN - EVALUATION RESULTS")
print("=" * 70)

# Check for errors
if parallel_errors:
    print(f"\n⚠️  Warning: {len(parallel_errors)} samples had errors (indices: {parallel_errors[:10]}{'...' if len(parallel_errors) > 10 else ''})")
    print(f"   These are excluded from metrics calculation.\n")
    
    valid_parallel_predictions = [p for p in parallel_predictions if p != "ERROR"]
    valid_parallel_ground_truths = [g for p, g in zip(parallel_predictions, parallel_ground_truths) if p != "ERROR"]
else:
    valid_parallel_predictions = parallel_predictions
    valid_parallel_ground_truths = parallel_ground_truths

# ============================================================
# Basic Metrics
# ============================================================

print("\n" + "─" * 70)
print("OVERALL METRICS")
print("─" * 70)

parallel_accuracy = accuracy_score(valid_parallel_ground_truths, valid_parallel_predictions)
parallel_precision_macro = precision_score(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES, average='macro', zero_division=0)
parallel_recall_macro = recall_score(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES, average='macro', zero_division=0)
parallel_f1_macro = f1_score(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES, average='macro', zero_division=0)

parallel_precision_weighted = precision_score(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
parallel_recall_weighted = recall_score(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
parallel_f1_weighted = f1_score(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES, average='weighted', zero_division=0)

print(f"\n  Total Samples:     {len(parallel_results)}")
print(f"  Valid Predictions: {len(valid_parallel_predictions)}")
print(f"  Error Rate:        {len(parallel_errors) / len(parallel_results) * 100:.1f}%")

print(f"\n  📊 Accuracy:        {parallel_accuracy:.4f}  ({parallel_accuracy * 100:.1f}%)")
print(f"\n  Macro Averages (treats all classes equally):")
print(f"     Precision:      {parallel_precision_macro:.4f}")
print(f"     Recall:         {parallel_recall_macro:.4f}")
print(f"     F1 Score:       {parallel_f1_macro:.4f}")

print(f"\n  Weighted Averages (accounts for class imbalance):")
print(f"     Precision:      {parallel_precision_weighted:.4f}")
print(f"     Recall:         {parallel_recall_weighted:.4f}")
print(f"     F1 Score:       {parallel_f1_weighted:.4f}")

# ============================================================
# Per-Class Metrics
# ============================================================

print("\n" + "─" * 70)
print("PER-CLASS METRICS")
print("─" * 70)

print("\n" + classification_report(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES, zero_division=0))

# ============================================================
# Confusion Matrix
# ============================================================

print("─" * 70)
print("CONFUSION MATRIX")
print("─" * 70)

parallel_cm = confusion_matrix(valid_parallel_ground_truths, valid_parallel_predictions, labels=CATEGORIES)

print(f"\n{'Predicted →':>15}", end="")
for cat in CATEGORIES:
    print(f"{cat:>10}", end="")
print("\n" + "─" * 55)

print("Actual ↓")
for i, cat in enumerate(CATEGORIES):
    print(f"{cat:>15}", end="")
    for j in range(len(CATEGORIES)):
        print(f"{parallel_cm[i][j]:>10}", end="")
    print()

# ============================================================
# Distribution Analysis
# ============================================================

print("\n" + "─" * 70)
print("DISTRIBUTION ANALYSIS")
print("─" * 70)

parallel_gt_dist = Counter(valid_parallel_ground_truths)
parallel_pred_dist = Counter(valid_parallel_predictions)

print(f"\n{'Category':<12} {'Ground Truth':>15} {'Predicted':>15} {'Difference':>12}")
print("─" * 55)
for cat in CATEGORIES:
    gt_count = parallel_gt_dist.get(cat, 0)
    pred_count = parallel_pred_dist.get(cat, 0)
    diff = pred_count - gt_count
    diff_str = f"+{diff}" if diff > 0 else str(diff)
    print(f"{cat:<12} {gt_count:>15} {pred_count:>15} {diff_str:>12}")

# ============================================================
# Detector Analysis (Unique to Parallelization)
# ============================================================

print("\n" + "─" * 70)
print("DETECTOR ANALYSIS")
print("─" * 70)

# Count how many detectors claimed each article
detector_claim_counts = Counter()
for result in parallel_results:
    claims = sum(1 for det_result in result['detector_results'].values() 
                 if det_result.get('belongs_to_category', False))
    detector_claim_counts[claims] += 1

print(f"\n  Number of Detectors Claiming Article:")
print(f"  {'Claims':>10} {'Count':>10} {'Percentage':>12}")
print("  " + "─" * 35)
for num_claims in sorted(detector_claim_counts.keys()):
    count = detector_claim_counts[num_claims]
    pct = count / len(parallel_results) * 100
    print(f"  {num_claims:>10} {count:>10} {pct:>11.1f}%")

# Fallback usage
fallback_count = sum(1 for r in parallel_results if r['aggregated_result'].get('used_fallback', False))
print(f"\n  Fallback Used (no detector claimed): {fallback_count} ({fallback_count / len(parallel_results) * 100:.1f}%)")

# Per-detector accuracy (when that detector claimed and was correct)
print(f"\n  Per-Detector Claim Statistics:")
print(f"  {'Detector':<12} {'Claimed':>10} {'Correct':>10} {'Accuracy':>12}")
print("  " + "─" * 47)

detector_stats = {
    'world': {'claimed': 0, 'correct': 0},
    'sports': {'claimed': 0, 'correct': 0},
    'business': {'claimed': 0, 'correct': 0},
    'scitech': {'claimed': 0, 'correct': 0}
}
category_map = {'world': 'World', 'sports': 'Sports', 'business': 'Business', 'scitech': 'Sci/Tech'}

for result in parallel_results:
    for det_name, det_result in result['detector_results'].items():
        if det_result.get('belongs_to_category', False):
            detector_stats[det_name]['claimed'] += 1
            if category_map[det_name] == result['ground_truth_name']:
                detector_stats[det_name]['correct'] += 1

for det_name in ['world', 'sports', 'business', 'scitech']:
    claimed = detector_stats[det_name]['claimed']
    correct = detector_stats[det_name]['correct']
    acc = correct / claimed if claimed > 0 else 0
    print(f"  {det_name.upper():<12} {claimed:>10} {correct:>10} {acc:>11.1%}")

# ============================================================
# Confidence Analysis
# ============================================================

print("\n" + "─" * 70)
print("CONFIDENCE ANALYSIS")
print("─" * 70)

# Bin confidence scores
confidence_bins = {"high (0.8-1.0)": [0, 0], "medium (0.5-0.8)": [0, 0], "low (0-0.5)": [0, 0]}

for result in parallel_results:
    conf = result['aggregated_result'].get('confidence', 0)
    predicted = result['aggregated_result'].get('classification', '')
    is_correct = predicted == result['ground_truth_name']
    
    if conf >= 0.8:
        bin_name = "high (0.8-1.0)"
    elif conf >= 0.5:
        bin_name = "medium (0.5-0.8)"
    else:
        bin_name = "low (0-0.5)"
    
    confidence_bins[bin_name][1] += 1
    if is_correct:
        confidence_bins[bin_name][0] += 1

print(f"\n{'Confidence':<20} {'Count':>10} {'Accuracy':>12}")
print("─" * 45)
for bin_name in ["high (0.8-1.0)", "medium (0.5-0.8)", "low (0-0.5)"]:
    correct, total = confidence_bins[bin_name]
    if total > 0:
        acc = correct / total
        print(f"{bin_name:<20} {total:>10} {acc:>11.1%}")
    else:
        print(f"{bin_name:<20} {total:>10} {'N/A':>12}")

# Average confidence
avg_confidence = sum(r['aggregated_result'].get('confidence', 0) for r in parallel_results) / len(parallel_results)
print(f"\n  Average Confidence: {avg_confidence:.3f}")

print("\n" + "=" * 70)
print("PARALLELIZATION PATTERN EVALUATION COMPLETE")
print("=" * 70)


PARALLELIZATION DESIGN PATTERN - EVALUATION RESULTS

──────────────────────────────────────────────────────────────────────
OVERALL METRICS
──────────────────────────────────────────────────────────────────────

  Total Samples:     100
  Valid Predictions: 100
  Error Rate:        0.0%

  📊 Accuracy:        0.7200  (72.0%)

  Macro Averages (treats all classes equally):
     Precision:      0.7180
     Recall:         0.7722
     F1 Score:       0.7167

  Weighted Averages (accounts for class imbalance):
     Precision:      0.7692
     Recall:         0.7200
     F1 Score:       0.7204

──────────────────────────────────────────────────────────────────────
PER-CLASS METRICS
──────────────────────────────────────────────────────────────────────

              precision    recall  f1-score   support

       World       0.88      0.70      0.78        30
      Sports       0.63      0.90      0.75        21
    Business       0.52      0.92      0.67        12
    Sci/Tech       0.84   

## 4. Orchestrator-Worker Design Pattern

### What is Orchestrator-Worker?

**Orchestrator-Worker** is an agentic workflow pattern where an LLM orchestrator dynamically coordinates multiple LLM workers, and an LLM synthesizer combines their outputs into a final result. Unlike parallelization (which uses code for coordination and aggregation), orchestrator-worker leverages LLM reasoning at every stage.

This pattern is particularly useful when:
- Tasks require dynamic, context-aware coordination
- Simple voting/aggregation rules are insufficient
- Conflict resolution requires reasoning, not just math
- Full explainability and audit trails are required
- Edge cases and ambiguities are common

### Key Components

1. **Orchestrator (LLM)**: Analyzes the input and creates customized instructions for each worker
2. **Workers (LLMs)**: Execute subtasks with guidance from the orchestrator
3. **Synthesizer (LLM)**: Reasons over all worker outputs to produce the final result

### How It Differs from Other Patterns

| Pattern | Coordination | Workers | Aggregation |
|---------|--------------|---------|-------------|
| **Prompt Chaining** | Sequential code | 1 LLM per step | N/A (pipeline) |
| **Routing** | LLM router | 1 LLM selected | N/A (single path) |
| **Parallelization** | Code | Multiple LLMs | Code (voting) |
| **Orchestrator-Worker** | LLM | Multiple LLMs | LLM (synthesis) |

---

### Generic Orchestrator-Worker Workflow Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ORCHESTRATOR-WORKER DESIGN PATTERN                       │
└─────────────────────────────────────────────────────────────────────────────┘

┌──────────────┐
│              │
│    INPUT     │
│              │
└──────┬───────┘
       │
       ▼
┌──────────────────────────────────────────────────────────────────┐
│                      🎯 ORCHESTRATOR (LLM)                       │
│                                                                  │
│  Responsibilities:                                               │
│  • Analyze the input task                                       │
│  • Decompose into subtasks                                      │
│  • Create CUSTOMIZED instructions for each worker               │
│  • Identify potential challenges or ambiguities                 │
│                                                                  │
│  Output: Task-specific guidance for each worker                 │
└──────────────────────────────┬───────────────────────────────────┘
                               │
                               │ Customized Instructions
                               │
       ┌───────────────────────┼───────────────────────┐
       │                       │                       │
       ▼                       ▼                       ▼
┌─────────────────┐   ┌─────────────────┐   ┌─────────────────┐
│                 │   │                 │   │                 │
│   WORKER A      │   │   WORKER B      │   │   WORKER C      │
│     (LLM)       │   │     (LLM)       │   │     (LLM)       │
│                 │   │                 │   │                 │
│  Receives:      │   │  Receives:      │   │  Receives:      │
│  • Original     │   │  • Original     │   │  • Original     │
│    input        │   │    input        │   │    input        │
│  • Custom       │   │  • Custom       │   │  • Custom       │
│    instructions │   │    instructions │   │    instructions │
│                 │   │                 │   │                 │
│  Returns:       │   │  Returns:       │   │  Returns:       │
│  • Analysis     │   │  • Analysis     │   │  • Analysis     │
│  • Reasoning    │   │  • Reasoning    │   │  • Reasoning    │
│                 │   │                 │   │                 │
└────────┬────────┘   └────────┬────────┘   └────────┬────────┘
         │                     │                     │
         │     All workers can run in PARALLEL      │
         │                     │                     │
         └─────────────────────┼─────────────────────┘
                               │
                               │ All Worker Outputs
                               │
                               ▼
┌──────────────────────────────────────────────────────────────────┐
│                      🔮 SYNTHESIZER (LLM)                        │
│                                                                  │
│  Responsibilities:                                               │
│  • Review ALL worker outputs                                    │
│  • Analyze agreements and conflicts                             │
│  • Reason through discrepancies                                 │
│  • Make final decision with full explanation                    │
│  • Can override workers if reasoning is flawed                  │
│                                                                  │
│  Output: Final result with reasoning                            │
└──────────────────────────────┬───────────────────────────────────┘
                               │
                               ▼
                        ┌──────────────┐
                        │              │
                        │    OUTPUT    │
                        │              │
                        └──────────────┘
```

---

### Advantages of Orchestrator-Worker

| Aspect | Benefit |
|--------|---------|
| **Adaptive Coordination** | Orchestrator tailors instructions based on input |
| **Intelligent Synthesis** | Synthesizer can reason through conflicts |
| **Context Awareness** | Each stage can adapt to the specific situation |
| **Explainability** | Full reasoning trail at every step |
| **Edge Case Handling** | LLM reasoning handles nuances better than rules |

### Trade-offs

| Aspect | Consideration |
|--------|---------------|
| **Cost** | More LLM calls = higher API costs |
| **Latency** | Sequential stages add latency |
| **Complexity** | More components to design and debug |
| **Consistency** | LLM reasoning can vary between runs |

---

### Execution Flow

```
┌─────────┐    ┌──────────────┐    ┌─────────┐    ┌─────────────┐    ┌────────┐
│  INPUT  │───▶│ ORCHESTRATOR │───▶│ WORKERS │───▶│ SYNTHESIZER │───▶│ OUTPUT │
└─────────┘    └──────────────┘    └─────────┘    └─────────────┘    └────────┘
                    Stage 1         Stage 2           Stage 3
                  (1 LLM call)   (N parallel calls)  (1 LLM call)

Total LLM Calls: 1 + N + 1 = N + 2
Total Latency: ~3 sequential stages (orchestrator → workers → synthesizer)
```


### Our Implementation: Orchestrator-Worker for AG News Classification

For our AG News classification task, we'll implement the orchestrator-worker pattern with an LLM orchestrator, 4 category workers, and an LLM synthesizer.

---

#### Architecture Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│           ORCHESTRATOR-WORKER FOR AG NEWS CLASSIFICATION                    │
└─────────────────────────────────────────────────────────────────────────────┘

┌──────────────┐
│  NEWS        │
│  ARTICLE     │
└──────┬───────┘
       │
       ▼
┌──────────────────────────────────────────────────────────────────┐
│                 🎯 ORCHESTRATOR (gpt-5-mini)                     │
│                                                                  │
│  • Analyzes the article content                                 │
│  • Identifies key entities, topics, and signals                 │
│  • Creates CUSTOMIZED hints for each worker                     │
│  • Flags potential ambiguities                                  │
│                                                                  │
└──────────────────────────────┬───────────────────────────────────┘
                               │
           ┌───────────────────┼───────────────────┐
           │                   │                   │
           ▼                   ▼                   ▼
┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
│ 🌍 WORLD        │  │ ⚽ SPORTS       │  │ 💼 BUSINESS     │  │ 🔬 SCI/TECH     │
│    WORKER       │  │    WORKER       │  │    WORKER       │  │    WORKER       │
│                 │  │                 │  │                 │  │                 │
│ claude-haiku    │  │ claude-haiku    │  │ claude-haiku    │  │ claude-haiku    │
│ -4-5-20251001   │  │ -4-5-20251001   │  │ -4-5-20251001   │  │ -4-5-20251001   │
│                 │  │                 │  │                 │  │                 │
│ Input:          │  │ Input:          │  │ Input:          │  │ Input:          │
│ • Article       │  │ • Article       │  │ • Article       │  │ • Article       │
│ • Custom hint   │  │ • Custom hint   │  │ • Custom hint   │  │ • Custom hint   │
│   from orch.    │  │   from orch.    │  │   from orch.    │  │   from orch.    │
└────────┬────────┘  └────────┬────────┘  └────────┬────────┘  └────────┬────────┘
         │                    │                    │                    │
         │              PARALLEL EXECUTION         │                    │
         │                    │                    │                    │
         └────────────────────┼────────────────────┴────────────────────┘
                              │
                              ▼
┌──────────────────────────────────────────────────────────────────┐
│                   🔮 SYNTHESIZER (gpt-5.1)                       │
│                                                                  │
│  • Reviews orchestrator analysis                                │
│  • Examines all 4 worker outputs                                │
│  • Reasons through agreements/conflicts                         │
│  • Makes final classification with explanation                  │
│                                                                  │
└──────────────────────────────┬───────────────────────────────────┘
                               │
                               ▼
                        ┌──────────────┐
                        │   FINAL      │
                        │   CATEGORY   │
                        └──────────────┘
```

---

#### Models Used

| Component | Model | Rationale |
|-----------|-------|-----------|
| Orchestrator | `gpt-5-mini` | Fast, good at analysis and instruction generation |
| World Worker | `claude-haiku-4-5-20251001` | Fast, cost-effective for detection |
| Sports Worker | `claude-haiku-4-5-20251001` | Fast, cost-effective for detection |
| Business Worker | `claude-haiku-4-5-20251001` | Fast, cost-effective for detection |
| Sci/Tech Worker | `claude-haiku-4-5-20251001` | Fast, cost-effective for detection |
| Synthesizer | `gpt-5.1` | Powerful reasoning for conflict resolution |

**Total LLM Calls per Article:** 6 (1 orchestrator + 4 workers + 1 synthesizer)

---

### Orchestrator (gpt-5-mini) 🎯

#### Input

Raw news article text (same as other patterns).

#### System Prompt

```
You are an intelligent orchestrator for a news classification system.

Your task is to analyze a news article and prepare customized instructions for 4 specialized workers who will each evaluate whether the article belongs to their category:
- World Worker: Evaluates World/Politics news
- Sports Worker: Evaluates Sports news
- Business Worker: Evaluates Business/Finance news
- Sci/Tech Worker: Evaluates Science/Technology news

Analyze the article and provide:
1. A brief summary of the article's content
2. Key entities mentioned (people, organizations, locations)
3. Primary topics and themes
4. Which categories seem most likely and why
5. CUSTOMIZED hints for each worker - what specific aspects they should focus on

Your hints should help workers make better decisions by:
- Highlighting relevant evidence for their category
- Warning about potential false positives (e.g., "Apple" could be tech OR business)
- Pointing out ambiguities they should address

Return your analysis as JSON.

Output format:
{
    "summary": "Brief 1-2 sentence summary",
    "key_entities": ["entity1", "entity2", ...],
    "primary_topics": ["topic1", "topic2", ...],
    "likely_categories": ["Category1", "Category2"],
    "ambiguity_notes": "Any potential confusion between categories",
    "worker_hints": {
        "world": "Specific guidance for World worker",
        "sports": "Specific guidance for Sports worker",
        "business": "Specific guidance for Business worker",
        "scitech": "Specific guidance for Sci/Tech worker"
    }
}

Rules:
- Be concise but informative
- Provide actionable hints, not just descriptions
- Return ONLY valid JSON, no additional text
```

#### User Prompt Format

```
Analyze the following news article and prepare instructions for the classification workers:

---
[Article text]
---
```

#### Output Format

```json
{
    "summary": "Apple announces new iPhone with advanced AI features at product launch event",
    "key_entities": ["Apple", "iPhone", "Tim Cook"],
    "primary_topics": ["product launch", "AI technology", "smartphones"],
    "likely_categories": ["Sci/Tech", "Business"],
    "ambiguity_notes": "Could be Sci/Tech (product features) or Business (company announcement). Focus on substance.",
    "worker_hints": {
        "world": "No political/government angle apparent. Quick assessment sufficient.",
        "sports": "No sports connection. Quick negative assessment.",
        "business": "Apple is a company, but check if focus is on stock/financials or product features. If product-focused, likely Sci/Tech.",
        "scitech": "Primary candidate. Article describes technical features (AI, processor). Evaluate if tech substance dominates over business aspects."
    }
}
```

---

### Workers (claude-haiku-4-5-20251001) 🌍⚽💼🔬

All 4 workers share the same structure but with category-specific system prompts.

#### Input

- Original article text
- Orchestrator's customized hint for this worker

#### System Prompt (World Worker Example)

```
You are a WORLD NEWS DETECTOR working as part of a classification team.

Your task is to determine if a news article belongs to the "World" category.

The "World" category includes:
- International politics and diplomacy
- Government policies, elections, and political events
- Military conflicts, wars, and peacekeeping
- Humanitarian crises and global health issues
- International organizations (UN, NATO, EU, WHO, etc.)

You will receive:
1. The news article
2. A hint from the orchestrator with specific guidance for your evaluation

Use the orchestrator's hint to focus your analysis, but make your own independent judgment.

Return your assessment as JSON.

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Your analysis explaining your decision",
    "orchestrator_hint_addressed": "How you considered the orchestrator's guidance"
}

Rules:
- Make an independent judgment based on the article content
- Use the orchestrator's hint as guidance, not as the answer
- Return ONLY valid JSON, no additional text
```

#### User Prompt Format

```
Evaluate if this article belongs to the WORLD category.

ORCHESTRATOR'S HINT FOR YOU:
[Orchestrator's hint for this worker]

ARTICLE:
---
[Article text]
---
```

#### Output Format

```json
{
    "belongs_to_category": false,
    "confidence": 0.15,
    "reasoning": "Article focuses on Apple's product announcement with AI features. No political, diplomatic, or international affairs content present.",
    "orchestrator_hint_addressed": "Orchestrator correctly identified no political angle. Confirmed with quick assessment."
}
```

---

### Synthesizer (gpt-5.1) 🔮

#### Input

- Original article text
- Orchestrator's full analysis
- All 4 worker outputs

#### System Prompt

```
You are the final decision-maker in a news classification system.

You will receive:
1. The original news article
2. The orchestrator's analysis (summary, key entities, likely categories, hints given)
3. Outputs from 4 specialized workers (World, Sports, Business, Sci/Tech)

Each worker has evaluated whether the article belongs to their category and provided:
- belongs_to_category: true/false
- confidence: 0.0-1.0
- reasoning: their analysis

Your task is to synthesize all this information and make the FINAL classification decision.

Consider:
- Which workers claimed the article and with what confidence?
- Are there conflicts? If so, whose reasoning is more compelling?
- Does the orchestrator's analysis align with worker assessments?
- What is the PRIMARY focus of the article?

You may OVERRIDE a worker's claim if their reasoning is flawed.

Return your final decision as JSON.

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "reasoning": "Detailed explanation of your decision",
    "worker_agreement_summary": "Which workers agreed/disagreed and why",
    "conflict_resolution": "If there were conflicts, how you resolved them",
    "orchestrator_alignment": "Whether your decision aligns with orchestrator's prediction"
}

Rules:
- You MUST choose exactly ONE category
- Provide clear reasoning for your decision
- If overriding a worker, explain why
- Return ONLY valid JSON, no additional text
```

#### User Prompt Format

```
Make the final classification decision based on all available information.

ORIGINAL ARTICLE:
---
[Article text]
---

ORCHESTRATOR ANALYSIS:
[JSON of orchestrator output]

WORKER OUTPUTS:

WORLD WORKER:
[JSON of world worker output]

SPORTS WORKER:
[JSON of sports worker output]

BUSINESS WORKER:
[JSON of business worker output]

SCI/TECH WORKER:
[JSON of scitech worker output]
```

#### Output Format

```json
{
    "classification": "Sci/Tech",
    "confidence": "high",
    "reasoning": "The article's primary focus is on technological features - AI capabilities, processor improvements, and product functionality. While Apple is a business entity, the substance of this article is about technology, not financials or corporate strategy.",
    "worker_agreement_summary": "Sci/Tech worker (claimed, 0.92 confidence) and Business worker (claimed, 0.65 confidence) both identified relevant signals. Sports and World workers correctly declined.",
    "conflict_resolution": "Both Sci/Tech and Business workers claimed the article. Sci/Tech worker's reasoning about technical features being the primary focus is more compelling than Business worker's reasoning about company announcement. The article describes WHAT the product does, not its financial implications.",
    "orchestrator_alignment": "Aligned. Orchestrator identified Sci/Tech as primary candidate and correctly noted the tech-vs-business ambiguity."
}
```

---

### Summary Table

| Component | Model | Input | Output |
|-----------|-------|-------|--------|
| **Orchestrator** | `gpt-5-mini` | Article text | Analysis + worker hints |
| **World Worker** | `claude-haiku-4-5-20251001` | Article + hint | Detection result |
| **Sports Worker** | `claude-haiku-4-5-20251001` | Article + hint | Detection result |
| **Business Worker** | `claude-haiku-4-5-20251001` | Article + hint | Detection result |
| **Sci/Tech Worker** | `claude-haiku-4-5-20251001` | Article + hint | Detection result |
| **Synthesizer** | `gpt-5.1` | Article + orchestrator + all workers | Final classification |

**Execution Flow:**
1. Orchestrator analyzes article → produces hints (1 LLM call)
2. 4 Workers evaluate in parallel → produce assessments (4 LLM calls, parallel)
3. Synthesizer makes final decision → produces classification (1 LLM call)

**Total: 6 LLM calls per article** (with workers running in parallel)


In [23]:
# ============================================================
# ORCHESTRATOR-WORKER DESIGN PATTERN IMPLEMENTATION
# ============================================================

from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Rate limiting: 4 Anthropic calls per article (workers), 2 OpenAI calls (orchestrator + synthesizer)
# Anthropic has 50 requests/minute limit
# With 4 parallel Anthropic requests per article, we can process ~12 articles/minute
REQUESTS_PER_ARTICLE_ANTHROPIC = 4
RATE_LIMIT_PER_MINUTE = 50
MIN_SECONDS_PER_ARTICLE = (REQUESTS_PER_ARTICLE_ANTHROPIC / RATE_LIMIT_PER_MINUTE) * 60 + 0.5  # ~5.3 seconds

# ============================================================
# ORCHESTRATOR (gpt-5-mini)
# ============================================================

ORCHESTRATOR_SYSTEM_PROMPT = """You are an intelligent orchestrator for a news classification system.

Your task is to analyze a news article and prepare customized instructions for 4 specialized workers who will each evaluate whether the article belongs to their category:
- World Worker: Evaluates World/Politics news
- Sports Worker: Evaluates Sports news
- Business Worker: Evaluates Business/Finance news
- Sci/Tech Worker: Evaluates Science/Technology news

Analyze the article and provide:
1. A brief summary of the article's content
2. Key entities mentioned (people, organizations, locations)
3. Primary topics and themes
4. Which categories seem most likely and why
5. CUSTOMIZED hints for each worker - what specific aspects they should focus on

Your hints should help workers make better decisions by:
- Highlighting relevant evidence for their category
- Warning about potential false positives (e.g., "Apple" could be tech OR business)
- Pointing out ambiguities they should address

Return your analysis as JSON.

Output format:
{
    "summary": "Brief 1-2 sentence summary",
    "key_entities": ["entity1", "entity2", ...],
    "primary_topics": ["topic1", "topic2", ...],
    "likely_categories": ["Category1", "Category2"],
    "ambiguity_notes": "Any potential confusion between categories",
    "worker_hints": {
        "world": "Specific guidance for World worker",
        "sports": "Specific guidance for Sports worker",
        "business": "Specific guidance for Business worker",
        "scitech": "Specific guidance for Sci/Tech worker"
    }
}

Rules:
- Be concise but informative
- Provide actionable hints, not just descriptions
- Return ONLY valid JSON, no additional text"""

def run_orchestrator(article_text: str) -> dict:
    """Run the orchestrator to analyze article and generate worker hints."""
    user_prompt = f"""Analyze the following news article and prepare instructions for the classification workers:

---
{article_text}
---"""
    
    response = openai_client.chat.completions.create(
        model="gpt-5-mini",
        max_completion_tokens=2048,
        messages=[
            {"role": "system", "content": ORCHESTRATOR_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    response_text = response.choices[0].message.content
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {
            "error": "Failed to parse orchestrator response",
            "raw_response": response_text,
            "summary": "",
            "key_entities": [],
            "primary_topics": [],
            "likely_categories": [],
            "ambiguity_notes": "",
            "worker_hints": {
                "world": "Evaluate if this is World news.",
                "sports": "Evaluate if this is Sports news.",
                "business": "Evaluate if this is Business news.",
                "scitech": "Evaluate if this is Sci/Tech news."
            }
        }

# ============================================================
# WORLD WORKER (claude-haiku-4-5-20251001)
# ============================================================

WORLD_WORKER_SYSTEM_PROMPT = """You are a WORLD NEWS DETECTOR working as part of a classification team.

Your task is to determine if a news article belongs to the "World" category.

The "World" category includes:
- International politics and diplomacy
- Government policies, elections, and political events
- Military conflicts, wars, and peacekeeping
- Humanitarian crises and global health issues
- International organizations (UN, NATO, EU, WHO, etc.)

You will receive:
1. The news article
2. A hint from the orchestrator with specific guidance for your evaluation

Use the orchestrator's hint to focus your analysis, but make your own independent judgment.

Return your assessment as JSON.

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Your analysis explaining your decision",
    "orchestrator_hint_addressed": "How you considered the orchestrator's guidance"
}

Rules:
- Make an independent judgment based on the article content
- Use the orchestrator's hint as guidance, not as the answer
- Return ONLY valid JSON, no additional text"""

def run_world_worker(article_text: str, orchestrator_hint: str) -> dict:
    """World worker: Detect if article belongs to World category."""
    user_prompt = f"""Evaluate if this article belongs to the WORLD category.

ORCHESTRATOR'S HINT FOR YOU:
{orchestrator_hint}

ARTICLE:
---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=WORLD_WORKER_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'World'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", 
                "orchestrator_hint_addressed": "N/A", "category": "World"}

# ============================================================
# SPORTS WORKER (claude-haiku-4-5-20251001)
# ============================================================

SPORTS_WORKER_SYSTEM_PROMPT = """You are a SPORTS NEWS DETECTOR working as part of a classification team.

Your task is to determine if a news article belongs to the "Sports" category.

The "Sports" category includes:
- Professional and amateur athletic competitions
- Sports teams, players, coaches, and athletes
- Game results, scores, standings, and statistics
- Sports leagues (NFL, NBA, MLB, NHL, FIFA, etc.)
- Olympic games and international sports competitions
- Sports trades, contracts, and team management

You will receive:
1. The news article
2. A hint from the orchestrator with specific guidance for your evaluation

Use the orchestrator's hint to focus your analysis, but make your own independent judgment.

Return your assessment as JSON.

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Your analysis explaining your decision",
    "orchestrator_hint_addressed": "How you considered the orchestrator's guidance"
}

Rules:
- Make an independent judgment based on the article content
- Use the orchestrator's hint as guidance, not as the answer
- Return ONLY valid JSON, no additional text"""

def run_sports_worker(article_text: str, orchestrator_hint: str) -> dict:
    """Sports worker: Detect if article belongs to Sports category."""
    user_prompt = f"""Evaluate if this article belongs to the SPORTS category.

ORCHESTRATOR'S HINT FOR YOU:
{orchestrator_hint}

ARTICLE:
---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=SPORTS_WORKER_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'Sports'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", 
                "orchestrator_hint_addressed": "N/A", "category": "Sports"}

# ============================================================
# BUSINESS WORKER (claude-haiku-4-5-20251001)
# ============================================================

BUSINESS_WORKER_SYSTEM_PROMPT = """You are a BUSINESS NEWS DETECTOR working as part of a classification team.

Your task is to determine if a news article belongs to the "Business" category.

The "Business" category includes:
- Stock markets, trading, and investments
- Company earnings, revenue, and financial reports
- Mergers, acquisitions, and corporate restructuring
- Economic indicators (GDP, inflation, employment)
- Corporate leadership and executive changes
- Industry analysis and market trends
- Banking, finance, and monetary policy

You will receive:
1. The news article
2. A hint from the orchestrator with specific guidance for your evaluation

Use the orchestrator's hint to focus your analysis, but make your own independent judgment.

Return your assessment as JSON.

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Your analysis explaining your decision",
    "orchestrator_hint_addressed": "How you considered the orchestrator's guidance"
}

Rules:
- Make an independent judgment based on the article content
- Use the orchestrator's hint as guidance, not as the answer
- Return ONLY valid JSON, no additional text"""

def run_business_worker(article_text: str, orchestrator_hint: str) -> dict:
    """Business worker: Detect if article belongs to Business category."""
    user_prompt = f"""Evaluate if this article belongs to the BUSINESS category.

ORCHESTRATOR'S HINT FOR YOU:
{orchestrator_hint}

ARTICLE:
---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=BUSINESS_WORKER_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'Business'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", 
                "orchestrator_hint_addressed": "N/A", "category": "Business"}

# ============================================================
# SCI/TECH WORKER (claude-haiku-4-5-20251001)
# ============================================================

SCITECH_WORKER_SYSTEM_PROMPT = """You are a SCIENCE & TECHNOLOGY NEWS DETECTOR working as part of a classification team.

Your task is to determine if a news article belongs to the "Sci/Tech" category.

The "Sci/Tech" category includes:
- Technology products, software, and hardware
- Scientific research and discoveries
- Internet, social media, and digital platforms
- AI, machine learning, and emerging technologies
- Space exploration and astronomy
- Medical and health research breakthroughs
- Environmental science and climate research
- Consumer electronics and gadgets

You will receive:
1. The news article
2. A hint from the orchestrator with specific guidance for your evaluation

Use the orchestrator's hint to focus your analysis, but make your own independent judgment.

Return your assessment as JSON.

Output format:
{
    "belongs_to_category": true | false,
    "confidence": 0.0 to 1.0,
    "reasoning": "Your analysis explaining your decision",
    "orchestrator_hint_addressed": "How you considered the orchestrator's guidance"
}

Rules:
- Make an independent judgment based on the article content
- Use the orchestrator's hint as guidance, not as the answer
- Return ONLY valid JSON, no additional text"""

def run_scitech_worker(article_text: str, orchestrator_hint: str) -> dict:
    """Sci/Tech worker: Detect if article belongs to Sci/Tech category."""
    user_prompt = f"""Evaluate if this article belongs to the SCI/TECH category.

ORCHESTRATOR'S HINT FOR YOU:
{orchestrator_hint}

ARTICLE:
---
{article_text}
---"""
    
    response = anthropic_client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=SCITECH_WORKER_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        result = json.loads(cleaned)
        result['category'] = 'Sci/Tech'
        return result
    except json.JSONDecodeError:
        return {"belongs_to_category": False, "confidence": 0, "reasoning": "Parse error", 
                "orchestrator_hint_addressed": "N/A", "category": "Sci/Tech"}

# ============================================================
# SYNTHESIZER (gpt-5.1)
# ============================================================

SYNTHESIZER_SYSTEM_PROMPT = """You are the final decision-maker in a news classification system.

You will receive:
1. The original news article
2. The orchestrator's analysis (summary, key entities, likely categories, hints given)
3. Outputs from 4 specialized workers (World, Sports, Business, Sci/Tech)

Each worker has evaluated whether the article belongs to their category and provided:
- belongs_to_category: true/false
- confidence: 0.0-1.0
- reasoning: their analysis

Your task is to synthesize all this information and make the FINAL classification decision.

Consider:
- Which workers claimed the article and with what confidence?
- Are there conflicts? If so, whose reasoning is more compelling?
- Does the orchestrator's analysis align with worker assessments?
- What is the PRIMARY focus of the article?

You may OVERRIDE a worker's claim if their reasoning is flawed.

Return your final decision as JSON.

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "reasoning": "Detailed explanation of your decision",
    "worker_agreement_summary": "Which workers agreed/disagreed and why",
    "conflict_resolution": "If there were conflicts, how you resolved them",
    "orchestrator_alignment": "Whether your decision aligns with orchestrator's prediction"
}

Rules:
- You MUST choose exactly ONE category
- Provide clear reasoning for your decision
- If overriding a worker, explain why
- Return ONLY valid JSON, no additional text"""

def run_synthesizer(article_text: str, orchestrator_output: dict, worker_outputs: dict) -> dict:
    """Synthesizer: Make final classification decision based on all outputs."""
    user_prompt = f"""Make the final classification decision based on all available information.

ORIGINAL ARTICLE:
---
{article_text}
---

ORCHESTRATOR ANALYSIS:
{json.dumps(orchestrator_output, indent=2)}

WORKER OUTPUTS:

WORLD WORKER:
{json.dumps(worker_outputs['world'], indent=2)}

SPORTS WORKER:
{json.dumps(worker_outputs['sports'], indent=2)}

BUSINESS WORKER:
{json.dumps(worker_outputs['business'], indent=2)}

SCI/TECH WORKER:
{json.dumps(worker_outputs['scitech'], indent=2)}"""
    
    response = openai_client.chat.completions.create(
        model="gpt-5.1",
        max_completion_tokens=1024,
        messages=[
            {"role": "system", "content": SYNTHESIZER_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    response_text = response.choices[0].message.content
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {
            "error": "Failed to parse synthesizer response",
            "raw_response": response_text,
            "classification": "Unknown",
            "confidence": "low",
            "reasoning": "Parse error",
            "worker_agreement_summary": "N/A",
            "conflict_resolution": "N/A",
            "orchestrator_alignment": "N/A"
        }

# ============================================================
# ORCHESTRATOR-WORKER PIPELINE
# ============================================================

def orchestrator_worker_classify(article_text: str) -> dict:
    """Full orchestrator-worker pipeline: orchestrator → workers (parallel) → synthesizer."""
    
    # Stage 1: Orchestrator analyzes article and generates hints
    orchestrator_output = run_orchestrator(article_text)
    
    # Extract hints for each worker (with fallbacks)
    worker_hints = orchestrator_output.get('worker_hints', {})
    world_hint = worker_hints.get('world', 'Evaluate if this is World news.')
    sports_hint = worker_hints.get('sports', 'Evaluate if this is Sports news.')
    business_hint = worker_hints.get('business', 'Evaluate if this is Business news.')
    scitech_hint = worker_hints.get('scitech', 'Evaluate if this is Sci/Tech news.')
    
    # Stage 2: Run all 4 workers in parallel
    worker_outputs = {}
    with ThreadPoolExecutor(max_workers=4) as executor:
        # Submit all worker tasks
        futures = {
            executor.submit(run_world_worker, article_text, world_hint): "world",
            executor.submit(run_sports_worker, article_text, sports_hint): "sports",
            executor.submit(run_business_worker, article_text, business_hint): "business",
            executor.submit(run_scitech_worker, article_text, scitech_hint): "scitech"
        }
        
        # Collect results
        for future in as_completed(futures):
            worker_name = futures[future]
            worker_outputs[worker_name] = future.result()
    
    # Stage 3: Synthesizer makes final decision
    synthesizer_output = run_synthesizer(article_text, orchestrator_output, worker_outputs)
    
    return {
        "orchestrator_output": orchestrator_output,
        "worker_outputs": worker_outputs,
        "synthesizer_output": synthesizer_output
    }

# ============================================================
# PROCESS ALL TEST DATA
# ============================================================

# Store orchestrator-worker results
orchestrator_worker_results = []

# Number of results to print
NUM_TO_PRINT = 3

print("=" * 70)
print("ORCHESTRATOR-WORKER DESIGN PATTERN - Classification")
print("=" * 70)
print(f"\nProcessing {NUM_SAMPLES} samples using orchestrator-worker pattern...")
print(f"Pipeline: Orchestrator (gpt-5-mini) → 4 Workers (Claude Haiku, parallel) → Synthesizer (gpt-5.1)")
print(f"⚠️  Rate limiting enabled: ~{MIN_SECONDS_PER_ARTICLE:.1f}s delay between articles to stay under API limits.\n")

# Progress tracking
PROGRESS_INTERVAL = 10

for i in range(NUM_SAMPLES):
    start_time = time.time()
    
    sample = test_data[i]
    article_text = sample['text']
    ground_truth_label = sample['label']
    
    # Run the orchestrator-worker pipeline
    result = orchestrator_worker_classify(article_text)
    
    # Store result with metadata
    orchestrator_worker_results.append({
        "index": i,
        "original_text": article_text,
        "ground_truth_label": ground_truth_label,
        "ground_truth_name": label_names[ground_truth_label],
        "orchestrator_output": result["orchestrator_output"],
        "worker_outputs": result["worker_outputs"],
        "synthesizer_output": result["synthesizer_output"]
    })
    
    # Print progress
    if (i + 1) % PROGRESS_INTERVAL == 0 or (i + 1) == NUM_SAMPLES:
        print(f"  ✓ Processed {i + 1}/{NUM_SAMPLES} articles ({(i + 1) / NUM_SAMPLES * 100:.1f}%)")
    
    # Rate limiting: ensure minimum time between articles
    elapsed = time.time() - start_time
    if elapsed < MIN_SECONDS_PER_ARTICLE and i < NUM_SAMPLES - 1:
        sleep_time = MIN_SECONDS_PER_ARTICLE - elapsed
        time.sleep(sleep_time)

# ============================================================
# PRINT SAMPLE RESULTS
# ============================================================

print("\n" + "=" * 70)
print(f"ORCHESTRATOR-WORKER RESULTS (showing first {NUM_TO_PRINT} of {len(orchestrator_worker_results)})")
print("=" * 70)

for i in range(min(NUM_TO_PRINT, len(orchestrator_worker_results))):
    result = orchestrator_worker_results[i]
    synth = result['synthesizer_output']
    orch = result['orchestrator_output']
    
    print("─" * 70)
    print(f"Article {i + 1}")
    print(f"Ground Truth: {result['ground_truth_name']}")
    print("─" * 70)
    
    # Orchestrator summary
    print(f"\n📋 Orchestrator Analysis:")
    print(f"   Summary: {orch.get('summary', 'N/A')[:100]}...")
    print(f"   Likely Categories: {orch.get('likely_categories', [])}")
    
    # Worker votes
    print(f"\n👷 Worker Votes:")
    for worker_name, worker_result in result['worker_outputs'].items():
        claimed = "✓" if worker_result.get('belongs_to_category', False) else "✗"
        conf = worker_result.get('confidence', 0)
        print(f"   {worker_name.upper():10}: {claimed} (conf: {conf:.2f})")
    
    # Synthesizer decision
    predicted = synth.get('classification', 'N/A')
    confidence = synth.get('confidence', 'N/A')
    is_correct = predicted == result['ground_truth_name']
    status = "✓ CORRECT" if is_correct else "✗ INCORRECT"
    
    print(f"\n🔮 Synthesizer Decision:")
    print(f"   Classification: {predicted}")
    print(f"   Confidence: {confidence}")
    print(f"   Result: {status}")

print("\n" + "=" * 70)
print(f"Orchestrator-Worker complete. {len(orchestrator_worker_results)} results stored in 'orchestrator_worker_results'")
print("Ready for evaluation.")
print("=" * 70)


ORCHESTRATOR-WORKER DESIGN PATTERN - Classification

Processing 100 samples using orchestrator-worker pattern...
Pipeline: Orchestrator (gpt-5-mini) → 4 Workers (Claude Haiku, parallel) → Synthesizer (gpt-5.1)
⚠️  Rate limiting enabled: ~5.3s delay between articles to stay under API limits.

  ✓ Processed 10/100 articles (10.0%)
  ✓ Processed 20/100 articles (20.0%)
  ✓ Processed 30/100 articles (30.0%)
  ✓ Processed 40/100 articles (40.0%)
  ✓ Processed 50/100 articles (50.0%)
  ✓ Processed 60/100 articles (60.0%)
  ✓ Processed 70/100 articles (70.0%)
  ✓ Processed 80/100 articles (80.0%)
  ✓ Processed 90/100 articles (90.0%)
  ✓ Processed 100/100 articles (100.0%)

ORCHESTRATOR-WORKER RESULTS (showing first 3 of 100)
──────────────────────────────────────────────────────────────────────
Article 1
Ground Truth: Business
──────────────────────────────────────────────────────────────────────

📋 Orchestrator Analysis:
   Summary: Unions representing workers at Turner & Newall say they ar

In [24]:
# ============================================================
# EVALUATION: Orchestrator-Worker Design Pattern Performance
# ============================================================

# Extract predictions and ground truth from orchestrator-worker results
ow_predictions = []
ow_ground_truths = []
ow_errors = []

for result in orchestrator_worker_results:
    ground_truth = result['ground_truth_name']
    ow_ground_truths.append(ground_truth)
    
    # Get prediction from synthesizer_output
    synth = result['synthesizer_output']
    
    if 'classification' in synth and synth['classification'] in CATEGORIES:
        predicted = synth['classification']
        ow_predictions.append(predicted)
    else:
        # Handle errors
        ow_predictions.append("ERROR")
        ow_errors.append(result['index'])

print("=" * 70)
print("ORCHESTRATOR-WORKER DESIGN PATTERN - EVALUATION RESULTS")
print("=" * 70)

# Check for errors
if ow_errors:
    print(f"\n⚠️  Warning: {len(ow_errors)} samples had errors (indices: {ow_errors[:10]}{'...' if len(ow_errors) > 10 else ''})")
    print(f"   These are excluded from metrics calculation.\n")
    
    valid_ow_predictions = [p for p in ow_predictions if p != "ERROR"]
    valid_ow_ground_truths = [g for p, g in zip(ow_predictions, ow_ground_truths) if p != "ERROR"]
else:
    valid_ow_predictions = ow_predictions
    valid_ow_ground_truths = ow_ground_truths

# ============================================================
# Basic Metrics
# ============================================================

print("\n" + "─" * 70)
print("OVERALL METRICS")
print("─" * 70)

ow_accuracy = accuracy_score(valid_ow_ground_truths, valid_ow_predictions)
ow_precision_macro = precision_score(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES, average='macro', zero_division=0)
ow_recall_macro = recall_score(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES, average='macro', zero_division=0)
ow_f1_macro = f1_score(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES, average='macro', zero_division=0)

ow_precision_weighted = precision_score(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
ow_recall_weighted = recall_score(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
ow_f1_weighted = f1_score(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES, average='weighted', zero_division=0)

print(f"\n  Total Samples:     {len(orchestrator_worker_results)}")
print(f"  Valid Predictions: {len(valid_ow_predictions)}")
print(f"  Error Rate:        {len(ow_errors) / len(orchestrator_worker_results) * 100:.1f}%")

print(f"\n  📊 Accuracy:        {ow_accuracy:.4f}  ({ow_accuracy * 100:.1f}%)")
print(f"\n  Macro Averages (treats all classes equally):")
print(f"     Precision:      {ow_precision_macro:.4f}")
print(f"     Recall:         {ow_recall_macro:.4f}")
print(f"     F1 Score:       {ow_f1_macro:.4f}")

print(f"\n  Weighted Averages (accounts for class imbalance):")
print(f"     Precision:      {ow_precision_weighted:.4f}")
print(f"     Recall:         {ow_recall_weighted:.4f}")
print(f"     F1 Score:       {ow_f1_weighted:.4f}")

# ============================================================
# Per-Class Metrics
# ============================================================

print("\n" + "─" * 70)
print("PER-CLASS METRICS")
print("─" * 70)

print("\n" + classification_report(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES, zero_division=0))

# ============================================================
# Confusion Matrix
# ============================================================

print("─" * 70)
print("CONFUSION MATRIX")
print("─" * 70)

ow_cm = confusion_matrix(valid_ow_ground_truths, valid_ow_predictions, labels=CATEGORIES)

print(f"\n{'Predicted →':>15}", end="")
for cat in CATEGORIES:
    print(f"{cat:>10}", end="")
print("\n" + "─" * 55)

print("Actual ↓")
for i, cat in enumerate(CATEGORIES):
    print(f"{cat:>15}", end="")
    for j in range(len(CATEGORIES)):
        print(f"{ow_cm[i][j]:>10}", end="")
    print()

# ============================================================
# Distribution Analysis
# ============================================================

print("\n" + "─" * 70)
print("DISTRIBUTION ANALYSIS")
print("─" * 70)

ow_gt_dist = Counter(valid_ow_ground_truths)
ow_pred_dist = Counter(valid_ow_predictions)

print(f"\n{'Category':<12} {'Ground Truth':>15} {'Predicted':>15} {'Difference':>12}")
print("─" * 55)
for cat in CATEGORIES:
    gt_count = ow_gt_dist.get(cat, 0)
    pred_count = ow_pred_dist.get(cat, 0)
    diff = pred_count - gt_count
    diff_str = f"+{diff}" if diff > 0 else str(diff)
    print(f"{cat:<12} {gt_count:>15} {pred_count:>15} {diff_str:>12}")

# ============================================================
# Orchestrator Analysis (Unique to Orchestrator-Worker)
# ============================================================

print("\n" + "─" * 70)
print("ORCHESTRATOR ANALYSIS")
print("─" * 70)

# How often was the orchestrator's likely_categories prediction correct?
orch_correct_predictions = 0
orch_first_choice_correct = 0
orch_total_with_predictions = 0

for result in orchestrator_worker_results:
    orch = result['orchestrator_output']
    ground_truth = result['ground_truth_name']
    likely_cats = orch.get('likely_categories', [])
    
    if likely_cats:
        orch_total_with_predictions += 1
        # Check if ground truth is in likely categories
        if ground_truth in likely_cats:
            orch_correct_predictions += 1
        # Check if first choice is correct
        if likely_cats[0] == ground_truth:
            orch_first_choice_correct += 1

print(f"\n  Orchestrator Prediction Accuracy:")
if orch_total_with_predictions > 0:
    print(f"    Ground truth in likely_categories: {orch_correct_predictions}/{orch_total_with_predictions} ({orch_correct_predictions/orch_total_with_predictions*100:.1f}%)")
    print(f"    First choice correct:              {orch_first_choice_correct}/{orch_total_with_predictions} ({orch_first_choice_correct/orch_total_with_predictions*100:.1f}%)")
else:
    print(f"    No orchestrator predictions available")

# ============================================================
# Worker Analysis (Similar to Parallelization)
# ============================================================

print("\n" + "─" * 70)
print("WORKER ANALYSIS")
print("─" * 70)

# Count how many workers claimed each article
worker_claim_counts = Counter()
for result in orchestrator_worker_results:
    claims = sum(1 for worker_result in result['worker_outputs'].values() 
                 if worker_result.get('belongs_to_category', False))
    worker_claim_counts[claims] += 1

print(f"\n  Number of Workers Claiming Article:")
print(f"  {'Claims':>10} {'Count':>10} {'Percentage':>12}")
print("  " + "─" * 35)
for num_claims in sorted(worker_claim_counts.keys()):
    count = worker_claim_counts[num_claims]
    pct = count / len(orchestrator_worker_results) * 100
    print(f"  {num_claims:>10} {count:>10} {pct:>11.1f}%")

# Per-worker accuracy (when that worker claimed and was correct)
print(f"\n  Per-Worker Claim Statistics:")
print(f"  {'Worker':<12} {'Claimed':>10} {'Correct':>10} {'Accuracy':>12}")
print("  " + "─" * 47)

worker_stats = {
    'world': {'claimed': 0, 'correct': 0},
    'sports': {'claimed': 0, 'correct': 0},
    'business': {'claimed': 0, 'correct': 0},
    'scitech': {'claimed': 0, 'correct': 0}
}
category_map = {'world': 'World', 'sports': 'Sports', 'business': 'Business', 'scitech': 'Sci/Tech'}

for result in orchestrator_worker_results:
    for worker_name, worker_result in result['worker_outputs'].items():
        if worker_result.get('belongs_to_category', False):
            worker_stats[worker_name]['claimed'] += 1
            if category_map[worker_name] == result['ground_truth_name']:
                worker_stats[worker_name]['correct'] += 1

for worker_name in ['world', 'sports', 'business', 'scitech']:
    claimed = worker_stats[worker_name]['claimed']
    correct = worker_stats[worker_name]['correct']
    acc = correct / claimed if claimed > 0 else 0
    print(f"  {worker_name.upper():<12} {claimed:>10} {correct:>10} {acc:>11.1%}")

# ============================================================
# Synthesizer Analysis (Unique to Orchestrator-Worker)
# ============================================================

print("\n" + "─" * 70)
print("SYNTHESIZER ANALYSIS")
print("─" * 70)

# How often did synthesizer agree with orchestrator's first choice?
synth_orch_agreement = 0
synth_orch_total = 0

# How often did synthesizer override workers (pick category no worker claimed)?
synth_override_count = 0

# Synthesizer confidence distribution
synth_confidence_dist = Counter()

for result in orchestrator_worker_results:
    orch = result['orchestrator_output']
    synth = result['synthesizer_output']
    workers = result['worker_outputs']
    
    synth_classification = synth.get('classification', '')
    synth_confidence = synth.get('confidence', 'unknown')
    synth_confidence_dist[synth_confidence] += 1
    
    # Check orchestrator-synthesizer alignment
    likely_cats = orch.get('likely_categories', [])
    if likely_cats:
        synth_orch_total += 1
        if synth_classification == likely_cats[0]:
            synth_orch_agreement += 1
    
    # Check if synthesizer overrode workers
    claimed_categories = set()
    for worker_name, worker_result in workers.items():
        if worker_result.get('belongs_to_category', False):
            claimed_categories.add(category_map.get(worker_name, ''))
    
    if synth_classification and synth_classification not in claimed_categories and claimed_categories:
        synth_override_count += 1

print(f"\n  Synthesizer-Orchestrator Alignment:")
if synth_orch_total > 0:
    print(f"    Agreed with orchestrator's 1st choice: {synth_orch_agreement}/{synth_orch_total} ({synth_orch_agreement/synth_orch_total*100:.1f}%)")
else:
    print(f"    No orchestrator predictions to compare")

print(f"\n  Synthesizer Override Rate:")
print(f"    Overrode workers (picked unclaimed category): {synth_override_count}/{len(orchestrator_worker_results)} ({synth_override_count/len(orchestrator_worker_results)*100:.1f}%)")

print(f"\n  Synthesizer Confidence Distribution:")
print(f"  {'Confidence':<15} {'Count':>10} {'Percentage':>12}")
print("  " + "─" * 40)
for conf_level in ['high', 'medium', 'low', 'unknown']:
    count = synth_confidence_dist.get(conf_level, 0)
    if count > 0:
        pct = count / len(orchestrator_worker_results) * 100
        print(f"  {conf_level:<15} {count:>10} {pct:>11.1f}%")

# ============================================================
# Confidence vs Accuracy Analysis
# ============================================================

print("\n" + "─" * 70)
print("CONFIDENCE VS ACCURACY ANALYSIS")
print("─" * 70)

# Group by synthesizer confidence and calculate accuracy
confidence_accuracy = {"high": [0, 0], "medium": [0, 0], "low": [0, 0]}

for result in orchestrator_worker_results:
    synth = result['synthesizer_output']
    conf = synth.get('confidence', 'unknown')
    predicted = synth.get('classification', '')
    is_correct = predicted == result['ground_truth_name']
    
    if conf in confidence_accuracy:
        confidence_accuracy[conf][1] += 1
        if is_correct:
            confidence_accuracy[conf][0] += 1

print(f"\n{'Confidence':<15} {'Count':>10} {'Correct':>10} {'Accuracy':>12}")
print("─" * 50)
for conf_level in ['high', 'medium', 'low']:
    correct, total = confidence_accuracy[conf_level]
    if total > 0:
        acc = correct / total
        print(f"{conf_level:<15} {total:>10} {correct:>10} {acc:>11.1%}")
    else:
        print(f"{conf_level:<15} {0:>10} {0:>10} {'N/A':>12}")

# ============================================================
# End-to-End Pipeline Analysis
# ============================================================

print("\n" + "─" * 70)
print("END-TO-END PIPELINE ANALYSIS")
print("─" * 70)

# When orchestrator predicted correctly AND synthesizer got it right
orch_correct_synth_correct = 0
orch_correct_synth_wrong = 0
orch_wrong_synth_correct = 0
orch_wrong_synth_wrong = 0

for result in orchestrator_worker_results:
    orch = result['orchestrator_output']
    synth = result['synthesizer_output']
    ground_truth = result['ground_truth_name']
    
    likely_cats = orch.get('likely_categories', [])
    synth_classification = synth.get('classification', '')
    
    orch_correct = ground_truth in likely_cats if likely_cats else False
    synth_correct = synth_classification == ground_truth
    
    if orch_correct and synth_correct:
        orch_correct_synth_correct += 1
    elif orch_correct and not synth_correct:
        orch_correct_synth_wrong += 1
    elif not orch_correct and synth_correct:
        orch_wrong_synth_correct += 1
    else:
        orch_wrong_synth_wrong += 1

total = len(orchestrator_worker_results)
print(f"\n  Pipeline Error Analysis:")
print(f"  {'Scenario':<40} {'Count':>10} {'Percentage':>12}")
print("  " + "─" * 65)
print(f"  {'Orchestrator ✓ → Synthesizer ✓':<40} {orch_correct_synth_correct:>10} {orch_correct_synth_correct/total*100:>11.1f}%")
print(f"  {'Orchestrator ✓ → Synthesizer ✗ (error introduced)':<40} {orch_correct_synth_wrong:>10} {orch_correct_synth_wrong/total*100:>11.1f}%")
print(f"  {'Orchestrator ✗ → Synthesizer ✓ (error corrected)':<40} {orch_wrong_synth_correct:>10} {orch_wrong_synth_correct/total*100:>11.1f}%")
print(f"  {'Orchestrator ✗ → Synthesizer ✗':<40} {orch_wrong_synth_wrong:>10} {orch_wrong_synth_wrong/total*100:>11.1f}%")

print("\n" + "=" * 70)
print("ORCHESTRATOR-WORKER PATTERN EVALUATION COMPLETE")
print("=" * 70)


ORCHESTRATOR-WORKER DESIGN PATTERN - EVALUATION RESULTS

──────────────────────────────────────────────────────────────────────
OVERALL METRICS
──────────────────────────────────────────────────────────────────────

  Total Samples:     100
  Valid Predictions: 100
  Error Rate:        0.0%

  📊 Accuracy:        0.8000  (80.0%)

  Macro Averages (treats all classes equally):
     Precision:      0.8134
     Recall:         0.8550
     F1 Score:       0.7924

  Weighted Averages (accounts for class imbalance):
     Precision:      0.8842
     Recall:         0.8000
     F1 Score:       0.8071

──────────────────────────────────────────────────────────────────────
PER-CLASS METRICS
──────────────────────────────────────────────────────────────────────

              precision    recall  f1-score   support

       World       0.90      0.90      0.90        30
      Sports       0.91      0.95      0.93        21
    Business       0.44      1.00      0.62        12
    Sci/Tech       1.0

## 5. Evaluator-Optimizer Design Pattern

### What is Evaluator-Optimizer?

**Evaluator-Optimizer** (also called Generator-Critic or Self-Refinement) is an agentic workflow pattern where an LLM generator produces an output, and an LLM evaluator critically assesses it. If the output doesn't meet quality criteria, feedback is sent back to the generator for refinement. This iterative loop continues until the output is approved or maximum iterations are reached.

This pattern is particularly useful when:
- Tasks benefit from iterative refinement
- Initial outputs may have subtle errors that can be caught through critical review
- Quality is more important than speed
- The system needs self-correction capability
- Ambiguous inputs require careful consideration

### Key Components

1. **Generator (LLM)**: Produces output with explicit reasoning
2. **Evaluator (LLM)**: Critically assesses the output and provides feedback
3. **Feedback Loop**: Mechanism to send evaluator's critique back to generator
4. **Termination Condition**: When to stop iterating (approval or max iterations)

### How It Differs from Other Patterns

| Pattern | Approach | Key Feature |
|---------|----------|-------------|
| **Prompt Chaining** | Sequential steps | Fixed pipeline, no backtracking |
| **Routing** | Expert selection | One-shot classification |
| **Parallelization** | Multiple perspectives | Consensus from parallel views |
| **Orchestrator-Worker** | Coordinated workers | LLM coordination and synthesis |
| **Evaluator-Optimizer** | Iterative refinement | **Self-correction through feedback** |

---

### Generic Evaluator-Optimizer Workflow Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                   EVALUATOR-OPTIMIZER DESIGN PATTERN                        │
└─────────────────────────────────────────────────────────────────────────────┘

┌──────────────┐
│              │
│    INPUT     │
│              │
└──────┬───────┘
       │
       ▼
┌──────────────────────────────────────────────────────────────────┐
│                      📝 GENERATOR (LLM)                          │
│                                                                  │
│  Responsibilities:                                               │
│  • Analyze the input                                            │
│  • Produce output with explicit reasoning                       │
│  • Cite evidence supporting the output                          │
│  • Consider alternatives                                        │
│                                                                  │
│  Output: Result + Detailed Reasoning + Evidence                 │
└──────────────────────────────┬───────────────────────────────────┘
                               │
                               │ Output + Reasoning
                               │
                               ▼
┌──────────────────────────────────────────────────────────────────┐
│                      🔍 EVALUATOR (LLM)                          │
│                                                                  │
│  Responsibilities:                                               │
│  • Re-examine the original input                                │
│  • Critically assess generator's reasoning                      │
│  • Check for logical consistency                                │
│  • Identify overlooked evidence                                 │
│  • Consider alternative interpretations                         │
│  • Decide: APPROVE or REJECT with actionable feedback           │
│                                                                  │
│  Output: Decision (APPROVE/REJECT) + Feedback (if rejected)     │
└──────────────────────────────┬───────────────────────────────────┘
                               │
                    ┌──────────┴──────────┐
                    │                     │
                    ▼                     ▼
            ┌─────────────┐       ┌─────────────────┐
            │  APPROVED   │       │    REJECTED     │
            │             │       │  + FEEDBACK     │
            └──────┬──────┘       └────────┬────────┘
                   │                       │
                   │                       │ ┌─────────────────────┐
                   │                       │ │   FEEDBACK LOOP     │
                   │                       │ │                     │
                   │                       │ │ • Original input    │
                   │                       │ │ • Previous output   │
                   │                       │ │ • Evaluator feedback│
                   │                       │ │ • Iteration count   │
                   │                       │ └──────────┬──────────┘
                   │                       │            │
                   │                       │            ▼
                   │                       │   ┌───────────────┐
                   │                       └──▶│   GENERATOR   │
                   │                           │   (Retry)     │
                   │                           └───────┬───────┘
                   │                                   │
                   │                           (Back to Evaluator)
                   │                                   │
                   ▼                                   │
            ┌──────────────┐                          │
            │              │◀─────────────────────────┘
            │    OUTPUT    │   (After approval or max iterations)
            │              │
            └──────────────┘
```

---

### Advantages of Evaluator-Optimizer

| Aspect | Benefit |
|--------|---------|
| **Self-Correction** | System can catch and fix its own mistakes |
| **Quality Assurance** | Built-in review before final output |
| **Reasoning Verification** | Evaluator validates logical consistency |
| **Iterative Improvement** | Each round addresses specific issues |
| **Explainability** | Full audit trail of reasoning and refinements |

### Trade-offs

| Aspect | Consideration |
|--------|---------------|
| **Latency** | Multiple iterations increase response time |
| **Cost** | More LLM calls per input |
| **Complexity** | Feedback loop logic must be carefully designed |
| **Convergence** | No guarantee of reaching approval |
| **Diminishing Returns** | Later iterations may not improve quality |

---

### Execution Flow

```
┌─────────┐    ┌───────────┐    ┌───────────┐    ┌──────────┐
│  INPUT  │───▶│ GENERATOR │───▶│ EVALUATOR │───▶│ APPROVED │───▶ OUTPUT
└─────────┘    └───────────┘    └─────┬─────┘    └──────────┘
                    ▲                 │
                    │                 │ REJECTED
                    │                 ▼
                    │           ┌──────────┐
                    └───────────│ FEEDBACK │
                                └──────────┘

Iteration 1: Generator → Evaluator (approve/reject)
Iteration 2: Generator (with feedback) → Evaluator (approve/reject)
...
Iteration N: Generator (with feedback) → Evaluator → Final Output

Termination: Approved OR max_iterations reached
```

### LLM Calls per Input

| Scenario | Generator Calls | Evaluator Calls | Total |
|----------|-----------------|-----------------|-------|
| Approved on 1st try | 1 | 1 | **2** |
| Approved on 2nd try | 2 | 2 | **4** |
| Approved on 3rd try | 3 | 3 | **6** |
| Max iterations (3) reached | 3 | 3 | **6** |


### Our Implementation: Evaluator-Optimizer for AG News Classification

For our AG News classification task, we'll implement the evaluator-optimizer pattern with iterative refinement until the evaluator approves or max iterations (3) are reached.

---

#### Architecture Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│           EVALUATOR-OPTIMIZER FOR AG NEWS CLASSIFICATION                    │
└─────────────────────────────────────────────────────────────────────────────┘

┌──────────────┐
│  NEWS        │
│  ARTICLE     │
└──────┬───────┘
       │
       ▼
┌──────────────────────────────────────────────────────────────────┐
│                 📝 GENERATOR (gpt-5.1)                           │
│                                                                  │
│  • Analyzes the news article                                    │
│  • Extracts key signals and evidence                            │
│  • Makes classification decision                                │
│  • Provides detailed, verifiable reasoning                      │
│  • Considers alternative categories                             │
│                                                                  │
└──────────────────────────────┬───────────────────────────────────┘
                               │
                               │ Classification + Reasoning + Evidence
                               │
                               ▼
┌──────────────────────────────────────────────────────────────────┐
│             🔍 EVALUATOR (claude-opus-4-5-20251101)              │
│                                                                  │
│  • Re-reads the original article                                │
│  • Critically examines generator's reasoning                    │
│  • Checks logical consistency                                   │
│  • Identifies overlooked evidence                               │
│  • Considers if alternative was dismissed too quickly           │
│  • Decision: APPROVE or REJECT + actionable feedback            │
│                                                                  │
└──────────────────────────────┬───────────────────────────────────┘
                               │
                    ┌──────────┴──────────┐
                    │                     │
                    ▼                     ▼
            ┌─────────────┐       ┌─────────────────┐
            │  APPROVED   │       │    REJECTED     │
            │  ✓ Use      │       │  + FEEDBACK     │
            │  result     │       └────────┬────────┘
            └──────┬──────┘                │
                   │                       │ Feedback Loop
                   │                       ▼
                   │              ┌─────────────────────────────────┐
                   │              │  🔄 GENERATOR (Retry)           │
                   │              │  Input: Article + Previous +    │
                   │              │  Feedback + Iteration#          │
                   │              └────────────────┬────────────────┘
                   │                               │
                   │                        (Back to Evaluator)
                   ▼                               │
            ┌──────────────┐                      │
            │   FINAL      │◀─────────────────────┘
            │   CATEGORY   │   (After approval or max 3 iterations)
            └──────────────┘
```

---

#### Models Used

| Component | Model | Rationale |
|-----------|-------|-----------|
| Generator | `gpt-5.1` | Strong reasoning and evidence extraction |
| Evaluator | `claude-opus-4-5-20251101` | Superior critical analysis and nuanced assessment |

**Max Iterations:** 3 (balance between quality and cost)

---

### Generator (gpt-5.1) - Initial Classification 📝

#### Input

Raw news article text.

#### System Prompt

```
You are an expert news classifier. Your task is to classify a news article into exactly ONE of these four categories:
- World: International affairs, politics, government, diplomacy, conflicts
- Sports: Athletic events, teams, players, competitions, leagues
- Business: Companies, markets, economy, finance, corporate news
- Sci/Tech: Technology, science, software, research, innovation

Analyze the article thoroughly and provide your classification with EXPLICIT, VERIFIABLE reasoning.

Your response must include:
1. Classification decision
2. Confidence level (high/medium/low)
3. Key signals extracted from the article (entities, topics, keywords)
4. Detailed reasoning explaining WHY this category fits best
5. Alternative category considered and why it was rejected
6. Specific evidence from the article supporting your decision

Rules:
- Be thorough and precise in your analysis
- Cite specific phrases or facts from the article as evidence
- Consider edge cases (e.g., tech company financials could be Business OR Sci/Tech)
- Your reasoning must be detailed enough for critical review
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "key_signals": {
        "entities": ["list of people, organizations, teams mentioned"],
        "topics": ["main topics discussed"],
        "domain_keywords": ["significant domain-specific terms"]
    },
    "reasoning": "Detailed explanation of why this classification is correct",
    "alternative_considered": {
        "category": "The alternative category you considered",
        "why_rejected": "Specific reason this alternative does not fit as well"
    },
    "evidence": ["Direct quotes or specific facts from the article"]
}
```

#### User Prompt Format

```
Classify the following news article:

---
[Article text]
---
```

#### Output Format

```json
{
    "classification": "Business",
    "confidence": "high",
    "key_signals": {
        "entities": ["Apple", "Tim Cook", "Wall Street analysts"],
        "topics": ["quarterly earnings", "revenue growth", "stock performance"],
        "domain_keywords": ["earnings", "revenue", "profit margin", "stock price"]
    },
    "reasoning": "The article focuses on Apple's quarterly financial performance...",
    "alternative_considered": {
        "category": "Sci/Tech",
        "why_rejected": "No discussion of products, technology, or innovation."
    },
    "evidence": ["Apple reported quarterly revenue of $89.5 billion", "Stock rose 3%"]
}
```

---

### Evaluator (claude-opus-4-5-20251101) - Critical Assessment 🔍

#### Input

- Original news article
- Generator's classification output

#### System Prompt

```
You are a critical evaluator for a news classification system. Your job is to rigorously assess whether a classification decision is correct and well-reasoned.

You will receive:
1. The original news article
2. A classification produced by a generator, including its reasoning and evidence

Your task is to critically evaluate the classification by:
1. Re-reading the original article independently
2. Checking if the reasoning logically supports the conclusion
3. Verifying that cited evidence is accurate and relevant
4. Looking for overlooked evidence that might change the classification
5. Considering if the alternative category was dismissed too quickly
6. Assessing if the confidence level is appropriate

Evaluation Criteria:
- LOGICAL CONSISTENCY: Does the reasoning actually support the conclusion?
- EVIDENCE ACCURACY: Did the generator correctly identify and interpret signals?
- COMPLETENESS: Are there important signals the generator overlooked?
- ALTERNATIVE ANALYSIS: Was the alternative category fairly considered?
- CONFIDENCE CALIBRATION: Does the evidence strength match the confidence level?

Decision Guidelines:
- APPROVE if: Reasoning is sound, evidence is accurate, no significant oversights
- REJECT if: Logical flaws, misinterpreted evidence, overlooked critical signals, or wrong classification

If you REJECT, provide specific, actionable feedback telling the generator:
- What specific issue you found
- What evidence was overlooked or misinterpreted
- What the generator should reconsider

Rules:
- Be rigorous but fair - do not reject for trivial issues
- Your feedback must be specific and actionable
- Return ONLY valid JSON, no additional text

Output format:
{
    "decision": "APPROVE" | "REJECT",
    "evaluation": {
        "logical_consistency": "Assessment of reasoning",
        "evidence_accuracy": "Assessment of evidence interpretation",
        "completeness": "Assessment of signal coverage",
        "alternative_analysis": "Assessment of alternative consideration",
        "confidence_calibration": "Assessment of confidence appropriateness"
    },
    "issues_found": [
        {
            "type": "overlooked_evidence | logical_flaw | misinterpreted_signal | weak_reasoning",
            "description": "Specific description of the issue"
        }
    ],
    "feedback": "Specific, actionable feedback for generator (if REJECT)",
    "final_classification": "Classification to use (same as generator if APPROVE)"
}
```

#### User Prompt Format

```
Evaluate the following news classification:

ORIGINAL ARTICLE:
---
[Article text]
---

GENERATOR'S CLASSIFICATION:
[JSON of generator's output]
```

#### Output Format (APPROVE)

```json
{
    "decision": "APPROVE",
    "evaluation": {
        "logical_consistency": "Sound. Financial metrics correctly identified.",
        "evidence_accuracy": "Accurate. Revenue and stock data correctly cited.",
        "completeness": "Complete. No significant signals overlooked.",
        "alternative_analysis": "Fair. Sci/Tech reasonably rejected.",
        "confidence_calibration": "Appropriate. High confidence justified."
    },
    "issues_found": [],
    "feedback": null,
    "final_classification": "Business"
}
```

#### Output Format (REJECT)

```json
{
    "decision": "REJECT",
    "evaluation": {
        "logical_consistency": "Weak. Conclusion does not follow from evidence.",
        "evidence_accuracy": "Partial. Missed key technical content.",
        "completeness": "Incomplete. Overlooked AI chip discussion.",
        "alternative_analysis": "Insufficient. Sci/Tech dismissed too quickly.",
        "confidence_calibration": "Overconfident given mixed signals."
    },
    "issues_found": [
        {
            "type": "overlooked_evidence",
            "description": "Article discusses AI chip architecture in technical detail"
        }
    ],
    "feedback": "Reconsider classification. The PRIMARY content focuses on technology, not finances.",
    "final_classification": null
}
```

---

### Generator (gpt-5.1) - Retry with Feedback 🔄

When the evaluator rejects, the generator receives feedback and must reconsider.

#### Input

- Original news article
- Previous classification attempt
- Evaluator's feedback
- Current iteration number

#### System Prompt (Retry)

```
You are an expert news classifier revising a previous classification based on evaluator feedback.

You previously classified a news article, but an evaluator found issues with your reasoning. You must now reconsider your classification while addressing the specific feedback provided.

Your task:
1. Re-read the original article carefully
2. Consider the evaluator's feedback seriously
3. Re-examine the evidence, especially areas highlighted in the feedback
4. Either REVISE your classification or DEFEND your original decision with better reasoning

Important:
- You MUST address each issue raised in the feedback
- If you change your classification, explain what new insight led to the change
- If you maintain your original classification, provide stronger evidence and reasoning
- Your confidence should reflect the difficulty of this classification

Categories:
- World: International affairs, politics, government, diplomacy, conflicts
- Sports: Athletic events, teams, players, competitions, leagues
- Business: Companies, markets, economy, finance, corporate news
- Sci/Tech: Technology, science, software, research, innovation

Rules:
- Explicitly address each point in the evaluator's feedback
- Provide even more detailed reasoning than before
- Cite specific evidence from the article
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "key_signals": {
        "entities": ["list of entities"],
        "topics": ["main topics"],
        "domain_keywords": ["domain-specific terms"]
    },
    "reasoning": "Detailed explanation addressing evaluator concerns",
    "feedback_addressed": {
        "issue": "What the evaluator raised",
        "response": "How you addressed it"
    },
    "changed_classification": true | false,
    "change_explanation": "Why you changed or maintained your classification",
    "evidence": ["Specific evidence from the article"]
}
```

#### User Prompt Format (Retry)

```
Revise your classification based on evaluator feedback.

ORIGINAL ARTICLE:
---
[Article text]
---

YOUR PREVIOUS CLASSIFICATION:
[JSON of previous generator output]

EVALUATOR'S FEEDBACK:
[Evaluator's feedback string]

ITERATION: [2 or 3] of 3

Please reconsider your classification, addressing the evaluator's concerns.
```

#### Output Format (Retry)

```json
{
    "classification": "Sci/Tech",
    "confidence": "high",
    "key_signals": {
        "entities": ["Apple", "Neural Engine", "A17 Pro chip"],
        "topics": ["AI chip technology", "neural processing"],
        "domain_keywords": ["neural engine", "transistors", "machine learning"]
    },
    "reasoning": "After re-examining with evaluator feedback, the article's PRIMARY content is technical...",
    "feedback_addressed": {
        "issue": "Overlooked AI chip technical discussion",
        "response": "Re-examined and found detailed technical specifications as primary focus"
    },
    "changed_classification": true,
    "change_explanation": "Changed from Business to Sci/Tech. Original focused on financial framing, but substance is technology.",
    "evidence": ["New A17 Pro chip features 19 billion transistors", "16-core Neural Engine"]
}
```

---

### Summary Table

| Component | Model | When Used | Purpose |
|-----------|-------|-----------|---------|
| **Generator (Initial)** | `gpt-5.1` | First iteration | Initial classification with reasoning |
| **Evaluator** | `claude-opus-4-5-20251101` | After each generation | Critical assessment and feedback |
| **Generator (Retry)** | `gpt-5.1` | After rejection | Revised classification addressing feedback |

**Execution Flow:**
1. Generator produces initial classification (1 LLM call)
2. Evaluator assesses: APPROVE or REJECT (1 LLM call)
3. If REJECT: Generator retries with feedback (1 LLM call) → back to step 2
4. Loop until APPROVE or max iterations (3) reached

**LLM Calls:** 2 (best case) to 6 (worst case, 3 iterations)


In [26]:
# ============================================================
# EVALUATOR-OPTIMIZER DESIGN PATTERN IMPLEMENTATION
# ============================================================

# Configuration
MAX_ITERATIONS = 3

# ============================================================
# GENERATOR (gpt-5.1) - Initial Classification
# ============================================================

GENERATOR_SYSTEM_PROMPT = """You are an expert news classifier. Your task is to classify a news article into exactly ONE of these four categories:
- World: International affairs, politics, government, diplomacy, conflicts
- Sports: Athletic events, teams, players, competitions, leagues
- Business: Companies, markets, economy, finance, corporate news
- Sci/Tech: Technology, science, software, research, innovation

Analyze the article thoroughly and provide your classification with EXPLICIT, VERIFIABLE reasoning.

Your response must include:
1. Classification decision
2. Confidence level (high/medium/low)
3. Key signals extracted from the article (entities, topics, keywords)
4. Detailed reasoning explaining WHY this category fits best
5. Alternative category considered and why it was rejected
6. Specific evidence from the article supporting your decision

Rules:
- Be thorough and precise in your analysis
- Cite specific phrases or facts from the article as evidence
- Consider edge cases (e.g., tech company financials could be Business OR Sci/Tech)
- Your reasoning must be detailed enough for critical review
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "key_signals": {
        "entities": ["list of people, organizations, teams mentioned"],
        "topics": ["main topics discussed"],
        "domain_keywords": ["significant domain-specific terms"]
    },
    "reasoning": "Detailed explanation of why this classification is correct",
    "alternative_considered": {
        "category": "The alternative category you considered",
        "why_rejected": "Specific reason this alternative does not fit as well"
    },
    "evidence": ["Direct quotes or specific facts from the article"]
}"""

def run_generator_initial(article_text: str) -> dict:
    """Generator: Initial classification with detailed reasoning."""
    user_prompt = f"""Classify the following news article:

---
{article_text}
---"""
    
    response = openai_client.chat.completions.create(
        model="gpt-5.1",
        max_completion_tokens=4096,
        messages=[
            {"role": "system", "content": GENERATOR_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    response_text = response.choices[0].message.content
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {
            "error": "Failed to parse generator response",
            "raw_response": response_text,
            "classification": "Unknown",
            "confidence": "low",
            "reasoning": "Parse error"
        }

# ============================================================
# EVALUATOR (claude-opus-4-5-20251101) - Critical Assessment
# ============================================================

EVALUATOR_SYSTEM_PROMPT = """You are a critical evaluator for a news classification system. Your job is to rigorously assess whether a classification decision is correct and well-reasoned.

You will receive:
1. The original news article
2. A classification produced by a generator, including its reasoning and evidence

Your task is to critically evaluate the classification by:
1. Re-reading the original article independently
2. Checking if the reasoning logically supports the conclusion
3. Verifying that cited evidence is accurate and relevant
4. Looking for overlooked evidence that might change the classification
5. Considering if the alternative category was dismissed too quickly
6. Assessing if the confidence level is appropriate

Evaluation Criteria:
- LOGICAL CONSISTENCY: Does the reasoning actually support the conclusion?
- EVIDENCE ACCURACY: Did the generator correctly identify and interpret signals?
- COMPLETENESS: Are there important signals the generator overlooked?
- ALTERNATIVE ANALYSIS: Was the alternative category fairly considered?
- CONFIDENCE CALIBRATION: Does the evidence strength match the confidence level?

Decision Guidelines:
- APPROVE if: Reasoning is sound, evidence is accurate, no significant oversights
- REJECT if: Logical flaws, misinterpreted evidence, overlooked critical signals, or wrong classification

If you REJECT, provide specific, actionable feedback telling the generator:
- What specific issue you found
- What evidence was overlooked or misinterpreted
- What the generator should reconsider

Rules:
- Be rigorous but fair - do not reject for trivial issues
- Your feedback must be specific and actionable
- Return ONLY valid JSON, no additional text

Output format:
{
    "decision": "APPROVE" | "REJECT",
    "evaluation": {
        "logical_consistency": "Assessment of reasoning",
        "evidence_accuracy": "Assessment of evidence interpretation",
        "completeness": "Assessment of signal coverage",
        "alternative_analysis": "Assessment of alternative consideration",
        "confidence_calibration": "Assessment of confidence appropriateness"
    },
    "issues_found": [
        {
            "type": "overlooked_evidence | logical_flaw | misinterpreted_signal | weak_reasoning",
            "description": "Specific description of the issue"
        }
    ],
    "feedback": "Specific, actionable feedback for generator (if REJECT, null if APPROVE)",
    "final_classification": "Classification to use (same as generator if APPROVE, null if REJECT)"
}"""

def run_evaluator(article_text: str, generator_output: dict) -> dict:
    """Evaluator: Critically assess the generator's classification."""
    user_prompt = f"""Evaluate the following news classification:

ORIGINAL ARTICLE:
---
{article_text}
---

GENERATOR'S CLASSIFICATION:
{json.dumps(generator_output, indent=2)}"""
    
    response = anthropic_client.messages.create(
        model="claude-opus-4-5-20251101",
        max_tokens=4096,
        system=EVALUATOR_SYSTEM_PROMPT,
        messages=[{"role": "user", "content": user_prompt}]
    )
    
    response_text = response.content[0].text
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        # If parsing fails, assume approval to avoid infinite loops
        return {
            "error": "Failed to parse evaluator response",
            "raw_response": response_text,
            "decision": "APPROVE",
            "evaluation": {"note": "Parse error, defaulting to approve"},
            "issues_found": [],
            "feedback": None,
            "final_classification": generator_output.get("classification", "Unknown")
        }

# ============================================================
# GENERATOR (gpt-5.1) - Retry with Feedback
# ============================================================

GENERATOR_RETRY_SYSTEM_PROMPT = """You are an expert news classifier revising a previous classification based on evaluator feedback.

You previously classified a news article, but an evaluator found issues with your reasoning. You must now reconsider your classification while addressing the specific feedback provided.

Your task:
1. Re-read the original article carefully
2. Consider the evaluator's feedback seriously
3. Re-examine the evidence, especially areas highlighted in the feedback
4. Either REVISE your classification or DEFEND your original decision with better reasoning

Important:
- You MUST address each issue raised in the feedback
- If you change your classification, explain what new insight led to the change
- If you maintain your original classification, provide stronger evidence and reasoning
- Your confidence should reflect the difficulty of this classification

Categories:
- World: International affairs, politics, government, diplomacy, conflicts
- Sports: Athletic events, teams, players, competitions, leagues
- Business: Companies, markets, economy, finance, corporate news
- Sci/Tech: Technology, science, software, research, innovation

Rules:
- Explicitly address each point in the evaluator's feedback
- Provide even more detailed reasoning than before
- Cite specific evidence from the article
- Return ONLY valid JSON, no additional text

Output format:
{
    "classification": "World" | "Sports" | "Business" | "Sci/Tech",
    "confidence": "high" | "medium" | "low",
    "key_signals": {
        "entities": ["list of entities"],
        "topics": ["main topics"],
        "domain_keywords": ["domain-specific terms"]
    },
    "reasoning": "Detailed explanation addressing evaluator concerns",
    "feedback_addressed": {
        "issue": "What the evaluator raised",
        "response": "How you addressed it"
    },
    "changed_classification": true | false,
    "change_explanation": "Why you changed or maintained your classification",
    "evidence": ["Specific evidence from the article"]
}"""

def run_generator_retry(article_text: str, previous_output: dict, feedback: str, iteration: int) -> dict:
    """Generator: Retry classification addressing evaluator feedback."""
    user_prompt = f"""Revise your classification based on evaluator feedback.

ORIGINAL ARTICLE:
---
{article_text}
---

YOUR PREVIOUS CLASSIFICATION:
{json.dumps(previous_output, indent=2)}

EVALUATOR'S FEEDBACK:
{feedback}

ITERATION: {iteration} of {MAX_ITERATIONS}

Please reconsider your classification, addressing the evaluator's concerns."""
    
    response = openai_client.chat.completions.create(
        model="gpt-5.1",
        max_completion_tokens=2048,
        messages=[
            {"role": "system", "content": GENERATOR_RETRY_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )
    
    response_text = response.choices[0].message.content
    cleaned = clean_json_response(response_text)
    
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        return {
            "error": "Failed to parse generator retry response",
            "raw_response": response_text,
            "classification": previous_output.get("classification", "Unknown"),
            "confidence": "low",
            "reasoning": "Parse error, keeping previous classification"
        }

# ============================================================
# EVALUATOR-OPTIMIZER PIPELINE
# ============================================================

def evaluator_optimizer_classify(article_text: str) -> dict:
    """Full evaluator-optimizer pipeline with iterative refinement."""
    
    iteration_history = []
    
    # Iteration 1: Initial classification
    iteration = 1
    generator_output = run_generator_initial(article_text)
    
    iteration_history.append({
        "iteration": iteration,
        "generator_output": generator_output,
        "evaluator_output": None  # Will be filled below
    })
    
    # Evaluation loop
    while iteration <= MAX_ITERATIONS:
        # Run evaluator
        evaluator_output = run_evaluator(article_text, generator_output)
        iteration_history[-1]["evaluator_output"] = evaluator_output
        
        # Check if approved
        if evaluator_output.get("decision") == "APPROVE":
            final_classification = evaluator_output.get("final_classification", 
                                                        generator_output.get("classification", "Unknown"))
            return {
                "final_classification": final_classification,
                "final_confidence": generator_output.get("confidence", "low"),
                "iterations_used": iteration,
                "approved": True,
                "iteration_history": iteration_history
            }
        
        # If rejected and more iterations available, retry
        iteration += 1
        if iteration <= MAX_ITERATIONS:
            feedback = evaluator_output.get("feedback", "Please reconsider your classification.")
            generator_output = run_generator_retry(article_text, generator_output, feedback, iteration)
            
            iteration_history.append({
                "iteration": iteration,
                "generator_output": generator_output,
                "evaluator_output": None
            })
    
    # Max iterations reached without approval
    # Use the last generator's classification
    final_classification = generator_output.get("classification", "Unknown")
    
    return {
        "final_classification": final_classification,
        "final_confidence": generator_output.get("confidence", "low"),
        "iterations_used": MAX_ITERATIONS,
        "approved": False,
        "iteration_history": iteration_history
    }

# ============================================================
# PROCESS ALL TEST DATA
# ============================================================

# Store evaluator-optimizer results
evaluator_optimizer_results = []

# Number of results to print
NUM_TO_PRINT = 3

print("=" * 70)
print("EVALUATOR-OPTIMIZER DESIGN PATTERN - Classification")
print("=" * 70)
print(f"\nProcessing {NUM_SAMPLES} samples using evaluator-optimizer pattern...")
print(f"Pipeline: Generator (gpt-5.1) ↔ Evaluator (Claude Opus 4.5)")
print(f"Max iterations: {MAX_ITERATIONS}\n")

# Progress tracking
PROGRESS_INTERVAL = 10

# Track iteration statistics
total_iterations = 0
approval_counts = {1: 0, 2: 0, 3: 0, "not_approved": 0}

for i in range(NUM_SAMPLES):
    sample = test_data[i]
    article_text = sample['text']
    ground_truth_label = sample['label']
    
    # Run the evaluator-optimizer pipeline
    result = evaluator_optimizer_classify(article_text)
    
    # Track statistics
    total_iterations += result["iterations_used"]
    if result["approved"]:
        approval_counts[result["iterations_used"]] += 1
    else:
        approval_counts["not_approved"] += 1
    
    # Store result with metadata
    evaluator_optimizer_results.append({
        "index": i,
        "original_text": article_text,
        "ground_truth_label": ground_truth_label,
        "ground_truth_name": label_names[ground_truth_label],
        "final_classification": result["final_classification"],
        "final_confidence": result["final_confidence"],
        "iterations_used": result["iterations_used"],
        "approved": result["approved"],
        "iteration_history": result["iteration_history"]
    })
    
    # Print progress
    if (i + 1) % PROGRESS_INTERVAL == 0 or (i + 1) == NUM_SAMPLES:
        avg_iter = total_iterations / (i + 1)
        print(f"  ✓ Processed {i + 1}/{NUM_SAMPLES} articles ({(i + 1) / NUM_SAMPLES * 100:.1f}%) | Avg iterations: {avg_iter:.2f}")

# ============================================================
# PRINT SAMPLE RESULTS
# ============================================================

print("\n" + "=" * 70)
print(f"EVALUATOR-OPTIMIZER RESULTS (showing first {NUM_TO_PRINT} of {len(evaluator_optimizer_results)})")
print("=" * 70)

for i in range(min(NUM_TO_PRINT, len(evaluator_optimizer_results))):
    result = evaluator_optimizer_results[i]
    
    print("─" * 70)
    print(f"Article {i + 1}")
    print(f"Ground Truth: {result['ground_truth_name']}")
    print("─" * 70)
    
    # Iteration summary
    print(f"\n🔄 Iterations: {result['iterations_used']} | Approved: {'✓' if result['approved'] else '✗'}")
    
    # Show iteration history
    for iter_info in result['iteration_history']:
        iter_num = iter_info['iteration']
        gen_out = iter_info['generator_output']
        eval_out = iter_info['evaluator_output']
        
        gen_class = gen_out.get('classification', 'N/A')
        gen_conf = gen_out.get('confidence', 'N/A')
        
        if eval_out:
            eval_decision = eval_out.get('decision', 'N/A')
            print(f"   Iter {iter_num}: Generator → {gen_class} ({gen_conf}) | Evaluator → {eval_decision}")
        else:
            print(f"   Iter {iter_num}: Generator → {gen_class} ({gen_conf}) | Evaluator → pending")
    
    # Final result
    predicted = result['final_classification']
    is_correct = predicted == result['ground_truth_name']
    status = "✓ CORRECT" if is_correct else "✗ INCORRECT"
    
    print(f"\n📊 Final Classification: {predicted}")
    print(f"   Confidence: {result['final_confidence']}")
    print(f"   Result: {status}")

# Print iteration statistics
print("\n" + "=" * 70)
print("ITERATION STATISTICS")
print("=" * 70)
print(f"\n  Approved on iteration 1: {approval_counts[1]} ({approval_counts[1]/NUM_SAMPLES*100:.1f}%)")
print(f"  Approved on iteration 2: {approval_counts[2]} ({approval_counts[2]/NUM_SAMPLES*100:.1f}%)")
print(f"  Approved on iteration 3: {approval_counts[3]} ({approval_counts[3]/NUM_SAMPLES*100:.1f}%)")
print(f"  Not approved (max iter): {approval_counts['not_approved']} ({approval_counts['not_approved']/NUM_SAMPLES*100:.1f}%)")
print(f"\n  Average iterations per article: {total_iterations/NUM_SAMPLES:.2f}")
print(f"  Total LLM calls: {sum(r['iterations_used'] * 2 for r in evaluator_optimizer_results)}")

print("\n" + "=" * 70)
print(f"Evaluator-Optimizer complete. {len(evaluator_optimizer_results)} results stored in 'evaluator_optimizer_results'")
print("Ready for evaluation.")
print("=" * 70)


EVALUATOR-OPTIMIZER DESIGN PATTERN - Classification

Processing 100 samples using evaluator-optimizer pattern...
Pipeline: Generator (gpt-5.1) ↔ Evaluator (Claude Opus 4.5)
Max iterations: 3

  ✓ Processed 10/100 articles (10.0%) | Avg iterations: 1.30
  ✓ Processed 20/100 articles (20.0%) | Avg iterations: 1.15
  ✓ Processed 30/100 articles (30.0%) | Avg iterations: 1.10
  ✓ Processed 40/100 articles (40.0%) | Avg iterations: 1.18
  ✓ Processed 50/100 articles (50.0%) | Avg iterations: 1.18
  ✓ Processed 60/100 articles (60.0%) | Avg iterations: 1.17
  ✓ Processed 70/100 articles (70.0%) | Avg iterations: 1.19
  ✓ Processed 80/100 articles (80.0%) | Avg iterations: 1.19
  ✓ Processed 90/100 articles (90.0%) | Avg iterations: 1.18
  ✓ Processed 100/100 articles (100.0%) | Avg iterations: 1.16

EVALUATOR-OPTIMIZER RESULTS (showing first 3 of 100)
──────────────────────────────────────────────────────────────────────
Article 1
Ground Truth: Business
──────────────────────────────────────

In [27]:
# ============================================================
# EVALUATION: Evaluator-Optimizer Design Pattern Performance
# ============================================================

# Extract predictions and ground truth from evaluator-optimizer results
eo_predictions = []
eo_ground_truths = []
eo_errors = []

for result in evaluator_optimizer_results:
    ground_truth = result['ground_truth_name']
    eo_ground_truths.append(ground_truth)
    
    # Get final classification
    predicted = result['final_classification']
    
    if predicted in CATEGORIES:
        eo_predictions.append(predicted)
    else:
        eo_predictions.append("ERROR")
        eo_errors.append(result['index'])

print("=" * 70)
print("EVALUATOR-OPTIMIZER DESIGN PATTERN - EVALUATION RESULTS")
print("=" * 70)

# Check for errors
if eo_errors:
    print(f"\n⚠️  Warning: {len(eo_errors)} samples had errors (indices: {eo_errors[:10]}{'...' if len(eo_errors) > 10 else ''})")
    print(f"   These are excluded from metrics calculation.\n")
    
    valid_eo_predictions = [p for p in eo_predictions if p != "ERROR"]
    valid_eo_ground_truths = [g for p, g in zip(eo_predictions, eo_ground_truths) if p != "ERROR"]
else:
    valid_eo_predictions = eo_predictions
    valid_eo_ground_truths = eo_ground_truths

# ============================================================
# Basic Metrics
# ============================================================

print("\n" + "─" * 70)
print("OVERALL METRICS")
print("─" * 70)

eo_accuracy = accuracy_score(valid_eo_ground_truths, valid_eo_predictions)
eo_precision_macro = precision_score(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES, average='macro', zero_division=0)
eo_recall_macro = recall_score(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES, average='macro', zero_division=0)
eo_f1_macro = f1_score(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES, average='macro', zero_division=0)

eo_precision_weighted = precision_score(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
eo_recall_weighted = recall_score(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES, average='weighted', zero_division=0)
eo_f1_weighted = f1_score(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES, average='weighted', zero_division=0)

print(f"\n  Total Samples:     {len(evaluator_optimizer_results)}")
print(f"  Valid Predictions: {len(valid_eo_predictions)}")
print(f"  Error Rate:        {len(eo_errors) / len(evaluator_optimizer_results) * 100:.1f}%")

print(f"\n  📊 Accuracy:        {eo_accuracy:.4f}  ({eo_accuracy * 100:.1f}%)")
print(f"\n  Macro Averages (treats all classes equally):")
print(f"     Precision:      {eo_precision_macro:.4f}")
print(f"     Recall:         {eo_recall_macro:.4f}")
print(f"     F1 Score:       {eo_f1_macro:.4f}")

print(f"\n  Weighted Averages (accounts for class imbalance):")
print(f"     Precision:      {eo_precision_weighted:.4f}")
print(f"     Recall:         {eo_recall_weighted:.4f}")
print(f"     F1 Score:       {eo_f1_weighted:.4f}")

# ============================================================
# Per-Class Metrics
# ============================================================

print("\n" + "─" * 70)
print("PER-CLASS METRICS")
print("─" * 70)

print("\n" + classification_report(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES, zero_division=0))

# ============================================================
# Confusion Matrix
# ============================================================

print("─" * 70)
print("CONFUSION MATRIX")
print("─" * 70)

eo_cm = confusion_matrix(valid_eo_ground_truths, valid_eo_predictions, labels=CATEGORIES)

print(f"\n{'Predicted →':>15}", end="")
for cat in CATEGORIES:
    print(f"{cat:>10}", end="")
print("\n" + "─" * 55)

print("Actual ↓")
for i, cat in enumerate(CATEGORIES):
    print(f"{cat:>15}", end="")
    for j in range(len(CATEGORIES)):
        print(f"{eo_cm[i][j]:>10}", end="")
    print()

# ============================================================
# Distribution Analysis
# ============================================================

print("\n" + "─" * 70)
print("DISTRIBUTION ANALYSIS")
print("─" * 70)

eo_gt_dist = Counter(valid_eo_ground_truths)
eo_pred_dist = Counter(valid_eo_predictions)

print(f"\n{'Category':<12} {'Ground Truth':>15} {'Predicted':>15} {'Difference':>12}")
print("─" * 55)
for cat in CATEGORIES:
    gt_count = eo_gt_dist.get(cat, 0)
    pred_count = eo_pred_dist.get(cat, 0)
    diff = pred_count - gt_count
    diff_str = f"+{diff}" if diff > 0 else str(diff)
    print(f"{cat:<12} {gt_count:>15} {pred_count:>15} {diff_str:>12}")

# ============================================================
# Iteration Analysis (Unique to Evaluator-Optimizer)
# ============================================================

print("\n" + "─" * 70)
print("ITERATION ANALYSIS")
print("─" * 70)

# Count by iterations used
iter_counts = Counter(r['iterations_used'] for r in evaluator_optimizer_results)
approval_status = Counter(r['approved'] for r in evaluator_optimizer_results)

print(f"\n  Iterations Distribution:")
print(f"  {'Iterations':>12} {'Count':>10} {'Percentage':>12}")
print("  " + "─" * 37)
for num_iter in sorted(iter_counts.keys()):
    count = iter_counts[num_iter]
    pct = count / len(evaluator_optimizer_results) * 100
    print(f"  {num_iter:>12} {count:>10} {pct:>11.1f}%")

print(f"\n  Approval Status:")
print(f"    Approved:     {approval_status[True]:>5} ({approval_status[True]/len(evaluator_optimizer_results)*100:.1f}%)")
print(f"    Not Approved: {approval_status[False]:>5} ({approval_status[False]/len(evaluator_optimizer_results)*100:.1f}%)")

avg_iterations = sum(r['iterations_used'] for r in evaluator_optimizer_results) / len(evaluator_optimizer_results)
print(f"\n  Average Iterations: {avg_iterations:.2f}")
print(f"  Total LLM Calls:    {sum(r['iterations_used'] * 2 for r in evaluator_optimizer_results)}")

# ============================================================
# Accuracy by Iteration (Unique to Evaluator-Optimizer)
# ============================================================

print("\n" + "─" * 70)
print("ACCURACY BY ITERATION COUNT")
print("─" * 70)

# Group results by iterations used and calculate accuracy for each group
iter_accuracy = {}
for num_iter in sorted(iter_counts.keys()):
    iter_results = [r for r in evaluator_optimizer_results if r['iterations_used'] == num_iter]
    correct = sum(1 for r in iter_results if r['final_classification'] == r['ground_truth_name'])
    total = len(iter_results)
    iter_accuracy[num_iter] = (correct, total)

print(f"\n  {'Iterations':>12} {'Correct':>10} {'Total':>10} {'Accuracy':>12}")
print("  " + "─" * 47)
for num_iter in sorted(iter_accuracy.keys()):
    correct, total = iter_accuracy[num_iter]
    acc = correct / total if total > 0 else 0
    print(f"  {num_iter:>12} {correct:>10} {total:>10} {acc:>11.1%}")

# ============================================================
# Approval vs Accuracy Analysis (Unique to Evaluator-Optimizer)
# ============================================================

print("\n" + "─" * 70)
print("APPROVAL VS ACCURACY ANALYSIS")
print("─" * 70)

# Accuracy for approved vs not approved
approved_results = [r for r in evaluator_optimizer_results if r['approved']]
not_approved_results = [r for r in evaluator_optimizer_results if not r['approved']]

approved_correct = sum(1 for r in approved_results if r['final_classification'] == r['ground_truth_name'])
not_approved_correct = sum(1 for r in not_approved_results if r['final_classification'] == r['ground_truth_name'])

print(f"\n  {'Status':<20} {'Correct':>10} {'Total':>10} {'Accuracy':>12}")
print("  " + "─" * 55)
if approved_results:
    approved_acc = approved_correct / len(approved_results)
    print(f"  {'Approved':<20} {approved_correct:>10} {len(approved_results):>10} {approved_acc:>11.1%}")
if not_approved_results:
    not_approved_acc = not_approved_correct / len(not_approved_results)
    print(f"  {'Not Approved (max)':<20} {not_approved_correct:>10} {len(not_approved_results):>10} {not_approved_acc:>11.1%}")

# ============================================================
# Self-Correction Analysis (Unique to Evaluator-Optimizer)
# ============================================================

print("\n" + "─" * 70)
print("SELF-CORRECTION ANALYSIS")
print("─" * 70)

# Analyze cases where classification changed during iterations
changed_classification = 0
change_improved = 0
change_worsened = 0
change_no_effect = 0

for result in evaluator_optimizer_results:
    if result['iterations_used'] > 1:
        # Get first and last classification
        first_iter = result['iteration_history'][0]
        last_iter = result['iteration_history'][-1]
        
        first_class = first_iter['generator_output'].get('classification', '')
        last_class = last_iter['generator_output'].get('classification', '')
        ground_truth = result['ground_truth_name']
        
        if first_class != last_class:
            changed_classification += 1
            first_correct = first_class == ground_truth
            last_correct = last_class == ground_truth
            
            if not first_correct and last_correct:
                change_improved += 1
            elif first_correct and not last_correct:
                change_worsened += 1
            else:
                change_no_effect += 1

multi_iter_count = sum(1 for r in evaluator_optimizer_results if r['iterations_used'] > 1)

print(f"\n  Articles requiring multiple iterations: {multi_iter_count}")
print(f"  Classification changed during iterations: {changed_classification}")

if changed_classification > 0:
    print(f"\n  When Classification Changed:")
    print(f"    Improved (wrong → correct):  {change_improved:>5} ({change_improved/changed_classification*100:.1f}%)")
    print(f"    Worsened (correct → wrong):  {change_worsened:>5} ({change_worsened/changed_classification*100:.1f}%)")
    print(f"    No Effect (wrong → wrong):   {change_no_effect:>5} ({change_no_effect/changed_classification*100:.1f}%)")
    
    net_improvement = change_improved - change_worsened
    print(f"\n  Net Improvement from Changes: {'+' if net_improvement >= 0 else ''}{net_improvement} articles")

# ============================================================
# First Iteration vs Final Accuracy (Unique to Evaluator-Optimizer)
# ============================================================

print("\n" + "─" * 70)
print("FIRST ITERATION VS FINAL ACCURACY")
print("─" * 70)

first_iter_correct = 0
final_correct = 0

for result in evaluator_optimizer_results:
    ground_truth = result['ground_truth_name']
    first_class = result['iteration_history'][0]['generator_output'].get('classification', '')
    final_class = result['final_classification']
    
    if first_class == ground_truth:
        first_iter_correct += 1
    if final_class == ground_truth:
        final_correct += 1

total = len(evaluator_optimizer_results)
print(f"\n  First Iteration Accuracy: {first_iter_correct}/{total} ({first_iter_correct/total*100:.1f}%)")
print(f"  Final Accuracy:           {final_correct}/{total} ({final_correct/total*100:.1f}%)")
print(f"\n  Improvement from Iterations: {'+' if final_correct >= first_iter_correct else ''}{final_correct - first_iter_correct} articles ({(final_correct - first_iter_correct)/total*100:+.1f}%)")

# ============================================================
# Confidence Analysis
# ============================================================

print("\n" + "─" * 70)
print("CONFIDENCE ANALYSIS")
print("─" * 70)

# Group by final confidence and calculate accuracy
confidence_accuracy = {"high": [0, 0], "medium": [0, 0], "low": [0, 0]}

for result in evaluator_optimizer_results:
    conf = result['final_confidence']
    predicted = result['final_classification']
    is_correct = predicted == result['ground_truth_name']
    
    if conf in confidence_accuracy:
        confidence_accuracy[conf][1] += 1
        if is_correct:
            confidence_accuracy[conf][0] += 1

print(f"\n{'Confidence':<15} {'Count':>10} {'Correct':>10} {'Accuracy':>12}")
print("─" * 50)
for conf_level in ['high', 'medium', 'low']:
    correct, total = confidence_accuracy[conf_level]
    if total > 0:
        acc = correct / total
        print(f"{conf_level:<15} {total:>10} {correct:>10} {acc:>11.1%}")
    else:
        print(f"{conf_level:<15} {0:>10} {0:>10} {'N/A':>12}")

# ============================================================
# Evaluator Effectiveness Analysis (Unique to Evaluator-Optimizer)
# ============================================================

print("\n" + "─" * 70)
print("EVALUATOR EFFECTIVENESS ANALYSIS")
print("─" * 70)

# Analyze evaluator decisions
total_evaluations = sum(r['iterations_used'] for r in evaluator_optimizer_results)
total_approvals = sum(1 for r in evaluator_optimizer_results if r['approved'])
total_rejections = total_evaluations - total_approvals

# Was rejection followed by improvement?
rejection_outcomes = {"improved": 0, "worsened": 0, "same_correct": 0, "same_wrong": 0}

for result in evaluator_optimizer_results:
    history = result['iteration_history']
    for i in range(len(history) - 1):
        current_iter = history[i]
        next_iter = history[i + 1]
        
        if current_iter['evaluator_output'] and current_iter['evaluator_output'].get('decision') == 'REJECT':
            current_class = current_iter['generator_output'].get('classification', '')
            next_class = next_iter['generator_output'].get('classification', '')
            ground_truth = result['ground_truth_name']
            
            current_correct = current_class == ground_truth
            next_correct = next_class == ground_truth
            
            if not current_correct and next_correct:
                rejection_outcomes["improved"] += 1
            elif current_correct and not next_correct:
                rejection_outcomes["worsened"] += 1
            elif current_correct and next_correct:
                rejection_outcomes["same_correct"] += 1
            else:
                rejection_outcomes["same_wrong"] += 1

print(f"\n  Total Evaluations: {total_evaluations}")
print(f"  Total Approvals:   {total_approvals}")
print(f"  Total Rejections:  {total_rejections}")

if total_rejections > 0:
    print(f"\n  After Rejection (what happened next):")
    print(f"    Improved (wrong → correct):     {rejection_outcomes['improved']:>5}")
    print(f"    Worsened (correct → wrong):     {rejection_outcomes['worsened']:>5}")
    print(f"    Same (stayed correct):          {rejection_outcomes['same_correct']:>5}")
    print(f"    Same (stayed wrong):            {rejection_outcomes['same_wrong']:>5}")
    
    if rejection_outcomes['improved'] + rejection_outcomes['worsened'] > 0:
        rejection_effectiveness = rejection_outcomes['improved'] / (rejection_outcomes['improved'] + rejection_outcomes['worsened'])
        print(f"\n  Rejection Effectiveness: {rejection_effectiveness:.1%}")
        print(f"  (When rejection led to change, how often was it an improvement)")

print("\n" + "=" * 70)
print("EVALUATOR-OPTIMIZER PATTERN EVALUATION COMPLETE")
print("=" * 70)


EVALUATOR-OPTIMIZER DESIGN PATTERN - EVALUATION RESULTS

──────────────────────────────────────────────────────────────────────
OVERALL METRICS
──────────────────────────────────────────────────────────────────────

  Total Samples:     100
  Valid Predictions: 100
  Error Rate:        0.0%

  📊 Accuracy:        0.8100  (81.0%)

  Macro Averages (treats all classes equally):
     Precision:      0.8245
     Recall:         0.8653
     F1 Score:       0.8041

  Weighted Averages (accounts for class imbalance):
     Precision:      0.8971
     Recall:         0.8100
     F1 Score:       0.8224

──────────────────────────────────────────────────────────────────────
PER-CLASS METRICS
──────────────────────────────────────────────────────────────────────

              precision    recall  f1-score   support

       World       1.00      0.87      0.93        30
      Sports       0.91      1.00      0.95        21
    Business       0.43      1.00      0.60        12
    Sci/Tech       0.9

## Summary: Comparison of Agentic Workflow Design Patterns

### Overview

We evaluated 5 different agentic workflow design patterns for classifying news articles from the AG News dataset into 4 categories: World, Sports, Business, and Sci/Tech.

| # | Design Pattern | LLM Calls/Article | Models Used |
|---|----------------|-------------------|-------------|
| 1 | **Prompt Chaining** | 3 (sequential) | Claude Sonnet 4 → GPT-5.1 → Claude Haiku 4.5 |
| 2 | **Routing** | 2 (router + expert) | GPT-5-mini (router) + 3 specialized experts |
| 3 | **Parallelization** | 4 (parallel) | 4× Claude Haiku 4.5 detectors + code aggregator |
| 4 | **Orchestrator-Worker** | 6 (1+4+1) | GPT-5-mini + 4× Claude Haiku 4.5 + GPT-5.1 |
| 5 | **Evaluator-Optimizer** | 2-6 (iterative) | GPT-5.1 (generator) + Claude Opus 4.5 (evaluator) |

---

### Performance Comparison

#### Overall Metrics

| Design Pattern | Accuracy | Precision (Macro) | Recall (Macro) | F1 (Macro) |
|----------------|----------|-------------------|----------------|------------|
| **1. Prompt Chaining** | **83.0%** | 0.827 | **0.879** | **0.822** |
| 2. Routing | 82.0% | 0.809 | 0.844 | 0.806 |
| 5. Evaluator-Optimizer | 81.0% | **0.825** | 0.865 | 0.804 |
| 4. Orchestrator-Worker | 80.0% | 0.813 | 0.855 | 0.792 |
| 3. Parallelization | 72.0% | 0.718 | 0.772 | 0.717 |

#### Per-Class F1 Scores

| Design Pattern | World | Sports | Business | Sci/Tech |
|----------------|-------|--------|----------|----------|
| **1. Prompt Chaining** | 0.88 | **0.95** | **0.67** | 0.79 |
| 2. Routing | 0.89 | 0.88 | 0.65 | 0.81 |
| 5. Evaluator-Optimizer | **0.93** | **0.95** | 0.60 | 0.73 |
| 4. Orchestrator-Worker | 0.90 | 0.93 | 0.62 | 0.72 |
| 3. Parallelization | 0.78 | 0.75 | **0.67** | 0.68 |

---

### Key Observations

#### 1. Common Challenge: Business vs Sci/Tech Confusion

All 5 workflows struggle with the same classification boundary:

| Pattern | Business Precision | Business Over-prediction | Sci/Tech Under-prediction |
|---------|-------------------|--------------------------|---------------------------|
| Prompt Chaining | 0.50 | +12 | -13 |
| Routing | 0.50 | +10 | -10 |
| Orchestrator-Worker | 0.44 | +15 | -16 |
| Evaluator-Optimizer | 0.43 | +16 | -14 |
| Parallelization | 0.52 | +9 | -12 |

**Root Cause:** Many Sci/Tech articles discuss technology companies, which triggers "Business" signals. The models tend to classify tech company news as Business when the primary focus is actually on technology/products.

#### 2. Workflow-Specific Insights

**Prompt Chaining (83.0%)**
- Sequential pipeline works well for structured analysis
- Step 1 (extraction) + Step 2 (analysis) + Step 3 (classification) provides good decomposition
- 97% high confidence predictions with 82.5% accuracy

**Routing (82.0%)**
- `science_world_expert` achieved **96.4% accuracy** - excellent for World/Sci/Tech
- `business_expert` achieved only **50% accuracy** - the bottleneck
- Router correctly identified domain 77-96% of the time

**Parallelization (72.0%)**
- Lowest accuracy due to 15% fallback rate (no detector claimed)
- Individual detectors had high accuracy when they claimed (World: 100%, Sports: 90.5%, Sci/Tech: 95.7%)
- Business detector only 50% accurate
- Confidence voting aggregation struggled with ambiguous cases

**Orchestrator-Worker (80.0%)**
- Orchestrator's first choice was correct only **52%** of the time
- Workers + Synthesizer corrected **30%** of orchestrator errors
- But also introduced errors in **14%** of cases
- Complex pipeline with marginal benefit over simpler approaches

**Evaluator-Optimizer (81.0%)**
- **90%** approved on first iteration - evaluator rarely rejected
- Self-correction **hurt** performance: first iteration 82% → final 81%
- When classification changed, it worsened more often than improved (66.7% vs 33.3%)
- Rejection effectiveness only 44.4%

---

### Cost-Efficiency Analysis

| Design Pattern | LLM Calls (100 articles) | Accuracy | Calls per 1% Accuracy |
|----------------|--------------------------|----------|----------------------|
| **Routing** | 200 | 82.0% | **2.4** |
| **Prompt Chaining** | 300 | 83.0% | 3.6 |
| Parallelization | 400 | 72.0% | 5.6 |
| Orchestrator-Worker | 600 | 80.0% | 7.5 |
| Evaluator-Optimizer | 232 (avg) | 81.0% | 2.9 |

---

### Recommendation: Best Workflow for AG News Classification

## 🏆 Winner: **Prompt Chaining** (83.0% Accuracy)

### Reasons for Selection:

1. **Highest Overall Accuracy (83.0%)**
   - Outperforms all other patterns by 1-11 percentage points
   - Best macro F1 score (0.822)

2. **Excellent Sports Classification (95% F1)**
   - Perfect recall (1.00) for Sports category
   - Only 2 misclassifications

3. **Structured Decomposition Works Well**
   - Step 1: Entity & Keyword Extraction (Claude Sonnet 4)
   - Step 2: Domain Signal Analysis (GPT-5.1)
   - Step 3: Final Classification (Claude Haiku 4.5)
   - Each step builds on the previous, providing interpretable intermediate results

4. **Consistent High Confidence**
   - 97% of predictions were high confidence
   - High confidence predictions were 82.5% accurate

5. **Reasonable Cost**
   - 3 LLM calls per article is moderate
   - Better accuracy than patterns using more calls (Orchestrator-Worker: 6 calls, 80%)

6. **Simplicity**
   - Linear, predictable pipeline
   - Easy to debug (can inspect each step's output)
   - No complex coordination logic

### Runner-Up: **Routing** (82.0% Accuracy)

If cost is a primary concern, Routing is the most efficient:
- Only 2 LLM calls per article
- 82% accuracy
- Best cost-efficiency ratio (2.4 calls per 1% accuracy)

The `science_world_expert` (96.4% accuracy) demonstrates that specialized agents can excel in their domains.

### What to Avoid for This Dataset

**Parallelization** (72.0%) performed worst because:
- 15% of articles had no detector claim them (fallback to uncertain prediction)
- Simple confidence voting couldn't resolve ambiguous Business/Sci/Tech cases
- Binary detection approach loses nuance in edge cases

**Evaluator-Optimizer** showed that iterative refinement doesn't always help:
- Self-correction actually decreased accuracy (-1%)
- The evaluator rejected good classifications, leading to worse revisions
- High computational cost (potentially 6 calls) with no accuracy benefit

---

### Future Improvements

To address the Business/Sci/Tech confusion across all patterns:

1. **Better prompting**: Explicitly instruct models that tech company news about products/features is Sci/Tech, not Business
2. **Two-stage classification**: First classify Business vs Non-Business, then subdivide
3. **Example-based few-shot**: Provide examples of Business (financials) vs Sci/Tech (products) for tech companies
4. **Ensemble approach**: Combine Prompt Chaining with Routing for ambiguous cases
